<a href="https://colab.research.google.com/github/Ruochenge/learning_test/blob/main/Copy_of_Vixd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Valley Indexing**

![](https://drive.google.com/uc?export=view&id=1B7jy9JW2v-ahAwnyHSunro-pdSP5Lf1t)


In [ ]:
#        /\\                                                          /\\
#       /  \\             ^        ^^^            ^^^^               /  \\
#      /    \\           / \\     /   \\        /      \\           /    \\
#     /      \\         /   \\   /     \\      /        \\         /      \\
# ___/        \\_______/     \\_/       \\____/          \\_______/        \\___

#Packages and mounting drive

In [ ]:
#@markdown Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

Mounted at /content/drive


In [ ]:
#@markdown Packages

# Data manipulation:
import pandas as pd
import random  # Import the random module
from itertools import islice
#import sketch
from collections import  defaultdict, Counter
from prettytable import PrettyTable

# Debugging:
import traceback

# File I/O and path handling:
import os
import copy, os
import glob
import re
import requests
import json
from datetime import datetime
from typing import Dict, Any
from pathlib import Path


#Logging
import logging
from typing import List, Set, Dict, Tuple

# Setup basic logging for informational purposes
logging.basicConfig(level=logging.INFO)


# Numerical analysis and statistics:
import numpy as np
from scipy import stats
from statistics import mode, multimode  # Consider removing if unused
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


#Visualisation
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import plotly.express as px
import plotly.graph_objects as go

# Third-party modules (Commented out modules can be imported as needed):
#from Bio import SeqIO  # Only import if used
# from icecream import ic  # Import on demand if needed

##Variables from Experiments

Please edit the [GoogleSheet: Vidx data dictionary](https://docs.google.com/spreadsheets/d/1JYAoajIlxGgYo9oRjupLfjAJlrrcTuDAsvsLLiYdIg4/edit?usp=sharing') to change these values
Use [Vixd_variables](https://colab.research.google.com/drive/1XatrJbbcX1J8sFgmLBNc9x82wo3v4GyK#scrollTo=7UXOhIXbU9Hb) to update the [sheet](https://docs.google.com/spreadsheets/d/1JYAoajIlxGgYo9oRjupLfjAJlrrcTuDAsvsLLiYdIg4/edit?usp=sharing) - these sheet is not updated in this code-block

### Variables for Samples are:-

Filename | Source | Sample_Type | Species | Sample | Prep | Reduction | Alkylation | Enzyme | Enzyme_supplier | Enzyme_concentration | Enzyme_digestion_temperature | Enzyme_digestion_time | Publication |


### Varialbes for Sequences are:-

Index | GCN | Spp | Species | Offset | Sequence |

### Study Names


1.   Plasters
2.   Cow
1.   Mice
2.   Next

Data is saved to
 /content/drive/MyDrive/Colab_Notebooks/pFind/{STUDY_NAME}/{STUDY_NAME}_{year-date-hour}_vidx.csv



In [ ]:
# MS_METHOD = 'pFind'
# STUDY_NAME = 'plaster'
# BASE_PATH = f'/content/drive/MyDrive/Colab_Notebooks/MS_output/{MS_METHOD}/{STUDY_NAME}'

#-----------------------------Importing the dictionaries
#name = 'plaster'
file_directory_base = '/content/drive/MyDrive/Colab_Notebooks/Dictionaries'



#@title Import saved file
#Data saved to /content/drive/MyDrive/Colab_Notebooks/MS_output/pFind/plaster/plaster_2024-05-30_21_vidx.csv
#filename = "Mice_2024-05-02_10_vidx.csv"
#filename = "Cow_2024-05-23_20_vidx.csv"

#@markdown **Study**
STUDY_NAME = 'plaster' #@param {type:"string"}
#@markdown **Sequences**
FASTA_NAME = 'plaster' #@param {type:"string"}
#@markdown **Method**
MS_METHOD = 'pFind' #@param {type:"string"}
BASE_PATH = f'/content/drive/MyDrive/Colab_Notebooks/MS_output/{MS_METHOD}/{STUDY_NAME}'

file_directory_base = '/content/drive/MyDrive/Colab_Notebooks/Dictionaries'


# df = pd.read_csv(f"{BASE_PATH}/{filename}")

# Other Varibles

#Functions

##Dictionary Functions

In [ ]:
def load_json_as_dict(file_path):
    """
    Loads a JSON file and returns its content as a dictionary.

    Args:
        file_path: The path to the JSON file.

    Returns:
        A dictionary containing the JSON file content.

    Raises:
        FileNotFoundError: If the JSON file is not found.
    """
    try:
        with open(file_path, 'r') as json_file:
            return json.load(json_file)
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
        return {}

def process_json_data(file_directory_base, sequences_name, sample_name):
    """
    Processes JSON files for sequences and renames based on the specified names.

    Args:
        file_directory_base: The base directory for the JSON files.
        sequences_name: The name used to identify the sequences JSON file.
        rename_name: The name used to identify the rename JSON file.

    Returns:
        A tuple containing two dictionaries: sequences_dict and rename_dict.

    Raises:
        FileNotFoundError: If any of the JSON files are not found.
    """

    sequences_file_path = os.path.join(file_directory_base, "FASTA_dicts", f"{sequences_name}_sequences.json")
    rename_file_path = os.path.join(file_directory_base, "Samples", f"{sample_name}_samples.json")

    # Load the JSON files
    sequences_dict = load_json_as_dict(sequences_file_path)
    rename_dict = load_json_as_dict(rename_file_path)

    # Convert the offset values to integers in sequences_dict
    for key, value in sequences_dict.items():
        value['offset'] = int(value['offset'])

    return sequences_dict, rename_dict

def find_entries_by_field(sequences_dict, field_name, target_value):
    """
    Finds all entries in a dictionary where a specified field equals a specific value.

    Parameters:
        sequences_dict (dict): The dictionary to search through. Assumes each value is itself a dictionary.
        field_name (str): The name of the field to search for in the dictionary's values.
        target_value (str): The value to search for in the specified field.

    Returns:
        dict: A dictionary containing all entries where the specified field matches the target value.
    """
    matching_entries = {}

    for key, value in sequences_dict.items():
        if value.get(field_name) == target_value:
            matching_entries[key] = value

    return matching_entries

### Directory functions

### Dictionary Manipulations

In [ ]:
# def levenshtein_distance(s1, s2):
#     """
#     Calculate the Levenshtein distance between two strings.
#     The Levenshtein distance measures the minimum number of single-character edits
#     (insertions, deletions, or substitutions) required to change one word into the other.

#     Parameters:
#     - s1 (str): The first string to compare.
#     - s2 (str): The second string to compare.

#     Returns:
#     - int: The Levenshtein distance between the two strings.
#     """
#     if len(s1) < len(s2):
#         return levenshtein_distance(s2, s1)

#     if len(s2) == 0:
#         return len(s1)

#     previous_row = range(len(s2) + 1)
#     for i, c1 in enumerate(s1):
#         current_row = [i + 1]
#         for j, c2 in enumerate(s2):
#             insertions = previous_row[j + 1] + 1
#             deletions = current_row[j] + 1
#             substitutions = previous_row[j] + (c1 != c2)
#             current_row.append(min(insertions, deletions, substitutions))
#         previous_row = current_row

#     return previous_row[-1]



# def find_match_and_update(row, sequence_position_dict, NCP_peptide_positions_dict):
#     """
#     Update the row based on matches found in sequence_position_dict and NCP_peptide_positions_dict.

#     Parameters:
#     - row: A pandas Series representing a row in a DataFrame.
#     - sequence_position_dict: A dictionary containing matched sequences with gene data.
#     - NCP_peptide_positions_dict: A dictionary containing NCP peptide positions.

#     Returns:
#     - Updated row with match data from sequence_position_dict or NCP_peptide_positions_dict.
#     """
#     sequence = row['Sequence']
#     # Check for a match in the sequence_position_dict
#     if sequence in sequence_position_dict:
#         # Use the extract_gene_identifier function to simplify gene name extraction
#         gene = extract_gene_identifier(sequence_position_dict[sequence]['gene'])
#         # Update 'GNC&spp' with the extracted gene, append if already exists
#         row['GNC&spp'] = f"{gene}" if pd.isnull(row['GNC&spp']) else f"{row['GNC&spp']},{gene}"

#     if sequence in NCP_peptide_positions_dict:
#         gene = extract_gene_identifier(NCP_peptide_positions_dict[sequence]['gene'])
#         # Append or set the gene identifier for NCP matches
#         row['GNC&spp'] = f"{gene}" if pd.isnull(row['GNC&spp']) else f"{row['GNC&spp']},{gene}"

#     return row



###Find positions and create Sequence_postion dictionary Functions

In [ ]:
def match_peptides(peptides, sequences):
    """
    Match peptides exactly against sequences in the provided dictionary and record the match details,
    including an indication of zero mismatches for exact matches.

    Parameters:
    - peptides (list): List of peptide sequences to match.
    - sequences (dict): Dictionary of gene sequences with details, including GNC (Gene Name Convention).
    """
    global sequence_position_dict, matched_peptides
    for peptide in peptides:
        for gene, details in sequences.items():
            sequence = details['sequence']
            offset = details['offset']
            position = sequence.find(peptide)
            if position != -1:  # Peptide found in the sequence
                corrected_position = position + offset  # Adjust for sequence offset
                # Record match details including GNC, and indicate zero mismatches for an exact match
                sequence_position_dict[peptide] = {
                    'gene': gene,
                    'GNC': details['GNC'],  # Include the Gene Name Convention (GNC)
                    'species': details['species'],  # Include species information
                    'position': corrected_position,  # Include the corrected position within the gene sequence
                    'length': len(peptide),  # Include the length of the matched peptide
                    'mismatches': 0  # Indicate that there are zero mismatches for exact matches
                }
                matched_peptides.add(peptide)
                break


def is_one_amino_acid_difference(peptide1, peptide2):
    """
    Check if two peptides of the same length differ by exactly one amino acid.
    """
    if len(peptide1) != len(peptide2):
        return False

    diff_count = sum(1 for a, b in zip(peptide1, peptide2) if a != b)
    return diff_count == 1

def is_within_mismatch_limit(peptide1, peptide2, max_mismatches):
    """
    Check if two peptides of the same length differ by a number of amino acids up to the max_mismatches limit.
    Returns a tuple of (bool, int) indicating whether the peptides are within the mismatch limit and the number of mismatches.
    """
    if len(peptide1) != len(peptide2):
        return False, 0  # No match and zero mismatches in case of different lengths

    diff_count = sum(1 for a, b in zip(peptide1, peptide2) if a != b)
    return diff_count <= max_mismatches, diff_count


def match_unmatched_peptides(remaining_unmatched_peptides, sequence_position_dict):
    """
    Attempt to match remaining unmatched peptides with those in sequence_position_dict,
    allowing for one amino acid substitution.
    """
    newly_matched_peptides = set()

    for unmatched_peptide in remaining_unmatched_peptides:
        for matched_peptide in sequence_position_dict.keys():
            if is_one_amino_acid_difference(unmatched_peptide, matched_peptide):
                newly_matched_peptides.add(unmatched_peptide)
                break

    remaining_unmatched_peptides -= newly_matched_peptides
    return remaining_unmatched_peptides

def iterative_match_unmatched_peptides(unmatched_peptides, sequence_position_dict, sequences, iterations=1):
    """
    Iteratively match remaining unmatched peptides with those in sequence_position_dict,
    allowing for an increasing number of amino acid substitutions up to the specified iterations.
    Each newly matched peptide is added to sequence_position_dict with details, including mismatch count and additional gene information such as GNC.
    """
    for i in range(1, iterations + 1):  # Allow mismatches up to 'iterations'
        newly_matched_peptides = {}

        for unmatched_peptide in unmatched_peptides:
            for matched_peptide, details in sequence_position_dict.items():
                within_limit, mismatches = is_within_mismatch_limit(unmatched_peptide, matched_peptide, i)
                if within_limit:
                    # Retrieve gene name from the existing match details to access GNC and potentially other details from sequences dictionary
                    gene_name = details['gene']
                    original_details = sequences[gene_name]  # Access original gene details from sequences

                    # Update with mismatch count and ensure GNC (and potentially other details) are included
                    updated_details = {
                        **details,
                        'mismatches': mismatches,
                        'GNC': original_details.get('GNC', 'Unknown'),  # Include GNC, defaulting to 'Unknown' if not found
                        # Include any other desired details from original_details here
                    }
                    newly_matched_peptides[unmatched_peptide] = updated_details
                    break

        # Add newly matched peptides to sequence_position_dict and remove from unmatched_peptides
        for peptide, details in newly_matched_peptides.items():
            sequence_position_dict[peptide] = details
            unmatched_peptides.remove(peptide)

        # Optional: Report progress after each iteration
        print(f"Iteration {i}: Found {len(newly_matched_peptides)} new matches.")

    return unmatched_peptides


def is_within_mismatch_limit(unmatched_peptide, matched_peptide, limit):
    """
    Placeholder function to evaluate if the unmatched peptide is within the allowable limit
    of mismatches compared to an already matched peptide.

    Parameters:
    - unmatched_peptide (str): The peptide sequence being matched.
    - matched_peptide (str): The peptide sequence already matched.
    - limit (int): The maximum allowed mismatches.

    Returns:
    Tuple[bool, int]: A tuple containing a boolean indicating if within limit and the count of mismatches.
    """
    # Placeholder logic - replace with actual mismatch counting logic
    mismatches = 0  # Example mismatch count
    within_limit = mismatches <= limit
    return within_limit, mismatches


# def iterative_match_unmatched_peptides(unmatched_peptides, sequence_position_dict, sequences, iterations=1):
#     """
#     Iteratively match remaining unmatched peptides with those in sequence_position_dict,
#     allowing for an increasing number of amino acid substitutions up to the specified iterations.
#     Each newly matched peptide is added to sequence_position_dict with details, including mismatch count.
#     """
#     for i in range(1, iterations + 1):  # Allow mismatches up to 'iterations'
#         newly_matched_peptides = {}

#         for unmatched_peptide in unmatched_peptides:
#             for matched_peptide, details in sequence_position_dict.items():
#                 within_limit, mismatches = is_within_mismatch_limit(unmatched_peptide, matched_peptide, i)
#                 if within_limit:
#                     # Assume details is a structure that can be directly updated or augmented
#                     # Here we augment with the 'mismatches' information
#                     updated_details = {**details, 'mismatches': mismatches}  # Update with mismatch count
#                     newly_matched_peptides[unmatched_peptide] = updated_details
#                     break

#         # Add newly matched peptides to sequence_position_dict and remove from unmatched_peptides
#         for peptide, details in newly_matched_peptides.items():
#             sequence_position_dict[peptide] = details
#             unmatched_peptides.remove(peptide)

#         # Optional: Report progress after each iteration
#         print(f"Iteration {i}: Found {len(newly_matched_peptides)} new matches.")

#     return unmatched_peptides


def annotate_row(row, positions_dict):
    """
    Annotate a single row with gene and start position based on a given positions dictionary.

    Parameters:
    - row: A row of a pandas DataFrame.
    - positions_dict: A dictionary with sequences as keys and dictionaries with 'gene' and 'position' as values.

    Returns:
    - row: The modified row with 'GNC&spp' and 'start_position' annotated if the sequence is found in positions_dict.
    """
    sequence = row['Sequence']
    if sequence in positions_dict:
        row['GNC&spp'] = positions_dict[sequence].get('gene')
        row['GNC'] = positions_dict[sequence].get('GNC')
        row['start_position'] = positions_dict[sequence].get('position')
        row['mismatch_to_sequence_dict'] = positions_dict[sequence].get('mismatches')
    return row


###Dataframe manipulation

####Identify PTMS and Parse peptide

In [ ]:
def parse_modifications(mod_str):
    """
    Parses a string containing modification data into a structured list.

    This function extracts the position, type of modification, and the amino acid
    involved from a given modification string. The expected format of the modification
    string is "position,Modification[AminoAcid];".

    Parameters:
        mod_str (str): A string representing modifications, or NaN/None for no modifications.

    Returns:
        list of tuples: A list where each tuple contains (position, modification type, amino acid).
    """
    # Check if mod_str is not a string (e.g., NaN represented as float in pandas)
    if not isinstance(mod_str, str):
        return []

    parsed_mods = []
    mods = mod_str.strip(';').split(';')
    for mod in mods:
        parts = mod.split(',')
        if len(parts) == 2:
            pos, mod_detail = parts
            try:
                mod_type, aa = mod_detail.split('[')
                aa = aa.rstrip(']')
                parsed_mods.append((int(pos), mod_type, aa))
            except ValueError:
                # Skip modification if parsing fails
                continue

    return parsed_mods

def parse_peptide(peptide):
    """
    Parses peptide strings to identify amino acids and extract them along with their positions.

    :param peptide: A string representation of a peptide.
    :return: A list of tuples, each representing an amino acid and its position in the peptide.
    """
    # A simple pattern that matches each amino acid in the sequence.
    aa_pattern = re.compile(r'[A-Z]')
    parsed_peptides = []
    pos = 1  # Position initialization

    for match in aa_pattern.finditer(peptide):
        aa = match.group()  # Extract the amino acid
        parsed_peptides.append((aa, pos))
        pos += 1  # Increment position for each amino acid

    return parsed_peptides

In [ ]:
def parse_and_update_df(sequence, sequence_position_dict): #combined_df
    """
    Check if the sequence matches an entry in the sequence_position_dict and return gene and position.

    Parameters:
    - sequence (str): The peptide sequence to check.
    - sequence_position_dict (dict): Dictionary with peptide sequences as keys and details as values.

    Returns:
    - tuple: (gene, position) if match is found; (None, None) otherwise.
    """
    if sequence in sequence_position_dict:
        # Extract gene and position from the matched entry
        gene = sequence_position_dict[sequence]['gene']
        position = sequence_position_dict[sequence]['start_position']
        return gene, position
    else:
        # Return None or placeholders if no match is found
        return None, None

def update_NCP_matches(row, NCP_peptide_positions_dict):
    """
    Update the row with NCP peptide positions if the sequence matches an NCP peptide.

    Parameters:
    - row: A pandas Series representing a row in a DataFrame.
    - NCP_peptide_positions_dict: A dictionary containing NCP peptide positions.

    Returns:
    - Updated row with NCP peptide match data.
    """
    sequence = row['Sequence']
    if sequence in NCP_peptide_positions_dict:
        match_data = NCP_peptide_positions_dict[sequence]
        # Check and process the gene name specifically if it exists
        if 'gene' in match_data:
            gene = match_data['gene'].split('-', 1)[0]  # Extract gene name before the underscore
            row['GNC&spp'] = gene  # Assuming you're updating to a gene-specific column
        else:
            for key, value in match_data.items():
                row[key] = value
    return row


def extract_protein_frequencies_and_associations(df, column_name):
    """
    Extracts the frequency of each protein ID and the five most common proteins
    listed with each ID from a specified column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing protein data.
    column_name (str): The name of the column with protein IDs separated by '/'.

    Returns:
    pd.DataFrame: A DataFrame with each protein's frequency and its five most common associations.
    """
    # Parse the specified column to create a list of proteins per row
    #df['Proteins_List'] = df[column_name].apply(lambda x: x.split('/'))
    df['Proteins_List'] = df[column_name].apply(lambda x: [p.split('&') for p in x.split('/')])

    # Initialize a dictionary to keep count of each protein's frequency
    protein_counts = Counter()
    # Initialize a dictionary to keep track of proteins listed together
    protein_associations = defaultdict(lambda: Counter())

    # Count frequencies and associations
    for proteins in df['Proteins_List']:
        for protein in proteins:
            protein_counts[protein] += 1
            # Add counts for other proteins in the list to the associations
            for associated_protein in proteins:
                if associated_protein != protein:
                    protein_associations[protein][associated_protein] += 1

    # Prepare the final report data
    report_data = [{
        'Protein': protein,
        'Frequency': count,
        'Most_Common_Associations': [assoc for assoc, _ in protein_associations[protein].most_common(5)]
    } for protein, count in protein_counts.items()]

    # Convert report_data to a DataFrame
    report_df = pd.DataFrame(report_data)

    return report_df

def expand_df_to(df):
    """
    Expands a DataFrame by processing each peptide sequence and constructing detailed rows
    for each amino acid within the peptide sequences, including modification information and
    simplified sample name.

    Parameters:
        df (pd.DataFrame): A pandas DataFrame containing peptide information.

    Returns:
        pd.DataFrame: An expanded DataFrame with detailed amino acid and modification information.
    """
    expanded_rows = []
    # for _, row in df.iterrows():
    #     try:
    #         start_position = float(row['start_position'])
    #         if not start_position.is_integer():
    #             continue  # Skip if start_position is not a whole number
    #         start_position = int(start_position)
    #     except (ValueError, TypeError):
    #         continue  # Skip if start_position is not convertible to float
    for _, row in df.iterrows():
        try:
            # Directly convert start_position to an integer
            start_position = int(row['start_position'])
        except ValueError:
            # Skip this row if start_position is not a valid integer
            continue

        modifications = parse_modifications(row.get('Modification', ''))
        mod_dict = {mod[0]: {'type': mod[1], 'aa': mod[2]} for mod in modifications}
        amino_acids = parse_peptide(row['Sequence'])

        gnc_spp_split = row.get('GNC&spp', '').split('-', maxsplit=1)
        if len(gnc_spp_split) != 2:
            continue  # Skip if the split does not result in two parts

        GNC, spp = gnc_spp_split
        simplified_file_name = "_".join(row.get('File_Name', '').split("_")[:2])

        for aa, pos in amino_acids:
            mod_info = mod_dict.get(pos, {'type': '', 'aa': ''})
            expanded_row = {
                'id': None,
                'sample_name': row['sample_name'],
                'GNC': GNC,  #Genome Nomenclature
                'spp': spp,  #Species
                'aa': aa,    #amino acid
                'position': pos - 1 + start_position, #-1 as python counts from 0
                'ptm': mod_info['type'], # infomation on any post translational modification
                'confidence': None,  #confidence score for the amino acid
                'spectraId': row['Scan_No'], #the ID of the MS/MS scan number
                'mz': row['Exp.MH+'],  #mass of the peptide detected (m/z, mass over charge)
                'z': row['Charge'],   #charge
                'pepMass': row['Calc.MH+'],  #mass
                'err': row['Mass_Shift(Exp.-Calc.)'],  #error between the observed and calculated mass
                'score': row['Final_Score'], # score for the peptide - note this is difference from the 'confidence' which is for the individual amion acid
                'scanNum': row['Scan_No'], #scan number
                'peptide_gene_name': row['Proteins'], #full information about the ID from mapping algorithm (e.g. MaxQuant, pFind, Orthrus)
                'RT': None, #Retention time
                'ppm': None, #error in ppm
            }
            expanded_rows.append(expanded_row)

    return pd.DataFrame(expanded_rows)

In [ ]:
def get_top_n(df, column, n=3):
    """
    Returns the top N items from a dataframe based on the count of a specified column.
    """
    return df[column].value_counts().nlargest(n).reset_index().rename(columns={'index': column, column: f'n_{column}'})

def get_scores_info(df, aa, prefix):
    """
    Aggregate score information for a given amino acid.
    """
    filtered_df = df[df['aa'] == aa]
    top_scores = filtered_df.nlargest(3, 'score')[['score', 'spectraId']]
    score_info = {
        f'{prefix}_score_mean': filtered_df['score'].mean(),
        f'{prefix}_score_max': filtered_df['score'].max(),
        f'{prefix}_score_SD': filtered_df['score'].std(),
    }
    for i, (score, spectraId) in enumerate(zip(top_scores['score'], top_scores['spectraId']), start=1):
        score_info.update({
            f'{prefix}_score{i}': score,
            f'{prefix}_spectraId{i}': spectraId,
        })
    return score_info

def get_ptm_info(df, aa, prefix):
    """
    Aggregate PTM information for a given amino acid.
    """
    # Filtering for non-null PTMs
    filtered_df = df[(df['aa'] == aa) & (df['ptm'].notnull())]
    if filtered_df.empty:
        return {f'{prefix}_ptm': None}  # Return None or appropriate value if no PTMs are found

    # Assuming PTM data is categorical and can be directly counted
    top_ptms = filtered_df['ptm'].value_counts().nlargest(2).index
    ptm_info = {}
    for i, ptm in enumerate(top_ptms, start=1):
        ptm_group = filtered_df[filtered_df['ptm'] == ptm]
        ptm_info.update({
            f'{prefix}_ptm{i}': ptm,
            f'{prefix}_ptm{i}_count': ptm_group.shape[0],
            f'{prefix}_ptm{i}_percentage': (ptm_group.shape[0] / filtered_df.shape[0]) * 100,
            # Additional PTM-specific stats can be added here
        })

    return ptm_info

def aggregate_info(group):
    """
    Aggregates detailed information for each group of data.
    """
    top_aas = get_top_n(group, 'aa')
    aggregated_data = []

    for _, row in top_aas.iterrows():
        aa = row['aa']
        aa_info = get_scores_info(group, aa, 'aa1')
        ptm_info = get_ptm_info(group, aa, 'aa1')

        record = {
            'aa': aa,
            'n_aa': row['n_aa'],
            **aa_info,
            **ptm_info
        }

        aggregated_data.append(record)

    return pd.DataFrame(aggregated_data)

### Export functions

In [ ]:
def generate_sequence_with_gaps(group):
    """
    Generate a sequence with gaps for missing positions from a sorted group of sequences.

    Parameters:
    - group: A pandas DataFrame containing 'position' and 'aa' (amino acid) columns.

    Returns:
    - A string representing the sequence with gaps for missing positions.
    """
    group_sorted = group.sort_values('position')
    full_range = np.arange(group_sorted['position'].min(), group_sorted['position'].max() + 1)
    sequence_parts = []

    for position in full_range:
        if position not in group_sorted['position'].values:
            sequence_parts.append("-")
        else:
            aa_list = group_sorted.loc[group_sorted['position'] == position, 'aa'].unique()
            sequence_parts.append(f"({''.join(aa_list)})" if len(aa_list) > 1 else ''.join(aa_list))

    return ''.join(sequence_parts)

def format_fasta_header(row):
    """
    Format a FASTA header string based on the sample name and GNC from a DataFrame row.

    Parameters:
    - row: A pandas Series representing a row in a DataFrame.

    Returns:
    - A string representing the formatted FASTA header.
    """
    return f">{row['sample_name']}_{row['GNC'].replace(' ', '_')}"



###Plotting Functions

### Protein aggregation functions

In [ ]:
def process_group(group):
    """
    Process a pandas DataFrame group to extract specific analytics and return results as a pd.Series.

    This function computes various metrics including the unique GNC, position, and
    sample_name values, counts for each amino acid, and detailed statistics for the
    top 3 amino acids. It includes mean, max, and standard deviation of scores,
    top scores and spectraIds, and PTM analysis. The result is returned as a pd.Series
    to facilitate easy integration with pandas data structures.

    Parameters:
    group (pandas.DataFrame): The DataFrame group to process.

    Returns:
    pandas.Series: A Series containing the computed metrics and statistics.
    """

    gnc, position, sample_name = group['GNC'].iloc[0], group['position'].iloc[0], group['sample_name'].iloc[0]

    aa_counts = group['aa'].value_counts()
    top_3_aas = aa_counts.index[:3]

    result = {'GNC': gnc, 'position': position, 'sample_name': sample_name}

    for i, aa in enumerate(top_3_aas, start=1):
        aa_group = group[group['aa'] == aa]
        scores = aa_group['score']
        top_scores = aa_group.nlargest(3, 'score')[['score', 'spectraId']]

        ptm_counts = aa_group['ptm'].value_counts()
        top_ptms = ptm_counts.index[:2]

        aa_info = {
            f'aa{i}': aa,
            f'n_aa{i}': aa_counts[aa],
            f'aa{i}_score_mean': scores.mean(),
            f'aa{i}_score_max': scores.max(),
            f'aa{i}_score_std': scores.std(),
        }

        aa_info.update({
            **{f'score{j}_aa{i}': score for j, (score, _) in enumerate(top_scores['score'], start=1)},
            **{f'spectraId{j}_aa{i}': spectra_id for j, (_, spectra_id) in enumerate(top_scores['spectraId'], start=1)}
        })

        for j, ptm in enumerate(top_ptms, start=1):
            ptm_group = aa_group[aa_group['ptm'] == ptm]
            scores = ptm_group['score']
            top_ptm_score = ptm_group.nlargest(1, 'score')[['score', 'spectraId']].iloc[0]

            aa_info.update({
                f'ptm{j}_aa{i}': ptm,
                f'%ptm{j}_aa{i}': (ptm_group.shape[0] / aa_group.shape[0]) * 100,
                f'ptm{j}_aa{i}_score_mean': scores.mean(),
                f'ptm{j}_aa{i}_score_max': scores.max(),
                f'ptm{j}_aa{i}_score_std': scores.std(),
                f'score1_ptm{j}_aa{i}': top_ptm_score['score'],
                f'spectraId1_ptm{j}_aa{i}': top_ptm_score['spectraId']
            })

        result.update(aa_info)

    return pd.Series(result)

# Load files

## Enzymatic Cleavage Sites

In [ ]:
#@title ###Amino acid & PTM colours from dictionary

def load_colors(colors_file: Path) -> Dict:
    """Loads all color definitions from a JSON file.

    Args:
        colors_file: Path object representing the path to the JSON file containing color definitions.

    Returns:
        A dictionary containing all color definitions from the JSON file.
    """
    with colors_file.open('r') as file:
        return json.load(file)

def get_amino_acid_colors(colors: Dict) -> Dict:
    """Retrieves amino acid colors from the loaded color definitions.

    Args:
        colors: Dictionary containing all color definitions loaded from the JSON file.

    Returns:
        A dictionary containing amino acid colors (if present in the JSON file).
    """
    return colors.get("amino_acids_colors", {})  # Safer retrieval

def get_ptm_colors(colors: Dict, software: str) -> Dict:
    """Retrieves PTM colors for a specific software from the loaded color definitions.

    Args:
        colors: Dictionary containing all color definitions loaded from the JSON file.
        software: Name of the software for which to retrieve PTM colors.

    Returns:
        A dictionary containing PTM colors for the specified software (if present in the JSON file).
    """
    color_dict_name = f"{software}_ptm_colors"
    return colors.get(color_dict_name, {})  # Safer retrieval

# # Usage
# colors_file = Path('/content/drive/MyDrive/Colab_Notebooks/Dictionaries/Colours/colors.json')
# colors = load_colors(colors_file)

# amino_acid_colors = get_amino_acid_colors(colors)
# ptm_colors = get_ptm_colors(colors, "pFind")  # Replace with desired software

# # Print the values to check them
# print("Amino Acid Colors:", amino_acid_colors)
# print("PTM Colors:", ptm_colors)


In [ ]:
#Functions to obtain files from the directory

def combine_spectra_files(base_paths):
    """
    Combines spectra files from multiple base paths into a single DataFrame.
    Adjusted to handle tab-separated values (TSV) by specifying the sep parameter.

    Args:
        base_paths: A list of base paths containing the "pFind-Filtered.spectra" files.

    Returns:
        A pandas DataFrame containing the combined spectra data (or None if no files found).
    """
    all_data = []
    for base_path in base_paths:
        spectra_file = f'{base_path}result/pFind-Filtered.spectra'
        try:
            # Read the spectra file using pandas.read_csv with sep='\t' for tab-separated files
            data = pd.read_csv(spectra_file, sep='\t')
            all_data.append(data)
        except FileNotFoundError:
            print(f"File not found: {spectra_file}")

    # Combine all dataframes into a single one (if any files were found)
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        print("No spectra files found in provided paths.")
        return None


# def find_spectra_files(base_path):
#     """
#     Walk through the directory structure starting from base_path to find all
#     pFind-Filtered.spectra files.

#     Parameters:
#     - base_path (str): The base directory to start the search from.

#     Returns:
#     - list: A list of paths to pFind-Filtered.spectra files.
#     """
#     spectra_files = []
#     for root, dirs, files in os.walk(base_path):
#         for file in files:
#             if file == 'pFind-Filtered.spectra':
#                 spectra_files.append(os.path.join(root, file))
#     return spectra_files

def combine_spectra_files_from_single_directory(spectra_files):
    """
    Combine spectra files into a single pandas DataFrame.

    Parameters:
    - spectra_files (list): A list of paths to pFind-Filtered.spectra files.

    Returns:
    - DataFrame: A combined DataFrame of all spectra files.
    """
    df_list = []
    for file in spectra_files:
        df = pd.read_csv(file, sep='\t')  # Assuming the file is tab-separated
        df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

In [ ]:
#@title ### Dictionaries
#-----------------------------Importing the dictionaries
#name = 'plaster'
file_directory_base = '/content/drive/MyDrive/Colab_Notebooks/Dictionaries'

sequences_name = STUDY_NAME #'mouse'  #Sequences dictionary - used for Indexing
#sample_name = STUDY_NAME #'cow'   #Sample  dictionary
sample_name = STUDY_NAME #'cow'   #Sample  dictionary
fasta_name = FASTA_NAME #'cow'   #Sample  dictionary

sequences_dict, rename_dict = process_json_data(file_directory_base, sequences_name, sample_name)

print("First few items in sequences_dict:")
for key in list(sequences_dict.keys())[:5]:  # Adjust the number as needed
    print(f"{key}: {sequences_dict[key]}")

print("\nFirst few items in rename_dict:")
for key in list(rename_dict.keys())[:5]:  # Adjust the number as needed
    print(f"{key}: {rename_dict[key]}")


First few items in sequences_dict:
AHSG-Cap_hir: {'Index': 'AHSG-Cap_hir', 'GNC': 'AHSG', 'spp': 'Cap_hir', 'species': 'Capra hircus', 'offset': 0, 'sequence': 'IPLDPIAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLANCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNNSQVVHAAEVALATFNAQNNGSYFQLVEISRAQFVPLPVSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVAVTCTLFQTQPVIPQPQPEGAEAGAPSAVPDAAVPAPSAAGLPVGSVVAGPSVVAVALPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVHLCPGRIRYFKI'}
AHSG-Ovi_are: {'Index': 'AHSG-Ovi_are', 'GNC': 'AHSG', 'spp': 'Ovi_are', 'species': 'Ovis aries', 'offset': 0, 'sequence': 'IPLDPIAGYKEPACDDPDTEQAALAAVDYINKHLPRGYKHTLNQIDSVKVWPRRPTGEVYDIEIDTLETTCHVLDPTPLVNCSVRQQTEHAVEGDCDIHVLKQDGQFSVLFTKCDSSPDSAEDVRKLCPDCPLLAPLNNSQVVHAAEVALATFNAQNNGSYFQLVEISRAQFVPLPGSVSVEFAVAATDCIAKEVVDPTKCNLLAEKQYGFCKGSVIQKALGGEDVTVTCTLFQTQPVIPQPQPEGAEAGAPSAVPDAAVPDAAVPAPSAAGLPVGSVVAGPSVVAVPLPLHRAHYDLRHTFSGVASVESASGEAFHVGKTPIVGQPSVPGGPVHLCPGRIRYFKI'}
AHSG-Bub_bub: {'Index'

In [ ]:
#@title ### Enzymatic Cleavage Sites

def find_cleavage_sites_with_offset(collagen_sequences: dict) -> dict:
    """
    Find the positions of the letters 'K' and 'R' in given protein sequences stored in a dictionary,
    adjusting for an offset provided for each sequence.

    Parameters:
        collagen_sequences (dict): A dictionary where each key is a unique identifier for a protein,
                                   and each value is another dictionary with keys 'offset' and 'sequence'
                                   representing the offset from the start of the protein and the amino
                                   acid sequence, respectively.

    Returns:
        dict: A dictionary where each key is the protein name (GNC) and each value is a list of adjusted
              positions (1-based) where either 'K' or 'R' occurs in the sequence, considering the offset.
    """
    cleavage_sites_results = {}

    for key, value in collagen_sequences.items():
        # Use the new extract_gene_identifier function to get the gene name
        protein_name = extract_gene_identifier(key)

        sequence = value['sequence']
        offset = value['offset']
        cleavage_sites = []

        for i, amino_acid in enumerate(sequence):
            if amino_acid in ['K', 'R']:
                position_with_offset = i + 1 + offset
                cleavage_sites.append(position_with_offset)

        cleavage_sites_results[protein_name] = cleavage_sites

    return cleavage_sites_results


def extract_gene_identifier(full_gene_name):
    """
    Extracts the gene identifier from the full gene name.

    Parameters:
    - full_gene_name (str): The full gene name, e.g., 'COL3A1-Bos_mut'.

    Returns:
    - str: The extracted gene identifier, e.g., 'COL3A1'.
    """
    # Split the full gene name at the hyphen and return the first part
    return full_gene_name.split('-')[0]

# Merge sequence dictionaries
all_sequences = {**sequences_dict,
                # **non_sequences_dict
                 }
# Find cleavage sites considering the offsets
cleavage_sites_with_offset = find_cleavage_sites_with_offset(all_sequences)

# for protein, sites in cleavage_sites_with_offset.items():
#     print(f"{protein}: {sites}")

##Data Path

### Multiple files below the directory

Thisis the primary way to use the tool - the others are alternatives

In [ ]:
# Find entries by field name
field_name = 'GCN'  # Field to search    # 'COL12A1', 'K2C1', 'K22E', 'KS1c9', 'PIV', 'DCN', 'KS2',
#'yNFM', 'COLunk', 'TPM1', 'KS9', 'KS1', 'COL6A2', 'KS10', 'COL5A1', 'COL1A1',
#'COL1A2', 'AHSG', 'COL3A1', 'FibMod', 'ALB', 'COL6A1', 'TRYUP'
target_value = 'COL3A1'  # Value to match
matching_entries = find_entries_by_field(sequences_dict, field_name, target_value)

print(f"Entries where {field_name} = {target_value}:")
for key, value in matching_entries.items():
    print(f"{key}: {value}")

Entries where GCN = COL3A1:


In [ ]:
# Get the first 5 rows using head method
first_five_rows = combined_df.head(5)

# Save the DataFrame to a csv file (replace 'first_five.csv' with your desired filename)
first_five_rows.to_csv('first_five.csv', index=False)

In [ ]:
# @title Reporting Block  -------------------------------------

# Assign the df name by variable
df_name = combined_df
df_column_name = "Scan_No"

print("Describe the DataFrame :")
#print(eval(df_name).describe())

print("Key information about the DataFrame :")
#print(eval(df_name).info())

# Get total rows and unique values from 'File_Name' column
total_rows = len(combined_df)
total_unique_values = combined_df[df_column_name].nunique()
unique_values = combined_df[df_column_name].unique()

# Randomly select 20 unique values
random_selection = random.sample(list(unique_values), 20)

# Print the random selection
print("Random Selection:")
print(random_selection)

# Print total rows
print("\nTotal Rows:")
print(total_rows)

# Print unique values
print("\nUnique Values:")
print(unique_values)

# Print summary statistics of the dataframe
print("\nDataFrame Summary:")
print(combined_df.describe())

# Print total unique values
print("\nTotal Unique Values:")
print(total_unique_values)


print(combined_df)

Describe the DataFrame :
Key information about the DataFrame :
Random Selection:
[4284, 4898, 3484, 1026, 3147, 1763, 1683, 3656, 4563, 3227, 5331, 5935, 3437, 4725, 2758, 3213, 6371, 4604, 6507, 4226]

Total Rows:
48116

Unique Values:
[6381 6226 6317 ...  600  561 7555]

DataFrame Summary:
            Scan_No       Exp.MH+        Charge       Q-value      Calc.MH+  \
count  48116.000000  48116.000000  48116.000000  48116.000000  48116.000000   
mean    3793.193345   1590.523277      2.335626      0.000625   1590.518984   
std     2117.734708    541.183236      0.522127      0.001739    541.181337   
min       12.000000    700.461465      2.000000      0.000000    700.460318   
25%     2113.000000   1183.610752      2.000000      0.000000   1183.606520   
50%     3356.000000   1457.669870      2.000000      0.000000   1457.665015   
75%     5312.000000   2028.067902      3.000000      0.000000   2028.062016   
max     9945.000000   3455.706914      6.000000      0.009923   3455.687804

In [ ]:
# def find_spectra_files(base_path):
#     """
#     Recursively find all 'pFind-Filtered.spectra' files within the base_path directory.

#     Args:
#     base_path (str): The base directory path where the search will begin.

#     Returns:
#     list: A list of file paths matching the 'pFind-Filtered.spectra' pattern.
#     """
#     search_pattern = os.path.join(base_path, '**', 'result', 'pFind-Filtered.spectra')
#     files = glob.glob(search_pattern, recursive=True)
#     return files

def combine_spectra_files_from_single_directory(file_paths):
    """
    Combine spectra data from multiple files into a single DataFrame.

    Args:
    file_paths (list): List of file paths to spectra files.

    Returns:
    DataFrame: A DataFrame combining all the spectra data.
    """
    df_list = [pd.read_csv(file, sep='\t')
    for file in file_paths]
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

In [ ]:

def find_spectra_files(base_path, ms_method):
    """
    Recursively find all spectra files within the base_path directory based on the given MS method.

    Args:
    base_path (str): The base directory path where the search will begin.
    ms_method (str): The MS method, which determines the search pattern ('pFind', 'SAGE', or 'NovorCloud').

    Returns:
    tuple: A tuple containing a list of file paths matching the specified pattern
           and a dictionary mapping directories to the count of files found in each directory.
    """
    if MS_METHOD == 'pFind':
        search_pattern = os.path.join(base_path, '**', 'result', 'pFind-Filtered.spectra')
    elif MS_METHOD == 'SAGE':
        search_pattern = os.path.join(base_path, '**', 'result', 'results.sage.tsv')
    elif MS_METHOD == 'NovorCloud':
        search_pattern = os.path.join(base_path, '**', 'result', '*.peps.txt')
    else:
        raise ValueError("Invalid MS method. Supported methods are: 'pFind', 'SAGE', 'NovorCloud'.")

    files = glob.glob(search_pattern, recursive=True)

    # Extract directory names
    directories = {}
    for file in files:
        directory = os.path.dirname(file)
        if directory not in directories:
            directories[directory] = 1
        else:
            directories[directory] += 1

    return files, directories

In [ ]:


# Find pFind-Filtered.spectra files and directories
spectra_files, directories = find_spectra_files(BASE_PATH, MS_METHOD)

# Combine into a single DataFrame
combined_df = combine_spectra_files_from_single_directory(spectra_files)

# Add the length of the peptide
combined_df['peptide_length'] = combined_df['Sequence'].apply(len)

# Reporting
print(f'Total number of unique sequences: {combined_df["Sequence"].nunique()}')

# Report directories and counts
print("Directories with matching files:")
for directory, count in directories.items():
    print(f"{directory}: {count} files")


Total number of unique sequences: 5093
Directories with matching files:
/content/drive/MyDrive/Colab_Notebooks/MS_output/pFind/plaster/pFind_CE&Tuuli/result: 1 files


In [ ]:
#-------------------Reporting Block------------
# Assign the df name by variable
df_name = combined_df

# Get total rows and unique values from 'File_Name' column
total_rows = len(df_name)
total_unique_values = df_name['File_Name'].nunique()
unique_values = df_name['File_Name'].unique()

# Randomly select 20 unique values
random_selection = random.sample(list(unique_values), 20)

# Print the random selection
print("Random Selection:")
print(random_selection)

# Print total rows
print("\nTotal Rows:")
print(total_rows)

# Print unique values
print("\nUnique Values:")
print(unique_values)

# Print summary statistics of the dataframe
print("\nDataFrame Summary:")
print(combined_df.describe())

# Print total unique values
print("\nTotal Unique Values:")
print(total_unique_values)


Random Selection:
['M2_20231109-1649_QEHF2_1007138_ONJ_TR_MJC_2_E8_AEIN1387_DA284.4061.4061.3.0.dta', '20240209-0048_QEHF2_1007811_ONJ_TR_MC_CE8.3814.3814.2.0.dta', '20240209-0112_QEHF2_1007812_ONJ_TR_MC_CE9.7460.7460.3.0.dta', '20240209-0510_QEHF2_1007822_ONJ_TR_MC_CE18.3599.3599.2.0.dta', 'P2_20231109-1824_QEHF2_1007142_ONJ_TR_MJC_2_E12_AEIN297_DA288.6150.6150.2.0.dta', '20240208-2337_QEHF2_1007808_ONJ_TR_MC_CE5.1448.1448.2.0.dta', '20240209-0048_QEHF2_1007811_ONJ_TR_MC_CE8.3275.3275.2.0.dta', '20240209-0622_QEHF2_1007825_ONJ_TR_MC_CE21.4482.4482.2.0.dta', '20240209-0422_QEHF2_1007820_ONJ_TR_MC_CE17.4124.4124.2.0.dta', '20240208-2201_QEHF2_1007804_ONJ_TR_MC_CE1.1973.1973.2.0.dta', 'P2_20231109-1824_QEHF2_1007142_ONJ_TR_MJC_2_E12_AEIN297_DA288.3767.3767.4.0.dta', '20240209-0422_QEHF2_1007820_ONJ_TR_MC_CE17.3245.3245.2.0.dta', '20240209-0510_QEHF2_1007822_ONJ_TR_MC_CE18.5769.5769.2.0.dta', '20240208-2249_QEHF2_1007806_ONJ_TR_MC_CE3.3084.3084.2.0.dta', '20240208-2337_QEHF2_1007808_ONJ_T

In [ ]:
print (df_name)

                                               File_Name  Scan_No  \
0      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.638...     6381   
1      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.622...     6226   
2      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.631...     6317   
3      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.635...     6355   
4      20240209-0223_QEHF2_1007815_ONJ_TR_MC_CE12.446...     4465   
...                                                  ...      ...   
48111  20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.368...     3684   
48112  20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.143...     1438   
48113  20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.521...     5211   
48114  20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.672...     6724   
48115  20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.269...     2696   

           Exp.MH+  Charge   Q-value               Sequence     Calc.MH+  \
0      2300.162702       2  0.000000   IITHPNFNGNTLDNDIMLIK  2300.159002   
1      2299.176369 

### Muli-directories

### Single directory

In [ ]:
# STUDY_NAME = 'Mice'
# #STUDY_NAME = 'Collagen_Sequences'
# PFIND_FOLDER = f'pFind_{STUDY_NAME}'
# #Find pFind-Filtered.spectra files
# #spectra_files = find_spectra_files(BASE_PATH)

# spectra_files = [f'/content/drive/MyDrive/Colab_Notebooks/pFind/{STUDY_NAME}/{PFIND_FOLDER}/result/pFind-Filtered.spectra']
# # Combine into a single DataFrame
# combined_df = combine_spectra_files_from_single_directory(spectra_files)
# #Add the length of the peptide

# combined_df['peptide_length'] = combined_df['Sequence'].apply(len)
# #----------------------Reporting
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# #combined_df.head()

In [ ]:
# STUDY_NAME = 'CE&Tuuli'
# #STUDY_NAME = 'Collagen_Sequences'
# PFIND_FOLDER = f'pFind_{STUDY_NAME}'
# #Find pFind-Filtered.spectra files
# #spectra_files = find_spectra_files(BASE_PATH)

# spectra_files = [f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME}/{PFIND_FOLDER}/result/pFind-Filtered.spectra']
# # Combine into a single DataFrame
# combined_df = combine_spectra_files_from_single_directory(spectra_files)
# #Add the length of the peptide

# combined_df['peptide_length'] = combined_df['Sequence'].apply(len)
# #----------------------Reporting
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# #combined_df.head()

###Find Collagen motif
Iterates through each peptide in unique_peptides.
Uses a generator expression within all() to check if every third character, starting from the offset determined by len(peptide) % 3, is a 'G'.
This method directly reflects the requirement for 'G' to be every third character after an initial offset and accounts for sequences that may not start with 'G' or might be truncated.

# Building the Dataframes

## AdaptiveSequenceMatching

The `AdaptiveSequenceMatching` process is designed to iteratively refine the matching of peptide sequences against a pre-defined dictionary of known sequences. This approach is especially beneficial for accounting for variations and potential mutations in sequences that have not been sequenced or are partially known. By allowing for controlled mismatches in the sequence matching criteria, this process improves the chances of identifying close matches that would be missed by strict matching criteria.

### Purpose

The main goal of `AdaptiveSequenceMatching` is to enhance the identification of peptide sequences by:

- **Accommodating Variations**: Allowing for a specified number of mismatches accounts for natural variations and mutations within peptide sequences.
- **Increasing Sensitivity**: Iteratively expanding the range of acceptable mismatches increases the sensitivity of the matching process, enabling the identification of sequences with minor differences from known sequences.

### How It Works

1. **Initial Matching**: Starts with an initial round of strict matching against known sequences.
2. **Adaptive Refinement**: Proceeds to iteratively refine the search by allowing an increasing number of mismatches, following a predefined stepping pattern. This pattern is designed to carefully expand the criteria for matching, balancing between sensitivity and specificity.
3. **Stepping Pattern**: The core of the process, where the pattern of allowed mismatches is predefined (e.g., `[1, 1, 2, 1, 1, 2, ...]`). This dictates how the matching criteria are adjusted in each iteration, enabling a flexible approach to identifying closely related sequences.

### Usage

To utilize the `AdaptiveSequenceMatching` process, users must define:

- A list of unique peptide sequences to be matched.
- A dictionary of known sequences with associated metadata (e.g., gene, position information).
- The stepping pattern to dictate the number of allowed mismatches in each iteration.

This process is encapsulated in functions that perform initial matching (`initial_peptide_matching`) and adaptive refinement (`refine_matching_with_iterations`), culminating in a comprehensive identification of matched and unmatched peptides.

### Benefits

The `AdaptiveSequenceMatching` process is particularly useful in research and applications where:

- The complete sequence database is not available or is incomplete.
- Minor variations in peptide sequences can significantly impact the matching outcome.
- There is a need to balance between the identification of known sequences and the discovery of closely related but previously unidentified sequences.

By providing a flexible and iterative approach to sequence matching, `AdaptiveSequenceMatching` facilitates deeper insights into protein variations and contributes to the advancement of proteomic research and related fields.

In [ ]:
# Initialize dictionary to store matched peptide details with sequence length
sequence_position_dict = {}

# Initialize set for matched peptides
matched_peptides = set()

# Extract unique peptides from DataFrame
unique_peptides = combined_df['Sequence'].unique().tolist()

# First round of matching
match_peptides(unique_peptides, sequences_dict)

# Calculate unmatched peptides after the first round
remaining_unmatched_unique_found_peptides = set(unique_peptides) - matched_peptides

# Define stepping pattern
stepping_pattern = [1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 3]

# Define the maximum number of steps to perform
max_steps = 3  # Adjust this parameter to limit the number of adaptive steps

# Slice the stepping pattern according to max_steps
effective_stepping_pattern = stepping_pattern[:max_steps]

# Track the number of matches before starting adaptive steps for new match calculation
previous_matched_count = len(sequence_position_dict)

# Adaptive matching process using the effective stepping pattern
for step, current_step_mismatches in enumerate(effective_stepping_pattern):
    unmatched_peptides_before_step = len(remaining_unmatched_unique_found_peptides)

    remaining_unmatched_unique_found_peptides = iterative_match_unmatched_peptides(
        remaining_unmatched_unique_found_peptides, sequence_position_dict, sequences_dict, iterations=current_step_mismatches)

    # Calculate the number of new matches found and update for the next iteration
    new_matches_found = len(sequence_position_dict) - previous_matched_count
    previous_matched_count = len(sequence_position_dict)

    # Calculate remaining unmatched peptides after the current step
    remaining_unmatched_count = len(remaining_unmatched_unique_found_peptides)

    # Print detailed feedback for the current adaptive step
    print(f"Iteration {step + 1}: Found {new_matches_found} new matches.")
    print(f"After adaptive step {step + 1} allowing up to {current_step_mismatches} mismatches, remaining unmatched peptides: {remaining_unmatched_count}")

# Final reporting
total_matched_after_adaptive_matching = len(sequence_position_dict)
total_unmatched_after_adaptive_matching = len(remaining_unmatched_unique_found_peptides)

print(f"Total matched peptides after adaptive matching: {total_matched_after_adaptive_matching}")
print(f"Total unmatched peptides after adaptive matching: {total_unmatched_after_adaptive_matching}")


Iteration 1: Found 4015 new matches.
Iteration 1: Found 4015 new matches.
After adaptive step 1 allowing up to 1 mismatches, remaining unmatched peptides: 0
Iteration 1: Found 0 new matches.
Iteration 2: Found 0 new matches.
After adaptive step 2 allowing up to 1 mismatches, remaining unmatched peptides: 0
Iteration 1: Found 0 new matches.
Iteration 2: Found 0 new matches.
Iteration 3: Found 0 new matches.
After adaptive step 3 allowing up to 2 mismatches, remaining unmatched peptides: 0
Total matched peptides after adaptive matching: 5093
Total unmatched peptides after adaptive matching: 0


In [ ]:
# # Initialize dictionary to store matched peptide details with sequence length
# sequence_position_dict = {}

# # Initialize set for matched peptides
# matched_peptides = set()

# # Extract unique peptides from DataFrame
# unique_peptides = combined_df['Sequence'].unique().tolist()

# # First round of matching
# match_peptides(unique_peptides, sequences_dict)

# # Calculate unmatched peptides after the first round
# remaining_unmatched_unique_found_peptides = set(unique_peptides) - matched_peptides

# # Iterative second round of matching, including sequences_dict for gene and position info
# remaining_unmatched_after_iterations = iterative_match_unmatched_peptides(remaining_unmatched_unique_found_peptides, sequence_position_dict, sequences_dict, iterations=3) #change this number to increase the number of interations

# # Reporting the outcome
# total_matched_after_iterations = len(sequence_position_dict)  # Updated total matches include initial and iterative matches
# total_unmatched_after_iterations = len(remaining_unmatched_after_iterations)


# print(f"Total matched peptides after iterations: {total_matched_after_iterations}")
# print(f"Total unmatched peptides after iterations: {total_unmatched_after_iterations}")


In [ ]:
#Peek at the output of the Dictionaries =============================== report
print("--------------------------Sequences dictionary")
random_entries_seq = random.sample(list(sequence_position_dict.items()), 2)
for key, value in random_entries_seq:
    print(f"Key: {key}, Value: {value}")

print(sequence_position_dict)
#  matched_peptides in sequence_position_dict  dictionary
for peptide, details in islice(sequence_position_dict.items(), 2):
    print(f"Peptide: {peptide}, Gene: {details['gene']}, Position: {details['position']}")

print("--------------------------Renaming dictionary")
#Peek at the output of the Dictionaries =============================== report
# Sample some random entries from rename_dict
random_entries_sample = random.sample(list(rename_dict.items()), 20)
for key, value in random_entries_sample:
    print(f"Key: {key}, Value: {value}")


# Iterate over first few items in rename_dict
for peptide, details in islice(rename_dict.items(), 2):
    print(f"Filename: {details['Filename']}, Source: {details['Source']}, Sample: {details['Sample']}")


--------------------------Sequences dictionary
Key: IITHPNFNGNTLDNDIMLI, Value: {'gene': 'TRYP-Sus_sco', 'GNC': 'TRYP', 'species': 'Sus scrofa', 'position': 69, 'length': 19, 'mismatches': 0}
Key: HTARFPISAAGAASLR, Value: {'gene': 'PIV-Pse_aer', 'GNC': 'PIV', 'species': 'Pseudomonas keratitis', 'position': 113, 'length': 16, 'mismatches': 0}
{'IITHPNFNGNTLDNDIMLIK': {'gene': 'TRYP-Sus_sco', 'GNC': 'TRYP', 'species': 'Sus scrofa', 'position': 69, 'length': 20, 'mismatches': 0}, 'LGEHNIDVLEGNEQFINAAK': {'gene': 'TRYP-Sus_sco', 'GNC': 'TRYP', 'species': 'Sus scrofa', 'position': 49, 'length': 20, 'mismatches': 0}, 'YSQGNVSAVGVTYDGHTALTR': {'gene': 'PIV-Pse_aer', 'GNC': 'PIV', 'species': 'Pseudomonas keratitis', 'position': 375, 'length': 21, 'mismatches': 0}, 'TPPAGVFYQGWSATPIANGSLGHDIHHPR': {'gene': 'PIV-Pse_aer', 'GNC': 'PIV', 'species': 'Pseudomonas keratitis', 'position': 341, 'length': 29, 'mismatches': 0}, 'SDLEMQYETLQEELMALKK': {'gene': 'KS9-Hom_sap', 'GNC': 'KS9', 'species': 'Homo

In [ ]:
# # Filter the dictionary to include only items with 'mismatches' > 8
# filtered_dict = {peptide: details for peptide, details in sequence_position_dict.items() if 'mismatches' in details and details['mismatches'] > 1}

# # Peek at some entries in the filtered dictionary
# random_entries = random.sample(list(filtered_dict.items()), min(100, len(filtered_dict)))
# for key, value in random_entries:
#     print(f"Key: {key}, Value: {value}")


####Building out the df

In [ ]:
# Initialize new columns to avoid KeyError
combined_df['GNC&spp'] = None
combined_df['GNC'] = None
combined_df['start_position'] = None
combined_df['PaddedStartPosition'] = None
combined_df['seq_index'] = None
combined_df['mismatch_to_sequence_dict'] = None
combined_df['short_name'] = None
combined_df['datetime'] = None
combined_df['sample_name'] = None

# Fill NaN values in 'Modification' column with a blank space
combined_df['Modification'] = combined_df['Modification'].fillna(" ")

# Annotate Rows
# Note: Ensure sequence_position_dict is correctly defined and available
combined_df = combined_df.apply(annotate_row, positions_dict=sequence_position_dict, axis=1)

# Create 'short_name' by extracting from 'File_Name'
combined_df['short_name'] = combined_df['File_Name'].str.split('.').str[0]

# Define a function to extract the desired field from the dictionary in rename_dict
def extract_sample_name(short_name, rename_dict, field='Sample'):
    entry = rename_dict.get(short_name)
    if entry:
        return entry.get(field)
    return None

# Apply the function to map 'short_name' to 'sample_name' using 'rename_dict'
combined_df['sample_name'] = combined_df['short_name'].apply(extract_sample_name, rename_dict=rename_dict)

# Extract date-time information from the 'File_Name' column
combined_df['datetime'] = combined_df['File_Name'].str.extract(r'^(\d{8}-\d{4})')

# Convert the extracted strings to datetime objects for sorting
# Invalid or missing entries will be NaT (Not a Time) after this operation
combined_df['datetime'] = pd.to_datetime(combined_df['datetime'], format='%Y%m%d-%H%M', errors='coerce')

# Step 1: Convert the 'start_position' column to numeric, safely handling non-numeric values
combined_df['start_position'] = pd.to_numeric(combined_df['start_position'], errors='coerce')

# Step 2: Convert the 'start_position' column to pandas nullable integer type
combined_df['start_position'] = combined_df['start_position'].astype('Int64')

# Step 3: Apply zero-padding
# For zero-padding, we convert to string, apply zero-padding to numbers, and handle NaN values
combined_df['PaddedStartPosition'] = combined_df['start_position'].astype(str).str.zfill(4)

# Step 4: Handle missing values in 'mismatch_to_sequence_dict'
combined_df['mismatch_to_sequence_dict'] = combined_df['mismatch_to_sequence_dict'].apply(
    lambda x: f"{int(x):02d}" if not pd.isna(x) else 'NaN'
)

# Step 5: Create the 'seq_index' column
combined_df['seq_index'] = (
    combined_df['GNC']
    + '|'
    + combined_df['PaddedStartPosition']
    + '|'
    + combined_df['Sequence']
    + '|'
    + combined_df['mismatch_to_sequence_dict']
    + '|'
    + combined_df['Modification']
)

# Check for non-null entries if nulls are np.nan (standard missing data marker in pandas)
non_null_entries = combined_df['sample_name'].notnull().sum()

print(f"Number of non-null/empty entries in 'sample_name': {non_null_entries}")

combined_df


Number of non-null/empty entries in 'sample_name': 43523


,File_Name,Scan_No,Exp.MH+,Charge,Q-value,Sequence,Calc.MH+,Mass_Shift(Exp.-Calc.),Raw_Score,Final_Score,...,peptide_length,GNC&spp,GNC,start_position,PaddedStartPosition,seq_index,mismatch_to_sequence_dict,short_name,datetime,sample_name
0,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.638...,6381,2300.162702,2,0.000000,IITHPNFNGNTLDNDIMLIK,2300.159002,0.003700,41.306849,1.528900e-10,...,20,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|8,Deamidated...",00,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16,2024-02-09 03:59:00,Cz16
1,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.622...,6226,2299.176369,2,0.000000,IITHPNFNGNTLDNDIMLIK,2299.174986,0.001383,38.866520,2.943950e-10,...,20,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|17,Oxidation...",00,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16,2024-02-09 03:59:00,Cz16
2,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.631...,6317,2300.164047,2,0.000000,IITHPNFNGNTLDNDIMLIK,2300.159002,0.005045,38.503685,5.677730e-10,...,20,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|8,Deamidated...",00,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16,2024-02-09 03:59:00,Cz16
3,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.635...,6355,2300.162845,2,0.000000,IITHPNFNGNTLDNDIMLIK,2300.159002,0.003843,39.159320,6.162310e-10,...,20,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|8,Deamidated...",00,20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16,2024-02-09 03:59:00,Cz16
4,20240209-0223_QEHF2_1007815_ONJ_TR_MC_CE12.446...,4465,2300.164123,2,0.000000,IITHPNFNGNTLDNDIMLIK,2300.159002,0.005121,39.978869,7.295450e-10,...,20,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|8,Deamidated...",00,20240209-0223_QEHF2_1007815_ONJ_TR_MC_CE12,2024-02-09 02:23:00,Cz12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48111,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.368...,3684,1296.660347,2,0.009923,GFPGLPGPAGEPGK,1296.658220,0.002127,9.373214,3.387720e-01,...,14,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|GFPGLPGPAGEPGK|00|8,Oxidation[P];",00,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14,2024-02-09 03:11:00,Cz14
48112,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.143...,1438,1845.901582,2,0.009923,TGPPGPSGISGPPGPPGPSGK,1845.897650,0.003932,13.103194,3.399330e-01,...,21,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|TGPPGPSGISGPPGPPGPSGK|00|3,Oxidation...",00,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14,2024-02-09 03:11:00,Cz14
48113,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.521...,5211,1188.602257,2,0.009923,QPGFLGLADAR,1188.600710,0.001547,10.195329,3.404520e-01,...,11,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|QPGFLGLADAR|00|0,Formyl[AnyN-term];2...",00,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14,2024-02-09 03:11:00,Cz14
48114,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14.672...,6724,1264.698667,2,0.009923,APGPLGIAGITGAR,1264.700750,-0.002083,7.586292,3.414180e-01,...,14,TRYP-Sus_sco,TRYP,69,0069,"TRYP|0069|APGPLGIAGITGAR|00|2,Pro->pyro-Glu[P];",00,20240209-0311_QEHF2_1007817_ONJ_TR_MC_CE14,2024-02-09 03:11:00,Cz14


In [ ]:
# # Initialize new columns to avoid KeyError
# combined_df['GNC&spp'] = None
# combined_df['GNC'] = None
# combined_df['start_position'] = None
# combined_df['PaddedStartPosition'] = None
# combined_df['seq_index'] = None
# combined_df['mismatch_to_sequence_dict'] = None
# combined_df['short_name'] = None
# combined_df['datetime'] = None
# combined_df['sample_name'] = None

# # Fill NaN values in 'Modification' column with a blank space
# combined_df['Modification'] = combined_df['Modification'].fillna(" ")

# # Annotate Rows
# # Note: Ensure sequence_position_dict is correctly defined and available
# combined_df = combined_df.apply(annotate_row, positions_dict=sequence_position_dict, axis=1)

# # Create 'short_name' by extracting from 'File_Name'
# combined_df['short_name'] = combined_df['File_Name'].str.split('.').str[0]

# # Map 'short_name' to 'sample_name' using 'rename_dict'
# # Note: Ensure rename_dict is correctly defined and available
# combined_df['sample_name'] = combined_df['short_name'].map(rename_dict)

# # Extract date-time information from the 'File_Name' column
# combined_df['datetime'] = combined_df['File_Name'].str.extract(r'^(\d{8}-\d{4})')

# # Convert the extracted strings to datetime objects for sorting
# # Invalid or missing entries will be NaT (Not a Time) after this operation
# combined_df['datetime'] = pd.to_datetime(combined_df['datetime'], format='%Y%m%d-%H%M', errors='coerce')

# # Step 1: Convert the 'start_position' column to numeric, safely handling non-numeric values
# combined_df['start_position'] = pd.to_numeric(combined_df['start_position'], errors='coerce')

# # Step 2: Convert the 'start_position' column to pandas nullable integer type
# combined_df['start_position'] = combined_df['start_position'].astype('Int64')

# # Step 3: Apply zero-padding
# # For zero-padding, we convert to string, apply zero-padding to numbers, and handle NaN values
# combined_df['PaddedStartPosition'] = combined_df['start_position'].astype(str).str.zfill(4)

# # Step 4: Handle missing values in 'mismatch_to_sequence_dict'
# combined_df['mismatch_to_sequence_dict'] = combined_df['mismatch_to_sequence_dict'].apply(
#     lambda x: f"{int(x):02d}" if not pd.isna(x) else 'NaN'
# )

# # Step 5: Create the 'seq_index' column
# combined_df['seq_index'] = (
#     combined_df['GNC']
#     + '|'
#     + combined_df['PaddedStartPosition']
#     + '|'
#     + combined_df['Sequence']
#     + '|'
#     + combined_df['mismatch_to_sequence_dict']
#     + '|'
#     + combined_df['Modification']
# )

# # Check for non-null entries if nulls are np.nan (standard missing data marker in pandas)
# non_null_entries = combined_df['sample_name'].notnull().sum()

# print(f"Number of non-null/empty entries in 'sample_name': {non_null_entries}")


In [ ]:
# Check for non-null entries if nulls are np.nan (standard missing data marker in pandas)
non_null_entries = combined_df['GNC&spp'].notnull().sum()

print(f"Number of non-null/empty entries in 'GNC&spp': {non_null_entries}")

Number of non-null/empty entries in 'GNC&spp': 48116


In [ ]:
# Check for non-null entries if nulls are np.nan (standard missing data marker in pandas)
non_null_entries = combined_df['sample_name'].notnull().sum()

print(f"Number of non-null/empty entries in 'sample_name': {non_null_entries}")

Number of non-null/empty entries in 'sample_name': 43523


In [ ]:
# # Initialize new columns to avoid KeyError  old block
# combined_df['GNC&spp'] = None
# combined_df['GNC'] = None
# combined_df['start_position'] = None
# combined_df['PaddedStartPosition'] = None
# combined_df['seq_index'] = None
# combined_df['mismatch_to_sequence_dict'] = None
# combined_df['short_name'] = None
# combined_df['datetime'] = None
# combined_df['sample_name'] = None

# # Fill NaN values in 'Modification' column with a blank space
# combined_df['Modification'] = combined_df['Modification'].fillna(" ")

# # Annotate Rows
# # Note: Ensure sequence_position_dict is correctly defined and available
# combined_df = combined_df.apply(annotate_row, positions_dict=sequence_position_dict, axis=1)

# # Create 'short_name' by extracting from 'File_Name'
# combined_df['short_name'] = combined_df['File_Name'].str.split('.').str[0]

# # Map 'short_name' to 'sample_name' using 'rename_dict'
# # Note: Ensure rename_dict is correctly defined and available
# combined_df['sample_name'] = combined_df['short_name'].map(rename_dict)

# # Extract date-time information sample_namefrom the 'File_Name' column
# combined_df['datetime'] = combined_df['File_Name'].str.extract(r'^(\d{8}-\d{4})')

# # Convert the extracted strings to datetime objects for sorting
# # Invalid or missing entries will be NaT (Not a Time) after this operation
# combined_df['datetime'] = pd.to_datetime(combined_df['datetime'], format='%Y%m%d-%H%M', errors='coerce')

# # Step 1: Convert the 'start_position' column to numeric, safely handling non-numeric values
# combined_df['start_position'] = pd.to_numeric(combined_df['start_position'], errors='coerce')

# # Step 2: Convert the 'start_position' column to pandas nullable integer type
# combined_df['start_position'] = combined_df['start_position'].astype('Int64')

# # Step 3: Apply zero-padding
# # For zero-padding, we convert to string, apply zero-padding to numbers, and handle NaN values
# combined_df['PaddedStartPosition'] = combined_df['start_position'].astype('float').astype(str)
# combined_df['PaddedStartPosition'] = combined_df['PaddedStartPosition'].str.split('.').str[0]  # Remove decimal part
# combined_df['PaddedStartPosition'] = combined_df['PaddedStartPosition'].apply(lambda x: x.zfill(4) if x != 'nan' else x)
# # Step 5: Apply zero-padding to mismatch_to_sequence_dict
# combined_df['mismatch_to_sequence_dict'] = combined_df['mismatch_to_sequence_dict'].apply(lambda x: f"{int(x):02d}" if not np.isnan(x) else 'NaN')

# # Step 6: Add a sequence index
# #combined_df['seq_index'] = combined_df['GNC'] + '-' + combined_df['start_position'].astype(str) + '-' + combined_df['Sequence']
# combined_df['seq_index'] = combined_df['GNC'] + '|' + combined_df['PaddedStartPosition'] + '|' + combined_df['Sequence'] + '|' + combined_df['mismatch_to_sequence_dict'] + '|' + combined_df['Modification']

# # Check for non-null entries if nulls are np.nan (standard missing data marker in pandas)
# non_null_entries = combined_df['GNC&spp'].notnull().sum()

# print(f"Number of non-null/empty entries in 'GNC&spp': {non_null_entries}")



In [ ]:
#-------------------Reporting Block
# Assign the df name by variable
df_name = combined_df

# Get total rows and unique values from 'File_Name' column
total_rows = len(df_name)
total_unique_values = df_name['short_name'].nunique()
unique_values = df_name['short_name'].unique()

# Randomly select 20 unique values
random_selection = random.sample(list(unique_values), 20)

# Print the random selection
print("Random Selection:")
print(random_selection)

# Print total rows
print("\nTotal Rows:")
print(total_rows)

# Print unique values
print("\nUnique Values:")
print(unique_values)

# Print summary statistics of the dataframe
print("\nDataFrame Summary:")
print(df_name.describe())

# Print total unique values
print("\nTotal Unique Values:")
print(total_unique_values)

# Print total unique values
unique_short_names = df_name['short_name'].unique()
np.savetxt("unique_short_names.csv", unique_short_names, delimiter=",", fmt="%s")

# Count missing values
missing_values_count = df_name['sample_name'].isna().sum()
print(f"The number of missing values in the 'sample_name' column is: {missing_values_count}")


print (df_name.head(2))

Random Selection:
['20240208-2225_QEHF2_1007805_ONJ_TR_MC_CE2', '20240208-2313_QEHF2_1007807_ONJ_TR_MC_CE4', 'P4_20231109-1912_QEHF2_1007144_ONJ_TR_MJC_2_F2_AEIN297_DA290', 'P3_20231109-1848_QEHF2_1007143_ONJ_TR_MJC_2_F1_AEIN297_DA289', 'M3_20231109-1713_QEHF2_1007139_ONJ_TR_MJC_2_E9_AEIN656_DA285', '20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16', '20240209-0247_QEHF2_1007816_ONJ_TR_MC_CE13', '20240209-0136_QEHF2_1007813_ONJ_TR_MC_CE10', '20240209-0159_QEHF2_1007814_ONJ_TR_MC_CE11', '20240209-0558_QEHF2_1007824_ONJ_TR_MC_CE20', '20240209-0510_QEHF2_1007822_ONJ_TR_MC_CE18', '20240209-0645_QEHF2_1007826_ONJ_TR_MC_CE22', '20240208-2337_QEHF2_1007808_ONJ_TR_MC_CE5', '20240209-0534_QEHF2_1007823_ONJ_TR_MC_CE19', '20240209-0335_QEHF2_1007818_ONJ_TR_MC_CE15', '20240209-0622_QEHF2_1007825_ONJ_TR_MC_CE21', '20240209-0422_QEHF2_1007820_ONJ_TR_MC_CE17', '20240209-0024_QEHF2_1007810_ONJ_TR_MC_CE7', '20240208-2249_QEHF2_1007806_ONJ_TR_MC_CE3', 'M1_20231109-1625_QEHF2_1007137_ONJ_TR_MJC_2_E7_AEIN1471_D

In [ ]:
print (df_name.head(2))

                                           File_Name  Scan_No      Exp.MH+  \
0  20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.638...     6381  2300.162702   
1  20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.622...     6226  2299.176369   

   Charge  Q-value              Sequence     Calc.MH+  Mass_Shift(Exp.-Calc.)  \
0       2      0.0  IITHPNFNGNTLDNDIMLIK  2300.159002                0.003700   
1       2      0.0  IITHPNFNGNTLDNDIMLIK  2299.174986                0.001383   

   Raw_Score   Final_Score  ... peptide_length       GNC&spp   GNC  \
0  41.306849  1.528900e-10  ...             20  TRYP-Sus_sco  TRYP   
1  38.866520  2.943950e-10  ...             20  TRYP-Sus_sco  TRYP   

  start_position PaddedStartPosition  \
0             69                0069   
1             69                0069   

                                           seq_index  \
0  TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|8,Deamidated...   
1  TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|17,Oxidation...   

   mismatch_to_seque

In [ ]:
# First, let's identify the missing values in the 'sample_name' column
missing_values = combined_df['sample_name'].isnull()

# Next, extract the unique missing values
unique_missing_values = combined_df.loc[missing_values, 'sample_name'].unique()

# Create a DataFrame with the unique missing values
unique_missing_df = pd.DataFrame({'missing_sample_name': unique_missing_values})

# Export the DataFrame to a CSV file
unique_missing_df.to_csv('unique_missing_sample_names.csv', index=False)

print(f"CSV file 'unique_missing_sample_names.csv' has been created with the unique missing sample names.")

CSV file 'unique_missing_sample_names.csv' has been created with the unique missing sample names.


In [ ]:
# Calculate string lengths
combined_df['seq_length'] = combined_df['seq_index'].str.len()

# Sort by string length (ascending for shortest, descending for longest)
shortest_strings = combined_df.nsmallest(5, 'seq_length')
longest_strings = combined_df.nlargest(5, 'seq_length')

# Display the results
print("Shortest Strings:")
print(shortest_strings['seq_index'])

print("\nLongest Strings:")
print(longest_strings['seq_index'])

Shortest Strings:
34860     PIV|0335|LLELKR|00| 
35637     PIV|0335|LLELKR|00| 
36666     PIV|0335|LLELKR|00| 
29486    TRYP|0069|IIELKR|00| 
29797    TRYP|0069|IIELKR|00| 
Name: seq_index, dtype: object

Longest Strings:
36956    TRYP|0069|GSDGQPGPPGPPGTSGFPGSPGAK|00|5,Deamid...
2471     COL1A1|0995|GPPGSAGSPGKDGLNGLPGPIGPPGPR|00|15,...
7280     COL1A1|0995|GPPGSAGSPGKDGLNGLPGPIGPPGPR|00|15,...
13959    TRYP|0069|GPPGSSGSPGKDGLNGLPGPIGPPGPR|00|15,De...
44719    TRYP|0069|IITHPNFNGNTLDNDIMLIKLSSPATLNSR|00|14...
Name: seq_index, dtype: object


In [ ]:
# if 'short_name' in combined_df.columns:
#     # Proceed with extracting unique values
#     unique_values = combined_df['short_name'].unique()
#     print("Unique values in 'short_name':")
#     print(unique_values)
#     print(f"Total number of unique values: {len(unique_values)}")
# else:
#     print("The column 'short_name' does not exist in the DataFrame.")

In [ ]:
# # Most common entries in 'GNC&spp'
# most_common_entries = combined_df['GNC&spp'].value_counts().head(40)  # Adjust the number inside head() as needed
# print("Most common entries in 'GNC&spp':")
# print(most_common_entries)

# # Calculating the percentage of rows with an entry in 'GNC&spp'
# total_rows = combined_df.shape[0]
# percentage_with_entry = (non_null_entries / total_rows) * 100

# print(f"Percentage of rows with an entry in 'GNC&spp': {percentage_with_entry:.2f}%")

##Pivot Table:  df_pt

In [ ]:
#====================Reporting=====================
# pd.set_option('display.max_columns', 28)
print(combined_df.head(2))

# Check for missing values
print(combined_df[['seq_index', 'sample_name']].isna().sum())

# Check data types
print(combined_df[['seq_index', 'sample_name']].dtypes)

# Check for common values
print(combined_df['seq_index'].nunique())
print(combined_df['sample_name'].nunique())

                                           File_Name  Scan_No      Exp.MH+  \
0  20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.638...     6381  2300.162702   
1  20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.622...     6226  2299.176369   

   Charge  Q-value              Sequence     Calc.MH+  Mass_Shift(Exp.-Calc.)  \
0       2      0.0  IITHPNFNGNTLDNDIMLIK  2300.159002                0.003700   
1       2      0.0  IITHPNFNGNTLDNDIMLIK  2299.174986                0.001383   

   Raw_Score   Final_Score  ...       GNC&spp   GNC start_position  \
0  41.306849  1.528900e-10  ...  TRYP-Sus_sco  TRYP             69   
1  38.866520  2.943950e-10  ...  TRYP-Sus_sco  TRYP             69   

  PaddedStartPosition                                          seq_index  \
0                0069  TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|8,Deamidated...   
1                0069  TRYP|0069|IITHPNFNGNTLDNDIMLIK|00|17,Oxidation...   

  mismatch_to_sequence_dict                                  short_name  \
0     

In [ ]:
# Creating a pivot table to count occurrences of each unique combination of attributes per sample
df_pt = combined_df.pivot_table(
    index=['seq_index'],
    columns='sample_name',
    values='GNC',
    aggfunc='count',
    fill_value=0)

In [ ]:
# =============================Reporting combined_df================================

# @title Reporting Block  -------------------------------------
import random
# Assign the df name by variable
df_name = combined_df
df_column_name = "sample_name"

# print("Describe the DataFrame :")
# #print(eval(df_name).describe())

# print("Key information about the DataFrame :")
# #print(eval(df_name).info())

# Get total rows and unique values from 'File_Name' column
total_rows = len(df_name)
total_unique_values = df_name[df_column_name].nunique()
unique_values = df_name[df_column_name].unique()

# Randomly select 20 unique values
random_selection = random.sample(list(unique_values), 20)

# Print the random selection
print("Random Selection:")
print(random_selection)

# # Print total rows
# print("\nTotal Rows:")
# print(total_rows)

# # # Print unique values
# # print("\nUnique Values:")
# # print(unique_values)

# # Print summary statistics of the dataframe
# print("\nDataFrame Summary:")
# print(df_name.describe())

# Print total unique values
print("\nTotal Unique Values:")
print(total_unique_values)

# Print a list of the column names
column_names = df_name.columns
print("List of Column Names:")
print(column_names)

print(df_name)

Random Selection:
['Cz13', 'P1', 'Cz8', 'Cz21', 'Cz6', 'Cz18', 'Cz1', 'P3', 'Cz3', 'Cz4', None, 'Cz22', 'P4', 'Cz14', 'Cz15', 'Cz19', 'Cz16', 'P2', 'Cz5', 'Cz12']

Total Unique Values:
27
List of Column Names:
Index(['File_Name', 'Scan_No', 'Exp.MH+', 'Charge', 'Q-value', 'Sequence',
       'Calc.MH+', 'Mass_Shift(Exp.-Calc.)', 'Raw_Score', 'Final_Score',
       'Modification', 'Specificity', 'Proteins', 'Positions', 'Label',
       'Target/Decoy', 'Miss.Clv.Sites', 'Avg.Frag.Mass.Shift', 'Others',
       'peptide_length', 'GNC&spp', 'GNC', 'start_position',
       'PaddedStartPosition', 'seq_index', 'mismatch_to_sequence_dict',
       'short_name', 'datetime', 'sample_name', 'seq_length'],
      dtype='object')
                                               File_Name  Scan_No  \
0      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.638...     6381   
1      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.622...     6226   
2      20240209-0359_QEHF2_1007819_ONJ_TR_MC_CE16.631...     6317   
3 

In [ ]:
# #====================Reporting=====================
# # pd.set_option('display.max_columns', 28)
#print(combined_df)
print(df_pt)

sample_name                                         Cz1  Cz10  Cz12  Cz13  \
seq_index                                                                   
COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamidat...    0     0     0     0   
COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|5,Oxidatio...    0     0     0     0   
COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|6,Oxidatio...    0     0     0     0   
COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|9,Oxidatio...    0     0     0     0   
COL1A1|0043|QGPPGEPGEPGSSGPMGPR|00|0,Gln->pyro-...    0     0     0     0   
...                                                 ...   ...   ...   ...   
TRYP|0117|ISGWGNTK|00|                                0     0     0     0   
TRYP|0117|ISGWGNTK|00|8,Methyl[K];                    0     0     0     0   
TRYP|0211|YVNWIQQTIAAN|00|                            0     2     5     0   
TRYP|0211|YVNWIQQTIAAN|00|6,Deamidated[Q];            0     0     0     0   
TRYP|0214|WIQQTIAAN|00|                               0     0     5     0   

In [ ]:
# Resetting the index so 'seq_index' becomes a column
df_pt.reset_index(inplace=True)

# Define a function to split 'seq_index' and extract components
def split_seq_index(seq_index):
    parts = seq_index.split('|')
    while len(parts) < 5:
        parts.append('')
    return parts[0], parts[1], parts[2], parts[3], parts[4]
    # # Check the length of parts and add np.nan if necessary
    # while len(parts) < 5:
    #     parts.append(np.nan)

    # return pd.Series(parts[:5])

# Apply the function and assign the results to new columns directly
df_pt[['GNC', 'seq_pos', 'seq', 'mismatches', 'ptms_str']] = df_pt['seq_index'].apply(
    lambda x: pd.Series(split_seq_index(x))
)

# Display the resulting dataframe
df_pt.head()

sample_name,seq_index,Cz1,Cz10,Cz12,Cz13,Cz14,Cz15,Cz16,Cz17,Cz18,...,M4,P1,P2,P3,P4,GNC,seq_pos,seq,mismatches,ptms_str
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,COL1A1,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];"
1,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|5,Oxidati...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,COL1A1,0041,GFQGPPGEPGEPGSSGPMGPR,00,"5,Oxidation[P];9,Oxidation[P];18,Oxidation[M];"
2,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|6,Oxidati...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,COL1A1,0041,GFQGPPGEPGEPGSSGPMGPR,00,"6,Oxidation[P];9,Oxidation[P];18,Oxidation[M];"
3,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|9,Oxidati...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,COL1A1,0041,GFQGPPGEPGEPGSSGPMGPR,00,"9,Oxidation[P];12,Oxidation[P];18,Oxidation[M];"
4,"COL1A1|0043|QGPPGEPGEPGSSGPMGPR|00|0,Gln->pyro...",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,COL1A1,0043,QGPPGEPGEPGSSGPMGPR,00,"0,Gln->pyro-Glu[AnyN-termQ];7,Oxidation[P];16,..."


In [ ]:
##---------This is new  - does not show restult but removes samples with no idenx Resetting the index so 'seq_index' becomes a column
# df_pt.reset_index(inplace=True)

# # Define a function to split 'seq_index' and extract components
# def split_seq_index(seq_index):
#     parts = seq_index.split('|')
#     return parts[0], parts[1], parts[2], parts[3], parts[4] if len(parts) > 4 else ''

# # Apply the function and assign the results to new columns directly
# df_pt[['GNC', 'seq_pos', 'seq', 'mismatches', 'ptms_str']] = df_pt['seq_index'].apply(
#     lambda x: pd.Series(split_seq_index(x))
# )


In [ ]:
# # # Resetting the index so 'seq_index' becomes a column Ensure 'seq_index' is a column
# df_pt.reset_index(inplace=True)

# # Define a function to split 'seq_index' and extract components
# def split_seq_index(seq_index):
#     parts = seq_index.split('|')
#     # Ensure the returned tuple always has 5 elements
#     while len(parts) < 5:
#         parts.append('')
#     return parts[0], parts[1], parts[2], parts[3], parts[4]

# # Apply the function and assign the results to new columns directly
# df_pt[['GNC', 'seq_pos', 'seq', 'mismatches', 'ptms_str']] = df_pt['seq_index'].apply(
#     lambda x: pd.Series(split_seq_index(x))
# )

# import ace_tools as tools; tools.display_dataframe_to_user(name="df_pt", dataframe=df_pt)


In [ ]:
def apply_ptms_to_sequence(sequence, ptms_str):
    # Handle empty PTM strings
    if not ptms_str:
        return sequence

    # Split PTMs, ensuring to strip whitespace and filter out empty entries
    ptms = [ptm.strip().split(',') for ptm in ptms_str.split(';') if ptm.strip()]
    # Convert positions to integers and ensure they're valid numbers, sort by position
    ptms = [[int(pos.strip()), mod] for pos, mod in ptms if pos.strip().isdigit()]
    ptms.sort(key=lambda x: x[0])

    # Initialize sequence list and special handling for N-terminal modifications
    seq_list = list(sequence)
    n_terminal_mods = ""

    # Apply PTMs
    for pos, mod in ptms:
        mod_name = mod  # We include the entire modification descriptor
        pos_index = pos - 1  # Adjust for 1-based indexing
        if pos == 0:  # N-terminal modification
            n_terminal_mods += f"({mod_name})"
        elif 0 <= pos_index < len(seq_list):
            seq_list[pos_index] = f"{seq_list[pos_index]}({mod_name})"

    # Combine N-terminal mods with the sequence
    modified_sequence = f"{n_terminal_mods}{''.join(seq_list)}"

    return modified_sequence

# Apply the function to each row in df_pt to create the 'seq_w_ptm' column
df_pt['seq_w_ptm'] = df_pt.apply(lambda row: apply_ptms_to_sequence(row['seq'], row['ptms_str']), axis=1)


##New Block

##Explode df_pt : df_pt_exp

In [ ]:
def prepare_for_explode(sequence, ptms_str, start_position):
    # Apply PTMs to sequence as before
    modified_sequence = apply_ptms_to_sequence(sequence, ptms_str)  # Assuming this function is already defined

    # Find all components (amino acids with or without modifications)
    components = re.findall(r'\(.*?\)[A-Z]|[A-Z]\(.*?\)|[A-Z]', modified_sequence)

    # Combine each component with its position, prefixing with the position
    combined_components = [f"{start_position + i}_{comp}" for i, comp in enumerate(components)]

    # Join the components with commas for easy explosion
    return ",".join(combined_components)

# Adjust the DataFrame processing to use the updated function
df_pt['seq_w_ptm_positioned'] = df_pt.apply(lambda row: prepare_for_explode(row['seq'], row['ptms_str'], int(row['seq_pos'])), axis=1)


In [ ]:
# Convert the 'seq_w_ptm_positioned' column into a list of elements for each row
df_pt['seq_w_ptm_positioned'] = df_pt['seq_w_ptm_positioned'].str.split(',')

# Explode the DataFrame based on this column
df_pt_exp = df_pt.explode('seq_w_ptm_positioned')

# Ensure the 'seq_w_ptm_positioned' column is treated as string
df_pt_exp['seq_w_ptm_positioned'] = df_pt_exp['seq_w_ptm_positioned'].astype(str)

# Optionally, split the 'seq_w_ptm_positioned' column into separate columns if needed
#df_pt_exp[['Amino_Acid_with_Mod', 'Position']] = df_pt_exp['seq_w_ptm_positioned'].str.extract(r'([A-Za-z\(\)\[\]-]+)(\d+)')

In [ ]:
# Regular expression to match the structure and capture relevant parts
regex_pattern = r'(?P<Position>\d+)_(?:\((?P<n_term_mod>.+?)\))?([A-Z])(?:\((?P<ptm>.+?)\))?'

# Apply the regex to extract and create new columns
df_pt_exp[['Position', 'n_term_mod', 'amino_acid', 'ptm']] = df_pt_exp['seq_w_ptm_positioned'].str.extract(regex_pattern)

# Convert 'Position' back to numeric, if it's not already
df_pt_exp['Position'] = pd.to_numeric(df_pt_exp['Position'])


In [ ]:
# pd.set_option('display.max_columns', 28)
df_pt_exp.head(5)

sample_name,seq_index,Cz1,Cz10,Cz12,Cz13,Cz14,Cz15,Cz16,Cz17,Cz18,...,seq_pos,seq,mismatches,ptms_str,seq_w_ptm,seq_w_ptm_positioned,Position,n_term_mod,amino_acid,ptm
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,41_G,41,NaN,G,NaN
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,42_F,42,NaN,F,NaN
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,43_Q(Deamidated[Q]),43,NaN,Q,Deamidated[Q]
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,44_G,44,NaN,G,NaN
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,45_P,45,NaN,P,NaN


### Aggregation

In [ ]:
# # Renaming and retyping columns for clarity  - this is a memory hog!
# df_pt_exp = df_pt_exp.rename(columns={
#     'GNC': 'genome_nomenclature',
#     'seq_pos': 'position',
#     'amino_acid': 'amino_acid',
#     'n_term_mod': 'n_terminal_mod',
#     'ptm': 'post_translational_mod'
# })
# df_pt_exp['position'] = df_pt_exp['position'].astype(int)
# df_pt_exp['amino_acid'] = df_pt_exp['amino_acid'].astype('category')
# df_pt_exp['n_terminal_mod'] = df_pt_exp['n_terminal_mod'].astype('category')
# df_pt_exp['post_translational_mod'] = df_pt_exp['post_translational_mod'].astype('category')

# # Selecting sample columns by their position (assuming the first column is at position 0 in Python indexing)
# sample_columns = df_pt_exp.columns[1:43]  # Adjusts for Python's 0-indexing, selects columns 1 to 42

# # Columns for grouping
# grouping_columns = ['genome_nomenclature', 'position', 'amino_acid', 'n_terminal_mod', 'post_translational_mod']

# # Perform the aggregation
# compressed_df = df_pt_exp.groupby(grouping_columns)[sample_columns].sum().reset_index()

# print(compressed_df)

In [ ]:
df_pt_exp

sample_name,seq_index,Cz1,Cz10,Cz12,Cz13,Cz14,Cz15,Cz16,Cz17,Cz18,...,seq_pos,seq,mismatches,ptms_str,seq_w_ptm,seq_w_ptm_positioned,Position,n_term_mod,amino_acid,ptm
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,41_G,41,NaN,G,NaN
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,42_F,42,NaN,F,NaN
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,43_Q(Deamidated[Q]),43,NaN,Q,Deamidated[Q]
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,44_G,44,NaN,G,NaN
0,"COL1A1|0041|GFQGPPGEPGEPGSSGPMGPR|00|3,Deamida...",0,0,0,0,0,0,0,0,0,...,0041,GFQGPPGEPGEPGSSGPMGPR,00,"3,Deamidated[Q];9,Oxidation[P];18,Oxidation[M];",GFQ(Deamidated[Q])GPPGEP(Oxidation[P])GEPGSSGP...,45_P,45,NaN,P,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7152,TRYP|0214|WIQQTIAAN|00|,0,0,5,0,0,0,5,0,0,...,0214,WIQQTIAAN,00,,WIQQTIAAN,218_T,218,NaN,T,NaN
7152,TRYP|0214|WIQQTIAAN|00|,0,0,5,0,0,0,5,0,0,...,0214,WIQQTIAAN,00,,WIQQTIAAN,219_I,219,NaN,I,NaN
7152,TRYP|0214|WIQQTIAAN|00|,0,0,5,0,0,0,5,0,0,...,0214,WIQQTIAAN,00,,WIQQTIAAN,220_A,220,NaN,A,NaN
7152,TRYP|0214|WIQQTIAAN|00|,0,0,5,0,0,0,5,0,0,...,0214,WIQQTIAAN,00,,WIQQTIAAN,221_A,221,NaN,A,NaN


In [ ]:
# @title Reporting
# =============================#Reporting================================

# #Check the df again!
# df_pt_exp.head()
#comp_exp_df.shape
#comp_exp_df.head(5).info()
#comp_exp_df.head(5)
df_pt_exp.info()
# #df_pt_exp.describe()
# df_pt_exp['Sequence'].nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 121882 entries, 0 to 7152
Data columns (total 39 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   seq_index             121882 non-null  object
 1   Cz1                   121882 non-null  int64 
 2   Cz10                  121882 non-null  int64 
 3   Cz12                  121882 non-null  int64 
 4   Cz13                  121882 non-null  int64 
 5   Cz14                  121882 non-null  int64 
 6   Cz15                  121882 non-null  int64 
 7   Cz16                  121882 non-null  int64 
 8   Cz17                  121882 non-null  int64 
 9   Cz18                  121882 non-null  int64 
 10  Cz19                  121882 non-null  int64 
 11  Cz21                  121882 non-null  int64 
 12  Cz22                  121882 non-null  int64 
 13  Cz3                   121882 non-null  int64 
 14  Cz4                   121882 non-null  int64 
 15  Cz5                   12

In [ ]:
# Define the maximum column index variable, excluding the last 11 columns of df_pt_exp
max_col_index = df_pt_exp.shape[1] - 11  # Subtracts 11 to ignore the last 11 columns
# ignoring
      #  53  GNC                   464748 non-null  object
      #  54  seq_pos               464748 non-null  object
      #  55  seq                   464748 non-null  object
      #  56  mismatches            464748 non-null  object
      #  57  ptms_str              464748 non-null  object
      #  58  seq_w_ptm             464748 non-null  object
      #  59  seq_w_ptm_positioned  464748 non-null  object
      #  60  Position              464748 non-null  int64
      #  61  n_term_mod            1139 non-null    object
      #  62  amino_acid            464748 non-null  object
      #  63  ptm                   37259 non-null   object

sample_columns = df_pt_exp.columns[1:max_col_index]  # Adjusts dynamically to the updated number of columns

In [ ]:
# # Define grouping columns (using lowercase for consistency)
# grouping_columns = ['GNC', 'Position', 'amino_acid', 'n_term_mod', 'ptm']

# # Fill missing values and group by the specified columns
# comp_exp_df = (
#     df_pt_exp.fillna('NA')
#     .groupby(grouping_columns, as_index=False)[sample_columns]
#     .sum()
# )

# # Ensure sample_columns are numeric, coercing errors to NaN
# comp_exp_df[sample_columns] = comp_exp_df[sample_columns].apply(pd.to_numeric, errors='coerce')

# # Calculate total observations
# comp_exp_df['total_observations'] = comp_exp_df[sample_columns].sum(axis=1)

# # Calculate the maximum total observations for each GNC
# max_total_observations = comp_exp_df.groupby('GNC')['total_observations'].transform('max')

# # Calculate normalised total observations
# comp_exp_df['normalised_total_observations'] = comp_exp_df['total_observations'] / max_total_observations



In [ ]:
#The error TypeError: can only concatenate str (not "int") to str suggests that there are non-numeric values in your sample_columns, causing the sum function to fail.
#To address this, ensure all values in sample_columns are numeric before performing the summation.

# # Define grouping columns (using lowercase for consistency)
# grouping_columns = ['GNC', 'Position', 'amino_acid', 'n_term_mod', 'ptm']

# # Perform aggregation
# comp_exp_df = (
#     df_pt_exp.fillna('NA')
#     .groupby(grouping_columns, as_index=False)[sample_columns]
#     .sum()
# )

# # Calculate total observations
# comp_exp_df['total_observations'] = comp_exp_df[sample_columns].sum(axis=1)

# # Calculate the maximum total observations for each GNC
# max_total_observations = comp_exp_df.groupby('GNC')['total_observations'].transform('max')

# # Calculate normalised total observations
# comp_exp_df['normalised_total_observations'] = comp_exp_df['total_observations'] / max_total_observations

# # Display the resulting DataFrame
# import ace_tools as tools; tools.display_dataframe_to_user(name="Computed Experimental Data", dataframe=comp_exp_df)

# # Output the DataFrame to check
# print(comp_exp_df)


In [ ]:
#-----------------------# Define grouping columns (using lowercase for consistency)
grouping_columns = ['GNC', 'Position', 'amino_acid', 'n_term_mod', 'ptm']

# Perform aggregation
comp_exp_df = (
    df_pt_exp.fillna('NA')
    .groupby(grouping_columns)[sample_columns]
    .sum()
    .reset_index()
)

# Calculate total observations
comp_exp_df['total_observations'] = comp_exp_df[sample_columns].sum(axis=1)

# Calculate the maximum total observations for each GNC
max_total_observations = comp_exp_df.groupby('GNC')['total_observations'].transform('max')

# Calculate normalized total observations
comp_exp_df['normalised_total_observations'] = comp_exp_df['total_observations'] / max_total_observations

## Save comp_exp_df

In [ ]:
# Output directory exists
output_directory = f'{BASE_PATH}/'
os.makedirs(output_directory, exist_ok=True)

# Get the current timestamp
now = datetime.now().strftime("%Y-%m-%d_%H")

# Create the filename with study name(s) and timestamp
filename = f"{output_directory}{STUDY_NAME}_{now}_vidx.csv"


# Save the DataFrame to CSV
comp_exp_df.to_csv(filename, index=False)  # Set index=False if you do not want to save row indices in the file

print(f"Data saved to {filename}")

Data saved to /content/drive/MyDrive/Colab_Notebooks/MS_output/pFind/plaster/plaster_2024-06-16_19_vidx.csv


In [ ]:
# #@title Import saved file
# directory = "Mice"
# filename = "Mice_2024-05-02_10_vidx.csv"

# comp_exp_df = pd.read_csv(f"content/drive/MyDrive/Colab_Notebooks/MS_output/pFind/{directory}/'+ filename)")

#Analysis

In [ ]:
import plotly.express as px

In [ ]:
#@title PTM counts
# Count the occurrences of n_term_mod and ptm, selecting top 20
n_term_mod_counts = comp_exp_df['n_term_mod'].value_counts().head(20)
ptm_counts = comp_exp_df['ptm'].value_counts().head(20)

# Print the n_term_mod counts from most to least common
print("n_term_mod Counts:")
print(n_term_mod_counts)

# Print the ptm counts from most to least common
print("ptm Counts:")
print(ptm_counts)

n_term_mod Counts:
n_term_mod
NA                                       6898
Formyl[AnyN-term]                          33
Carbamyl[AnyN-term]                        20
Gln->pyro-Glu[AnyN-termQ]                  19
Acetyl[ProteinN-term]                      17
Carbamidomethyl[AnyN-term]                 15
2)[AnyN-term]                              13
glycidamide[AnyN-term]                     11
Succinyl[AnyN-term]                        10
ICPL_13C(6)[AnyN-term]                      9
Glu->pyro-Glu[AnyN-termE]                   9
C+12[AnyN-term]                             9
1)[T]                                       8
1)[S]                                       8
Carboxymethyl[AnyN-term]                    8
Delta_H(2                                   7
Propionamide[AnyN-term]                     7
Methyl[AnyN-term]                           5
Dimethyl[AnyN-term](Ethyl[AnyN-term])       5
Acetyl[AnyN-term]                           4
Name: count, dtype: int64
ptm Counts:
ptm
NA      

In [ ]:
# @title Reporting
# =============================#Reporting================================

# #Check the df again!
# df_pt_exp.head()
#comp_exp_df.shape
#comp_exp_df.head(5).info()
#comp_exp_df.head(5)
#comp_exp_df.info()
# #df_pt_exp.describe()
# df_pt_exp['Sequence'].nunique()

In [ ]:
#-------------------Reporting Block
# Assign the df name by variable
df_name = comp_exp_df  # DataFrame name




print("Describe the DataFrame :")
print(df_name.describe())  # Access DataFrame directly

print("Key information about the DataFrame :")
print(df_name.info())  # Access DataFrame directly

# # Get total rows and unique values from 'File_Name' column
# df_column_name = "M2"total_rows = len(df_name)

# total_unique_values = df_name[df_column_name].nunique()  # Access column using bracket notation

# unique_values = df_name.loc[:, df_column_name].unique()  # Use .loc for column selection


# Randomly select 20 unique values
random_selection = random.sample(list(unique_values), 20)

# Print the random selection
print("Random Selection:")
print(random_selection)

# Print total rows
print("\nTotal Rows:")
print(total_rows)

# Print unique values
print("\nUnique Values:")
print(unique_values)

# Print summary statistics of the dataframe (assuming 'combined_df' exists)
print("\nDataFrame Summary:")
print(combined_df.describe())

# Print total unique values
print("\nTotal Unique Values:")
print(total_unique_values)


Describe the DataFrame :
sample_name     Position          Cz1         Cz10         Cz12         Cz13  \
count        7199.000000  7199.000000  7199.000000  7199.000000  7199.000000   
mean          364.415197     6.096819     1.447145     1.833032     1.208640   
std           286.013374    25.543472     7.836759    11.174618     6.403999   
min             4.000000     0.000000     0.000000     0.000000     0.000000   
25%            88.000000     0.000000     0.000000     0.000000     0.000000   
50%           311.000000     0.000000     0.000000     0.000000     0.000000   
75%           535.500000     3.000000     0.000000     0.000000     0.000000   
max          1061.000000   666.000000   200.000000   168.000000   136.000000   

sample_name         Cz14         Cz15         Cz16         Cz17         Cz18  \
count        7199.000000  7199.000000  7199.000000  7199.000000  7199.000000   
mean            6.320183     1.913738     2.535491     7.824281     4.439228   
std           

In [ ]:
# # Merge ptm_percentages with total_aa_counts on 'GNC' once outside the loop
# merged_df = pd.merge(ptm_counts, total_aa_counts, on='GNC')

# # Initialize a DataFrame to store percentages
# ptm_percentages = merged_df.copy()

# # Calculate percentages for all samples at once
# for sample in sample_columns:
#     total_aa_column = sample + '_total_aa'  # Ensuring we use the correct total amino acid count column name
#     if total_aa_column in merged_df.columns:
#         ptm_percentages[sample + '_percent'] = (merged_df[sample] / merged_df[total_aa_column]) * 100

# # Reset the index to remove potential duplicates
# ptm_percentages.reset_index(drop=True, inplace=True)


In [ ]:
import plotly.express as px

# Assuming ptm_colors is defined with appropriate color mappings for your PTMs

# Define the range of columns to plot, which is from the 5th to the second-to-last column
columns_to_plot = filtered_plot_df.columns[5:-2]  # Adjusting based on your specification

# Filter the DataFrame for the specified GNC and PTMs if not already filtered
selected_ptms = ["Oxidation[P]", "Deamidated[N]", "Deamidated[Q]", "Dioxidation[K]", "Dioxidation[E]"]
gnc = "COL1A1"
filtered_plot_df = all_ptm_percentages[(all_ptm_percentages['GNC'] == gnc) & (all_ptm_percentages['ptm'].isin(selected_ptms))]

# Prepare the scatter plot
fig = px.scatter(
    filtered_plot_df,
    x="Position",
    y=columns_to_plot,  # Dynamic selection of columns for y-axis
    size="total_peptides",  # Adjust this if you prefer different metrics for sizing
    color="ptm",  # Colour by PTM type
    labels={
        "Position": "Position in Protein Sequence",
        "value": "PTM Occurrence (%)",
        "total_peptides": "Total Peptides",
        "GNC": "Protein"
    },
    title="PTM Distribution by Protein (COL1A1) and PTM Type",
    color_discrete_map=pFind_ptm_colors,
    opacity=0.5  # Adjust marker opacity for better visualization
)

# Update hover data to show more details
fig.update_traces(
    hovertemplate="Position: %{x}<br>PTM: %{marker.color}<br>Occurrence (%): %{y}<br>Total Peptides: %{marker.size}<extra></extra>"
)

# Show the plot
fig.show()


NameError: name 'filtered_plot_df' is not defined

In [ ]:

# Filter the DataFrame for the specified GNC and PTMs
selected_ptms = ["Oxidation[P]", "Deamidated[N]", "Deamidated[Q]", "Dioxidation[K]", "Dioxidation[E]"]
gnc = "COL1A1"
filtered_plot_df = all_ptm_percentages[(all_ptm_percentages['GNC'] == gnc) & (all_ptm_percentages['ptm'].isin(selected_ptms))]


# Define the range of columns to plot, which is from the 5th to the second-to-last column
columns_to_plot = filtered_plot_df.columns[5:-2]


# Prepare the scatter plot
fig = px.scatter(
    filtered_plot_df,
    x="Position",
    y=[col for col in filtered_plot_df.columns if col.endswith('ptm_percentage')],  # Ensure this line correctly references your PTM percentage columns
    size="total_peptides",  # This sizes the markers based on the total peptides; adjust if another metric is preferred
    color="ptm",  # Colour by PTM type
    labels={
        "Position": "Position in Protein Sequence",
        "value": "PTM Occurrence (%)",
        "total_peptides": "Total Peptides",
        "GNC": "Protein"
    },
    title="PTM Distribution by Protein (COL1A1) and PTM Type",
    color_discrete_map=ptm_colors
)

# Update hover data to show more details
fig.update_traces(
    hovertemplate="Position: %{x}<br>PTM: %{marker.color}<br>Occurrence (%): %{y}<br>Total Peptides: %{marker.size}<extra></extra>"
)

# Show the plot
fig.show()


In [ ]:
import plotly.express as px

# Define the specific PTMs and GNC to plot
selected_ptms = ["Oxidation[P]", "Deamidated[N]", "Deamidated[Q]", "Dioxidation[K]", "Dioxidation[E]"]
gnc = "COL1A1"

# Filter the DataFrame for the specified GNC and PTMs
filtered_plot_df = all_ptm_percentages[(all_ptm_percentages['GNC'] == gnc) & (all_ptm_percentages['ptm'].isin(selected_ptms))]

# # Define PTM colors specific to this plot
# ptm_colors = {
#     "Oxidation[P]": "blue",
#     "Deamidated[N]": "green",
#     "Deamidated[Q]": "red",
#     "Dioxidation[K]": "purple",
#     "Dioxidation[E]": "orange"
# }

# Prepare the plot
# fig = px.scatter(
#     filtered_plot_df,
#     x="Position",
#     y=[col for col in filtered_plot_df.columns if 'ptm_percentage' in col],
#     size="total_peptides",
#     color="ptm",
#     labels={
#         "Position": "Position in Protein Sequence",
#         "value": "PTM Occurrence (%)",
#         "total_peptides": "Total Observations",
#         "GNC": "Protein"
#     },
#     title="PTM Distribution by Protein (COL1A1) and PTM",
#     color_discrete_map=ptm_colors
# )
fig = px.scatter(
    filtered_plot_df,
    x="Position",
    y=[col for col in filtered_plot_df.columns if 'ptm_percentage' in col],
    size="total_peptides",
    color="ptm",
    labels={
        "Position": "Position in Protein Sequence",
        "value": "PTM Occurrence (%)",
        "total_peptides": "Total Observations",
        "GNC": "Protein"
    },
    title="PTM Distribution by Protein (COL1A1) and PTM",
    color_discrete_map=ptm_colors,
    opacity=0.2  # Set marker opacity to 0.5 for 50% transparency
)


# Update hover data to show more details
fig.update_traces(
    hovertemplate="Position: %{x}<br>PTM: %{color}<br>Occurrence (%): %{y}<br>Total Observations: %{size}<extra></extra>"
)

# Show plot
fig.show()

# Save the plot as an HTML file
#output_path = 'path_to_save/protein_ptm_distribution.html'



In [ ]:
import plotly.express as px

# Assuming ptm_colors is defined as a dictionary mapping PTMs to colors
ptm_colors = {
    "Oxidation[P]": "blue",
    "Deamidated[Q]": "green",
    "Carbamidomethyl[C]": "red",
    "Acetyl[K]": "purple"
}

# Prepare the plot
fig = px.scatter(
    ptm_df,
    x="Position",
    y="ptm_percentage",
    size="total_obs",
    color="amino_acid",
    facet_col="GNC",
    facet_col_wrap=2,
    hover_data=["amino_acid", "ptm_percentage"],
    labels={
        "Position": "Position in Protein Sequence",
        "ptm_percentage": "PTM Occurrence (%)",
        "total_obs": "Total Observations",
        "GNC": "Protein"
    },
    title="PTM Distribution by Protein and Sample",
    color_discrete_map=ptm_colors
)

# Show plot
fig.show()

# Save the plot as an HTML file
output_path = '/path/to/save/protein_ptm_distribution.html'
fig.write_html(output_path)


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming comp_exp_df is your DataFrame
# Filtering the DataFrame for selected GNCs and PTMs
df_filtered = comp_exp_df[comp_exp_df['GNC'].isin(selected_gncs) & comp_exp_df['ptm'].isin(selected_ptms)]

# Grouping by protein, position, and amino acid to sum observations
grouped = df_filtered.groupby(['GNC', 'Position', 'amino_acid']).agg(
    total_aa=('total_observations', 'sum'),
    specific_ptm=('ptm', 'count')
).reset_index()

# Calculate PTM percentage
grouped['ptm_percentage'] = grouped['specific_ptm'] / grouped['total_aa']

# Preparing the plot
fig = px.scatter(
    grouped,
    x="Position",
    y="ptm_percentage",
    size="total_aa",
    color="amino_acid",
    facet_col="GNC",
    hover_data=["amino_acid", "ptm_percentage"],
    labels={
        "Position": "Position in Protein Sequence",
        "ptm_percentage": "PTM Occurrence (%)",
        "amino_acid": "Amino Acid",
        "GNC": "Protein"
    },
    title="PTM Distribution by Protein and Position"
)

# Adding interactive elements such as dropdown menus to filter based on additional attributes if needed
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {'method': 'restyle', 'label': 'All', 'args': [{'visible': [True]*len(grouped)}]},
                # Add more buttons for specific filters
            ],
            'direction': 'down',
            'showactive': True,
        }
    ]
)

# Show plot
fig.show()

# Save the plot to an HTML file
#fig.write_html('path_to_save/protein_ptm_distribution.html')


In [ ]:
import plotly.express as px
import pandas as pd

# Assuming your data is in the 'comp_exp_df' DataFrame
selected_gncs = ["COL1A1"]#, "COL1A2", "COL3A1", "TRYUP"]
selected_ptms = ["Oxidation[P]", "Deamidated[Q]", "Carbamidomethyl[C]", "Acetyl[K]"]

# Filter the DataFrame to include only the selected GNCs and PTMs
df_ptm_filtered = comp_exp_df[comp_exp_df['GNC'].isin(selected_gncs) & comp_exp_df['ptm'].isin(selected_ptms)]

# Calculate the total number of each amino acid at each position
df_ptm_filtered['total_aa'] = df_ptm_filtered.groupby(['GNC', 'Position', 'amino_acid'])['normalised_total_observations'].transform('sum')

# Calculate the percentage of each PTM for each amino acid at each position
for ptm in selected_ptms:
    df_ptm_filtered[f'%{ptm}_aa'] = (df_ptm_filtered[df_ptm_filtered['ptm'] == ptm]['normalised_total_observations'] / df_ptm_filtered['total_aa']) * 100

# Create the Plotly Express scatter plot with proteins side by side
fig = px.scatter(
    df_ptm_filtered,
    x="Position",
    y=["%" + ptm + "_aa" for ptm in selected_ptms],
    size="total_aa",
    color="amino_acid",
    facet_col="GNC",  # Faceting by protein to display them side by side
    facet_col_spacing=0.02,  # Adjust spacing between facets
    hover_data=["amino_acid"] + [f"%{ptm}_aa" for ptm in selected_ptms] + ["total_aa"],
    labels={
        "Position": "Position",
        "GNC": "Protein",
        "amino_acid": "Amino Acid",
        "total_aa": "Total Amino Acids"
    }
)

# Prepare dropdown menu for sample filtering
sample_names_for_filter = df_ptm_filtered['GNC'].unique()
buttons = [
    dict(
        label="All Proteins",
        method="update",
        args=[{"visible": [True] * len(fig.data)}]  # Initially, all traces are visible
    )
]

for sample in sample_names_for_filter:
    # Determine visibility: True for traces corresponding to the sample, False otherwise
    visibility = [sample_name == sample for sample_name in df_ptm_filtered['GNC']]
    buttons.append(
        dict(
            label=sample,
            method="update",
            args=[{"visible": visibility}]  # Adjust visibility based on sample
        )
    )

# Add dropdown menu to the figure
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.15,
            yanchor='top'
        )
    ],
    title="PTM Distribution by Protein and Position"
)

# Display the interactive plot
fig.show()

# Save the plot as an HTML file
#fig.write_html(f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME1}/{PFIND_FOLDER1}/result/protein_damage.html')

###Append sequences

### Append Sequences Testing

###Append Sequence Boolean Functions

In [ ]:
# def add_totals_and_normalize(df):
#     """
#     Adds a 'Total Observations' column to the DataFrame by summing observations across samples for each row.
#     Then adds a 'Normalized Observations' column by normalizing these totals against the sum of totals for each GNC.
#     """
#     # Summing observations across samples
#     df['Total Observations'] = df[['Sample_1', 'Sample_2', 'Sample_3']].sum(axis=1)

#     # Normalizing against the sum of totals for each GNC
#     gnc_totals = df.groupby('GNC')['Total Observations'].transform('sum')
#     df['Normalized Observations'] = df['Total Observations'] / gnc_totals

#     print("Added 'Total Observations' and 'Normalized Observations':")
#     print(df)

def check_gnc_species_in_dict(species_of_interest, sequences_dict):
    """
    Checks if the specified combinations of GNC and species are present in the sequence dictionary.

    Parameters:
    - species_of_interest (dict): A dictionary where keys are species codes and values are lists of GNCs.
    - sequences_dict (dict): The sequence dictionary mapping 'species-GNC' combinations to sequences.
    """
    missing_combinations = []  # To store any missing combinations for reporting

    # Iterate through each species and its GNCs of interest
    for species, gncs in species_of_interest.items():
        for gnc in gncs:
            # Construct the key used in the sequences dictionary
            dict_key = f'{species}-{gnc}'
            if dict_key not in sequences_dict:
                # If the key is missing, append to the list for reporting
                missing_combinations.append(dict_key)

    # Check if there are any missing combinations and report them
    if missing_combinations:
        print("Warning: The following GNC-Species combinations are missing from the dictionary:")
        for combo in missing_combinations:
            print(f"- {combo}")
    else:
        print("All specified GNC-Species combinations are present in the dictionary.")

# def map_enzymes_contaminants_presence(df, enzymes_list, contaminants_list):
#     # Map presence of enzymes
#     for enzyme in enzymes_list:
#         column_name = f'{enzyme}_Present'
#         df[column_name] = df['GNC'].apply(lambda x: x == enzyme)

#     # Map presence of contaminants
#     for contaminant in contaminants_list:
#         column_name = f'{contaminant}_Present'
#         df[column_name] = df['GNC'].apply(lambda x: x == contaminant)



# def map_species_sequences_presence(df, species_of_interest, sequences_dict):
#     for species, gncs in species_of_interest.items():
#         for gnc in gncs:
#             # Generate a column name for this species-GNC combination
#             column_name = f'{species}_{gnc}_Present'
#             # Initialize column to False
#             df[column_name] = False

#             # Fetch the sequence for this species-GNC combination from the dictionary
#             sequence_key = f'{species}-{gnc}'
#             if sequence_key in sequences_dict:
#                 sequence = sequences_dict[sequence_key]
#                 # Logic to update df[column_name] based on matching sequence
#                 # This will depend on the structure of your sequence data and how you're checking presence
#                 # For demonstration, let's assume a simplistic match for now:
#                 df[column_name] = df.apply(lambda x: x['GNC'] == gnc and len(sequence) > 0, axis=1)


def validate_presence(enzymes_list, contaminants_list, species_of_interest, sequences_dict):
    """
    Validates the presence of specified enzymes, contaminants, and species-GNC combinations
    in the provided sequences dictionary or lists. Prints warnings for any missing items.

    Args:
    - enzymes_list (list): List of enzyme GNCs to check.
    - contaminants_list (list): List of contaminant GNCs to check.
    - species_of_interest (dict): Dictionary mapping species codes to lists of GNCs.
    - sequences_dict (dict): Dictionary mapping species-GNC combinations to sequences.
    """
    # Combine enzymes and contaminants for a unified check
    combined_gncs = enzymes_list + contaminants_list

    # Check for enzymes and contaminants
    for gnc in combined_gncs:
        if gnc not in sequences_dict.keys():
            print(f"Warning: {gnc} not found in sequences dictionary.")

    # Check for species-GNC combinations
    for species, gncs in species_of_interest.items():
        for gnc in gncs:
            species_gnc_key = f'{species}-{gnc}'
            if species_gnc_key not in sequences_dict:
                print(f"Warning: Combination of {species} and {gnc} (key: {species_gnc_key}) not found in the sequences dictionary.")


In [ ]:
# #Debugging
# # Generating a mock-up DataFrame for testimg
# data = {
#     'GNC': ['ALB', 'COL1A1', 'COL1A1', 'COL1A1', 'COL1A1', 'COL1A2', 'COL3A1', 'KF1', 'KF9', 'PIV', 'TRYP', 'TRYP'],
#     'Position': [1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1],
#     'N-Terminal Modification': ['None', 'None', 'Pyro-Glu', 'None', 'Pyro-Glu', 'None', 'None', 'None', 'None', 'None', 'None', 'None'],
#     'PTM': ['None', 'Amination', 'Oxidation', 'None', 'Oxidation', 'Deamidation', 'None', 'None', 'None', 'None', 'None', 'None'],
#     'Sample_1': [30, 5, 10, 5, 10, 15, 5, 25, 20, 0, 0, 0],
#     'Sample_2': [31, 6, 11, 0, 11, 16, 6, 26, 1, 0, 0, 0],
#     'Sample_3': [32, 7, 12, 7, 12, 0, 7, 27, 22, 0, 0, 0]
# }

# df = pd.DataFrame(data)


# sequences_dict = {
#     'COL1A1-Bos_tau': {
#         'GNC': 'COL1A1',
#         'species': 'Bos taurus',
#         'offset': 0,
#         'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQG'
#     },
#    'COL1A1-Hip_amp': {'GNC': 'COL1A1','species': 'Hippopotamus amphibius','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGSSGPMGPRGPPGPPGKNGDDGEAGA'},
#    'COL1A2-Hip_amp': {'GNC': 'COL1A','species': 'Hippopotamus amphibius','offset': 0, 'sequence': 'QYDGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGSRGPPGPP'},
#    'COL1A1-Equ_asi': {'GNC': 'COL1A1','species': 'Equus asinus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARG'},
#    'COL1A1-Del_leu': {'GNC': 'COL1A1','species': 'Delphinapterus leucas','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGP'},

# }
# print(df)


In [ ]:
# Lists of GNCs for enzymes and contaminants
enzymes_list = ['TRYP', 'PIV']  # Example enzyme GNCs
contaminants_list = ['KF9', 'KF1', 'ALB']  # Example contaminant GNCs


# List of species to check, along with GNCs of interest for mapping presence
proteins = ['COL1A1', 'COL1A2', 'COL3A1']

speceies
species_of_interest = [Bos taurus, ]

# Call the function to check for presence in the dictionary
validate_presence(enzymes_list, contaminants_list, species_of_interest, sequences_dict)


In [ ]:

# Call to add totals and normalized counts to the DataFrame
add_totals_and_normalize(df)


# # Map species sequence presence in the DataFrame
# map_enzymes_contaminants_presence(df, enzymes_list, contaminants_list)

# # Map species sequence presence in the DataFrame
# map_species_sequence_presence(df, sequences_dict)

# # Print the final DataFrame to observe all modifications
# print("Final DataFrame after processing:")
# print(df)


###Other testing

In [ ]:
#file_path = '/content/drive/MyDrive/Colab_Notebooks/FASTA_dicts/sequences_dict.json'
file_path = '/content/drive/MyDrive/Colab_Notebooks/FASTA_dicts/mice_dict.json'
#file_path = '/content/drive/MyDrive/Colab_Notebooks/FASTA_dicts/small_sequences_dict.json'
# Reading the JSON file of the sequences_dict and converting it back into a dictionary
with open(file_path, 'r') as json_file:
    sequences_dict = json.load(json_file)


# # Print only the first few items of the dictionary
# for key in list(sequences_dict.keys())[:10]:  # Adjust the number as needed
#     print(f"{key}: {sequences_dict[key]}")


specific_key = "COL1A1-Bos_tau"  # Replace with the key you want to check
value = sequences_dict.get(specific_key)

if value is not None:
    print("Sequence is present:", value)
else:
    print("Sequence does not exist in the dictionary.")



In [ ]:
# Lists of indices for each category
species_indices = ['COL1A1-Bos_tau', 'COL1A1-Hip_amp', 'COL1A2-Hip_amp','COL1A1-Equ_asi', 'COL1A1-Del_leu']

sequences_dict = {
    'COL1A1-Bos_tau': {
        'GNC': 'COL1A1',
        'species': 'Bos taurus',
        'offset': 0,
        'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQG'
    },
   'COL1A1-Hip_amp': {'GNC': 'COL1A1','species': 'Hippopotamus amphibius','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGSSGPMGPRGPPGPPGKNGDDGEAGA'},
   'COL1A2-Hip_amp': {'GNC': 'COL1A','species': 'Hippopotamus amphibius','offset': 0, 'sequence': 'QYDGKGVGLGPGPMGLMGPRGPPGASGAPGPQGFQGPPGEPGEPGQTGPAGSRGPPGPP'},
   'COL1A1-Equ_asi': {'GNC': 'COL1A1','species': 'Equus asinus','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGPPGPQGARG'},
   'COL1A1-Del_leu': {'GNC': 'COL1A1','species': 'Delphinapterus leucas','offset': 0, 'sequence': 'GPMGPSGPRGLPGPPGAPGPQGFQGPPGEPGEPGASGPMGPRGPPGPPGKNGDDGEAGKPGRPGERGP'},

}


# Print only the first few items of the dictionary
for key in list(sequences_dict.keys())[:3]:  # Adjust the number as needed
    print(f"{key}: {sequences_dict[key]}")



In [ ]:
df_updated = pd.DataFrame()
df_updated1 = pd.DataFrame()
df = pd.DataFrame()


In [ ]:
# Lists of indices for each category
species_indices = ['COL1A1-Bos_tau', 'COL1A2-Bos_tau',#'COL3A1-Bos_tau' ,
                   #'COL1A1-Bub_bub', 'COL1A2-Bub_bub', #'COL3A1-Bub_bub',
                   'COL1A2-Cap_hir',
                   #'COL3A1-Ovi_are'
#                                                      'COL3A1-Sus_scr'
                  #'COL1A1-Del_leu' ,
                  # 'COL1A2-Del_leu' ,
                   #'COL3A1-Del_leu' ,
                  #'COL1A1-Lox_afr',#'COL1A2-Lox_afr',
                   #'COL3A1-Lox_afr',
#                   'COL1A1-Tur_tur',
 #                  'COL1A2-Can_lup',
                   'COL1A1-Hip_amp', #'COL1A2-Hip_amp',#'COL3A1-Hip_amp',
                   #'COL1A1-Equ_cab','COL1A2-Equ_cab',#'COL3A1-Equ_cab',
                  # 'COL1A1-Equ_asi','COL1A2-Equ_asi',#'COL3A1-Equ_asi',
                   ]  #
#enzymes_indices = ['TRYP-Sus_sco', 'PIV-Pse_aer']
#contaminants_indices = ['K2C1-Hom_sap', 'KS9-Hom_sap', 'KS10-Hom_sap', 'K22E-Hom_sap']


#Check species_indices against  entry = sequences_dict[index]


In [ ]:
def process_category(df, sequences_dict, indices, category_label):
    for index in indices:
        entry = sequences_dict[index]
        gnc = entry['GNC']
        sequence = entry['sequence']
        offset = entry['offset']-1

        species_identifier = index.split('-')[1]
        match_col_name = f"{species_identifier}"

        # Initialize the column with False - this will be updated below
        df[match_col_name] = False

        for position_offset, amino_acid in enumerate(sequence, start=1):
            position = position_offset + offset

            # Check if the amino acid is '-', if so, set the corresponding value to None
            if amino_acid == '-':
                df.loc[(df['GNC'] == gnc) & (df['Position'] == position), match_col_name] = None
            else:
                # Update the match column to True where there's a match, else it remains False or None
                df.loc[(df['GNC'] == gnc) & (df['Position'] == position) & (df['amino_acid'] == amino_acid), match_col_name] = True

    return df

# Example usage:
# Update your DataFrame with the new columns indicating matches
df_updated = process_category(comp_exp_df, sequences_dict, species_indices, 'species')
#df_updated1 = process_category(df, sequences_dict, species_indices, 'species')
#df_updated = process_category(df_updated, sequences_dict, enzymes_indices, 'enzyme')
#df_updated = process_category(df_updated, sequences_dict, contaminants_indices, 'contaminant')
df_updated1


In [ ]:
df_updated.head(5)

In [ ]:
# =============================Reporting df_pt================================

# #Check the df again!
comp_exp_df.head(5)
# df_pt.shape
#df_pt.info()
# #df_pt.describe()
# df_pt['Sequence'].nunique()

In [ ]:
# Update the DataFrame
df_updated = process_category(df, sequences_dict, species_indices, 'species')

df_updated

In [ ]:
# =============================Reporting df_pt_exp ================================

# #Check the df again!
#df_pt_exp.head()
#df_pt_exp.head(20)
# df_pt_exp.shape
comp_exp_df.info()
# #df_pt_exp.describe()
#df_pt_exp['ptm'].nunique()
#print(df_pt_exp.head(20).head(5))

In [ ]:
# =============================Reporting df_pt_exp ================================

# #Check the df again!
#comp_exp_df.head()
#comp_exp_df.head(20)
#comp_exp_df.shape
#comp_exp_df.info()
# #comp_exp_df.describe()
#df_pt_exp['ptm'].nunique()
print(comp_exp_df.head(10))

##Old Block

## Identifying Species-Specific Amino Acid Variations

In [ ]:
# Set the observation threshold for significance
threshold = 3

# Step 1: Identify positions with amino acid variability
# This creates a mask indicating rows belonging to positions with more than one type of amino acid
variability_mask = comp_exp_df.groupby(['GNC', 'Position'])['amino_acid'].transform('nunique') > 1

# Apply the mask to get a DataFrame with only the variable positions
variable_positions_df = comp_exp_df[variability_mask]

# Step 2: Filter based on total observations
# Now, filter these variable positions to keep only those with significant observations
significant_variations_df = variable_positions_df[variable_positions_df['total_observations'] >= threshold]

# # Additional filtering: Keep only rows where both n_term_mod and ptm are NA
# filtered_df = significant_variations_df[pd.isna(significant_variations_df['n_term_mod']) & pd.isna(significant_variations_df['ptm'])]
#sorted_df = significant_variations_df.sort_values(by='total_observations', ascending=False)

# sorted_grouped_df = significant_variations_df.sort_values(
#     by=['GNC', 'Position', 'total_observations'],
#     ascending=[True, True, False]
# )
# The resulting DataFrame (filtered_df) includes rows where there is significant amino acid variability
# and both n_term_mod and ptm are NA.


# First, create a temporary column that ranks total_observations within each group
significant_variations_df['rank'] = significant_variations_df.groupby(['GNC', 'Position'])['total_observations'].rank("dense", ascending=False)

# Then, sort the DataFrame by 'sample_name', 'GNC', 'Position', and 'rank' to ensure that
# the highest total_observations entry comes first, but all entries for the same position are grouped
sorted_grouped_df = significant_variations_df.sort_values(by=['GNC', 'Position', 'rank'])

#print(sorted_grouped_df.head(10))

In [ ]:
# Assuming sorted_grouped_df is your DataFrame and it includes the sample columns listed above
# Column names for the samples of interest
sample_columns = [
    '02_OBi_Pak_P', '03_OBO_Pak_P', '05_ICB_Pak_P', '06_ICB_Pak_FG',
    '09_ICB_Pak_FG', '10_ICB_Pak_Fiber', '12_ICib_Pak_P', '13_ICib_Pak_W',
    '14_OL_Pak_FG', '16_OL_Pak_FG', 'Cz1', 'Cz10', 'Cz11_CFrag_W', 'Cz12',
    'Cz13', 'Cz14', 'Cz15', 'Cz16', 'Cz17', 'Cz17_CFrag_W', 'Cz18', 'Cz19',
    'Cz21', 'Cz22', 'Cz3', 'Cz4', 'Cz5', 'Cz5b_CFrag_W', 'Cz6', 'Cz7',
    'Cz8', 'Cz9', 'INJECTION_BLANK', 'M1', 'M2', 'M3', 'M4', 'MACHINE_BSA',
    'P1', 'P2', 'P3', 'P4'
]

# Sum the values for each of the sample columns
summed_values = sorted_grouped_df[sample_columns].sum()

# Plot a bar graph of the summed values
plt.figure(figsize=(14, 8))
summed_values.plot(kind='bar')
plt.title('Summed Values for Each Sample Column')
plt.xlabel('Sample Columns')
plt.ylabel('Summed Values')
plt.xticks(rotation=90)  # Rotate labels to make them readable
plt.show()

In [ ]:
# Filter columns where the summed total is at least 1000
filtered_columns = summed_values[summed_values >= 10000].index.tolist()
filtered_df = sorted_grouped_df[filtered_columns]


In [ ]:
# Filter the original DataFrame for rows where GNC is COL1A1, COL1A2, or COL3A1
filtered_gnc_df = sorted_grouped_df[sorted_grouped_df['GNC'].isin(['COL1A1', 'COL1A2', 'COL3A1'])]

# Since we're focusing on columns (samples) as observations for PCA, we sum across rows (now filtered)
# to see if any further columns (samples) should be excluded based on the new subset
summed_values_filtered = filtered_gnc_df[filtered_columns].sum()

# Exclude columns with summed totals of less than 1000, as before
filtered_columns_after_gnc_filter = summed_values_filtered[summed_values_filtered >= 1000].index.tolist()
transposed_filtered_df = filtered_gnc_df[filtered_columns_after_gnc_filter].T


In [ ]:
# # Assuming summed_values already reflects the sum of original columns
# # Filter columns based on the summed totals criterion
# filtered_columns = summed_values[summed_values >= 1000].index.tolist()
# filtered_df = sorted_grouped_df[filtered_columns]

# # Transpose the DataFrame to treat columns as observations
# transposed_df = filtered_df.T


###Scoring Identity

Steps to Develop the Scoring Algorithm:
Score Calculation Based on Presence/Absence: For each species, calculate a preliminary score for each sample based on the presence (True) of specific amino acid positions associated with that species. This step identifies whether or not a particular amino acid position, indicative of a species, is found within a sample.

Normalization Using Overall Frequency: Utilize the column that aggregates the frequency of observed positions across all samples to normalize these preliminary scores. The normalization accounts for the differential detection efficiencies of peptides, ensuring that the scoring algorithm is not biased by the mass spectrometer's propensity to identify certain parts of the protein more readily than others.

Normalization Method: Divide the preliminary score for each species by the total observed frequency of each position to generate a normalized score. This adjustment ensures that positions that are heavily represented (due to better detection or genuine abundance) do not disproportionately influence the overall score.

Python Code for the Scoring Algorithm:
Assuming your DataFrame df has the following structure:

GNC and position columns identifying each amino acid position.
Boolean columns for species matches for each sample.
A total_observed column indicating the total number of times each amino acid position was observed across all samples.

This function calculates normalized scores for each sample by initially computing preliminary scores based on the presence of matches. These scores are then ln normalized using the total_observed column to account for variations in detection efficiency. The resulting normalized_scores_df DataFrame provides a comparative view of how well each sample matches the target species, adjusted for the observed frequency of amino acid positions.

In [ ]:
def generate_normalized_scoring(df, sample_columns, species_columns):
    """
    Generate normalized scores for each sample based on the presence of species-specific
    GNC-amino acid matches, normalized by the overall frequency of position observations.

    :param df: DataFrame with sample data, Boolean match columns, and total observed frequency.
    :param sample_columns: List of column names for the samples.
    :param species_columns: List of column names for the Boolean match columns for species.
    :return: A DataFrame with normalized scores for each sample.
    """
    # Initialize a DataFrame to store normalized scores
    normalized_scores_df = pd.DataFrame(index=sample_columns)

    # Calculate preliminary scores based on presence/absence of matches
    preliminary_scores = df[species_columns].multiply(df['total_observed'], axis=0)

    # Sum the preliminary scores for each sample
    for sample in sample_columns:
        normalized_scores_df.loc[sample, 'Normalized Score'] = preliminary_scores[sample].sum() / ln(df['total_observed'].sum())

    return normalized_scores_df.transpose()

# Example usage
sample_columns = ['Sample1', 'Sample2', 'Sample3']  # Actual sample column names
species_columns = ['species_COL1A1_match', 'species_COL1A2_match']  # Actual species Boolean column names
normalized_scores_df = generate_normalized_scoring(df, sample_columns, species_columns)


###PCA

In [ ]:
# Standardizing the transposed data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(transposed_filtered_df)

# Initializing PCA
pca = PCA(n_components=2)  # Reduce the data to 2 dimensions for visualization

# Fitting PCA on the dataset
pca_result = pca.fit_transform(scaled_data)

# # Plotting the PCA result for the transposed data
# plt.figure(figsize=(8, 6))
# plt.scatter(pca_result[:, 0], pca_result[:, 1])
# for i, txt in enumerate(transposed_df.index):
#     plt.annotate(txt, (pca_result[i, 0], pca_result[i, 1]))
# plt.title('PCA Result (Columns as Observations)')
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.grid(True)
# plt.show()

# # Explained variance ratio
# print("Explained variance ratio:", pca.explained_variance_ratio_)


In [ ]:
# Convert PCA results into a DataFrame for easier plotting
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df['sample'] = transposed_filtered_df.index

# Generate an interactive scatter plot with Plotly
fig = px.scatter(pca_df, x='PC1', y='PC2', color='sample', text='sample',
                 title="PCA Results (Interactive)",
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'})
fig.update_traces(marker=dict(size=15, opacity=0.5), textposition='top center')

fig.show()


In [ ]:
# Since we've filtered and then transposed the DataFrame for PCA,
# the correct 'labels' for loadings should correspond to the filtered rows before transposition
# Assuming 'sorted_grouped_df' is the original DataFrame before filtering by 'GNC'
filtered_rows = sorted_grouped_df[sorted_grouped_df['GNC'].isin(['COL1A1', 'COL1A2', 'COL3A1'])]

# Generate labels for these filtered rows, which are now features in the transposed PCA
labels_filtered = filtered_rows.apply(lambda row: f"{row['GNC']}_{row['Position']}_{row['amino_acid']}", axis=1)

# Create the loadings DataFrame using the filtered labels as the index
loadings_filtered = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=labels_filtered)


In [ ]:
# Create the loadings plot with hover info
fig_loadings_filtered = go.Figure()

# Add scatter plot points
fig_loadings_filtered.add_trace(go.Scatter(
    x=loadings_filtered['PC1'],
    y=loadings_filtered['PC2'],
    mode='markers',  # Display only markers, text appears on hover #  mode='markers+text' will nclude text for hover, markers for points
    marker=dict(size=12, color='RoyalBlue',opacity=0.4),  # Adjust marker appearance
    text=loadings_filtered.index,  # The text to display on hover
    hoverinfo='text+x+y'  # Custom hover text, showing the label and coordinates
))

# Update layout for readability
fig_loadings_filtered.update_layout(
    title="PCA Loadings for Filtered Data",
    xaxis_title="Principal Component 1 Loadings",
    yaxis_title="Principal Component 2 Loadings",
    title_x=0.5,  # Center the plot title
    hovermode='closest'  # Show hover info for the closest point
)

# Show the figure
fig_loadings_filtered.show()


In [ ]:
# Sort loadings by the magnitude of their contribution to the first principal component
sorted_loadings = loadings_filtered.abs().sort_values(by='PC1', ascending=False)

# Display the sorted loadings table
print(sorted_loadings)

In [ ]:
# =============================Reporting df_pt_exp ================================

# #Check the df again!
#sorted_grouped_df.head()
#sorted_grouped_df.head(20)
sorted_grouped_df.shape
sorted_grouped_df.info()
# #sorted_grouped_df.describe()
#sorted_grouped_df['ptm'].nunique()
#print(sorted_grouped_df.head(50))

In [ ]:
import pandas as pd
from prettytable import PrettyTable

def to_pretty_table(df):
  """
  Converts a dataframe to a PrettyTable instance, combining GNC, Position and amino acid as row indices and samples as columns.

  Args:
      df (pandas.DataFrame): The dataframe to convert.

  Returns:
      prettytable.PrettyTable: The PrettyTable representation of the dataframe.
  """
  table = PrettyTable()
  # Combine GNC, Position and amino_acid into new index
  df.index = df[["amino_acid"]].apply(lambda x: '-'.join(x.astype(str)), axis=1)
  # Set table fieldnames using new index
  table.fieldnames = df.index.to_list()
  # Include all columns starting from the sample data (from index 5 onwards)
  table.add_rows(df.iloc[:, 5:].values.tolist())
  return table

# Create a copy of the dataframe to avoid modifying the original
#df = significant_aa_variations_df.copy()
df = significant_variations_df.copy()



# Set sample names as column names (from index 5 onwards)
df.columns = df.columns[0:5] + list(df.iloc[0, 5:])

# Create a PrettyTable from the dataframe
table = to_pretty_table(df)

# Print the PrettyTable
print(table)




In [ ]:
# Assuming 'df' is your DataFrame and 'selected_gnc' is the GNC you're interested in
selected_gnc = 'COL1A1'  # Replace 'ExampleGNC' with your actual GNC of interest

# Filtering the DataFrame for the selected GNC using .query()
#gnc_df = significant_aa_variations_df.query("GNC == @selected_gnc")
gnc_df = significant_variations_df.query("GNC == @selected_gnc")
# Create a pivot table with positions as columns, samples as rows, and sum of observations as values
pivot_df = gnc_df.melt(id_vars=['GNC', 'Position'], value_vars=significant_variations_df.columns[5:47],
                                          var_name='Sample', value_name='Observations')

# Summing up observations for each position across all amino acids (if needed)
pivot_df = pivot_df.groupby(['Sample', 'Position'])['Observations'].sum().reset_index()

heatmap_data = pivot_df.pivot("Sample", "Position", "Observations").fillna(0)

fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Viridis'))

fig.update_layout(
    title='Amino Acid Variability Across Sequence Positions and Samples',
    xaxis_nticks=36,
    xaxis_title="Position",
    yaxis_title="Sample")

fig.show()



In [ ]:
# Define the function to create a plot for a selected GNC
def plot_amino_acid_variability_for_gnc(df, gnc, amino_acids_colors):
    # Filter the DataFrame for the selected GNC
    gnc_df = df[df['GNC'] == gnc]

    # Initialize the figure
    fig = go.Figure()

    # Iterate over each sample column
    for col in df.columns[5:47]:  # Adjust index as needed to match sample columns
        # Extract data for the current sample
        sample_df = gnc_df[gnc_df[col] > 0]  # Assuming a positive value indicates observation

        # Add a scatter trace for the current sample
        fig.add_trace(go.Scatter(
            x=sample_df['Position'],
            y=sample_df['amino_acid'],
            mode='markers',
            marker=dict(
                size=sample_df[col],  # Use the sample column value for marker size
                color=[amino_acids_colors.get(aa, '#000000') for aa in sample_df['amino_acid']],  # Map colors
                sizemode='area',
                sizeref=2.*max(df[col])/(40.**2),  # Adjust size reference for marker size scaling
                sizemin=4
            ),
            name=col  # Use the column name as the trace name
        ))

    # Update layout with titles and adjust axes
    fig.update_layout(
        title=f'Amino Acid Variability for {gnc}',
        xaxis_title='Position',
        yaxis_title='Amino Acid',
        yaxis=dict(categoryorder='category ascending')  # Ensure categorical y-axis is ordered correctly
    )

    # Show the figure
    fig.show()

# Example usage
plot_amino_acid_variability_for_gnc(significant_variations_df, 'COL1A2', amino_acids_colors)


In [ ]:


# Manually select a GNC (protein)
selected_gnc = 'COL1A2'

# Filter the DataFrame for the selected GNC
df_filtered = significant_variations_df[significant_variations_df['GNC'] == selected_gnc]

fig = go.Figure()

# Iterate over each sample in the filtered DataFrame
sample_groups = df_filtered['Sample_Group'].unique()
for sample in sample_groups:
    sample_df = df_filtered[df_filtered['Sample_Group'] == sample]
    for amino_acid in sample_df['amino_acid'].unique():
        aa_df = sample_df[sample_df['amino_acid'] == amino_acid]

        # Add scatter trace for each amino acid in each sample
        fig.add_trace(go.Scatter(
            x=aa_df['Position'],
            y=aa_df['amino_acid'],  # Y-axis could be numerical if you map amino acids to numbers
            mode='markers',
            marker=dict(
                size=aa_df['total_observations'],
                color=amino_acids_colors[amino_acid],  # Use your amino_acid_colors dictionary
                sizemode='area',
                sizeref=2.*max(df_filtered['total_observations'])/(40.**2),
                sizemin=4
            ),
            name=f"{sample} - {amino_acid}"
        ))

# Customize layout
fig.update_layout(
    title=f"Amino Acid Variability for {selected_gnc}",
    xaxis_title="Sequence Position",
    yaxis_title="Amino Acid",
    legend_title="Sample - Amino Acid",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# Show figure
fig.show()


In [ ]:
def create_fig_by_sample(df, gnc_selection, sample_selection):
    # Filter DataFrame based on GNC and sample selection
    df_filtered = df[(df['GNC'].isin(gnc_selection)) & (df['Sample_Group'] == sample_selection)]

    fig = go.Figure()

    for gnc in gnc_selection:
        for amino_acid in df_filtered['amino_acid'].unique():
            # Further filter DataFrame for each amino acid within the GNC
            aa_df = df_filtered[(df_filtered['amino_acid'] == amino_acid) & (df_filtered['GNC'] == gnc)]

            # Skip if empty
            if aa_df.empty:
                continue

            fig.add_trace(go.Scatter(
                x=aa_df['Position'],
                y=aa_df['amino_acid'],
                mode='markers',
                marker=dict(
                    size=aa_df['total_observations'],
                    color=amino_acids_colors.get(amino_acid, '#000'),  # Fallback color if not found
                    sizemode='area',
                    sizeref=2.*max(aa_df['total_observations'])/(40.**2),
                    sizemin=4
                ),
                name=f"{gnc} - {amino_acid}"
            ))

    fig.update_layout(
        title=f"Amino Acid Variability for Selected Samples",
        xaxis_title="Sequence Position",
        yaxis_title="Amino Acid",
        legend_title="GNC - Amino Acid"
    )

    return fig

In [ ]:
# Create a scatter plot for each GNC with dropdown to select
def create_fig_for_gnc(df, gnc):
    gnc_df = df[df['GNC'] == gnc]

    # Creating the figure
    fig = go.Figure()

    # Adding traces for each amino acid within the selected GNC
    for amino_acid in gnc_df['amino_acid'].unique():
        aa_df = gnc_df[gnc_df['amino_acid'] == amino_acid]

        # Add scatter trace for each amino acid
        fig.add_trace(go.Scatter(
            x=aa_df['Position'],
            y=aa_df['amino_acid'],  # Assuming the Y-axis is categorical with Sample_Group
            mode='markers',
            marker=dict(
                size=aa_df['total_observations'],  # Adjust size based on occurrence frequency
                color=amino_acids_colors[amino_acid],  # Color by amino acid
                sizemode='area',  # Ensure marker size scales by area
                sizeref=2.*max(aa_df['total_observations'])/(40.**2),  # Adjusts the scaling of the marker sizes
                sizemin=4  # Minimum size of markers
            ),
            name=amino_acid
        ))

    # Customizing the layout
    fig.update_layout(
        title=f"Amino Acid Variability for {gnc}",
        xaxis_title="Sequence Position",
        yaxis_title="Amino acid",
        legend_title="Amino Acid"
    )

    return fig

# Generate figures for all GNCs and prepare dropdown options
gnc_options = significant_variations_df['GNC'].unique()
figs = {gnc: create_fig_for_gnc(significant_variations_df, gnc) for gnc in gnc_options}

# Example to show figure for the first GNC
# To integrate a dropdown for user selection, additional UI handling is required outside of this snippet
first_gnc = list(figs.keys())[0]
figs[first_gnc].show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np  # Ensure NumPy is imported

# DataFrame containing PTM percentages
df = ptm_percentages  # This is your actual DataFrame variable name

selected_gncs = ["COL1A1", "COL1A2", "COL3A1", "TRYUP"]
selected_ptms = ["Oxidation[P]", "Deamidated[Q]", "Carbamidomethyl[C]", "Acetyl[K]"]

# Extract sample columns (considering the structure of the provided df)
sample_columns = [col for col in df.columns if 'percent' in col]

fig = make_subplots(rows=1, cols=len(selected_gncs), subplot_titles=selected_gncs)

for i, gnc in enumerate(selected_gncs, start=1):
    for ptm in selected_ptms:
        # Filter data for the current GNC and PTM
        df_filtered = df[(df['GNC'] == gnc) & (df['ptm'] == ptm)]

        if not df_filtered.empty:
            for sample_col in sample_columns:
                count_col = sample_col.replace('_percent', '')  # Assume count column has the same name minus '_percent'

                fig.add_trace(
                    go.Scatter(
                        x=df_filtered[sample_col],  # Use percentage data for the x-axis
                        y=[ptm] * len(df_filtered),  # Maintain the PTM label on the y-axis for clarity
                        mode='markers',
                        marker=dict(
                            size=df_filtered[count_col].replace(np.nan, 0) + 1,  # Adjust size based on count, replacing NaN with 0
                            sizemode='area',
                            sizeref=2.*max(df_filtered[count_col].replace(np.nan, 0) + 1)/(40.**2),
                            sizemin=4,
                            color=[ptm_colors.get(ptm.split('[')[0], '#333') for _ in df_filtered[sample_col]],  # Map colors
                        ),
                        name=f'{ptm} - {sample_col}'
                    ), row=1, col=i
                )

fig.update_layout(
    height=600,
    width=1200,
    title_text="PTM Percentages by Protein and Sample",
    showlegend=True
)
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assumed correct DataFrame name
df = ptm_percentages  # Replace with your actual DataFrame variable name containing PTM percentages

selected_gncs = ["COL1A1", "COL1A2", "COL3A1", "TRYUP"]
selected_ptms = ["Oxidation[P]", "Deamidated[Q]", "Carbamidomethyl[C]", "Acetyl[K]"]  # Example PTMs

# Extract sample columns (considering the structure of the provided df)
sample_columns = [col for col in df.columns if 'percent' in col]

fig = make_subplots(rows=1, cols=len(selected_gncs), subplot_titles=selected_gncs)

for i, gnc in enumerate(selected_gncs, start=1):
    for ptm in selected_ptms:
        # Filter data for the current GNC and PTM
        df_filtered = df[(df['GNC'] == gnc) & (df['ptm'] == ptm)]

        for sample_col in sample_columns:
            # Extracting the sample name from column names like '02_OBi_Pak_P_percent'
            sample_name = sample_col.replace('_percent', '')

            # Plotting
            if not df_filtered.empty:
                fig.add_trace(
                    go.Scatter(
                        x=[sample_name] * len(df_filtered),
                        y=df_filtered[sample_col],
                        mode='markers',
                        marker=dict(size=df_filtered[sample_col].replace(np.nan, 0) + 1,  # Adjust size, replace NaN with 0
                                    sizemode='area',
                                    sizeref=2.*max(df_filtered[sample_col].replace(np.nan, 0) + 1)/(40.**2),
                                    sizemin=4),
                        name=f'{ptm}',
                    ), row=1, col=i
                )

fig.update_layout(height=600, width=1200, title_text="PTM Percentages by Protein and Sample")
fig.update_layout(showlegend=True)
fig.show()


In [ ]:
# Aggregate the PTM percentages by GNC, PTM type, and Position
# Group by 'GNC', 'ptm', and 'Position', then sum up all percentages for each group
aggregated_ptm_percentages = ptm_percentages.groupby(['GNC', 'ptm', 'Position'])[sample_percent_columns].sum().reset_index()


In [ ]:
#Count PTMs

# Step 1: Count occurrences of each PTM for each GNC
ptm_counts = df_pt_exp.groupby(['GNC', 'ptm']).size().reset_index(name='PTM Count')

# Step 2: Calculate total amino acids for each GNC
total_amino_acids_per_gnc = df_pt_exp.groupby('GNC').size().reset_index(name='Total Amino Acids')

# Step 3: Merge PTM counts with total amino acids
# This adds the total amino acids count for each GNC to the PTM counts DataFrame
ptm_summary = pd.merge(ptm_counts, total_amino_acids_per_gnc, on='GNC')

# Now, ptm_summary DataFrame contains the desired information
# Columns are 'GNC', 'ptm', 'PTM Count', and 'Total Amino Acids'

print(ptm_summary.head())

# If you need to filter or arrange the DataFrame further, you can do so using standard pandas operations.


In [ ]:
# Step 1: Define the sample columns (assuming they are the second to the forty-third columns in your DataFrame)
sample_columns = df_pt_exp.columns[1:43].tolist()

# Step 2: Group by 'GNC' and 'ptm', and then aggregate by summing up the counts from each sample column
grouped_data = df_pt_exp.groupby(['GNC', 'ptm'])[sample_columns].sum().reset_index()

# Now, grouped_data contains the summed counts of amino acids in each sample for each GNC and ptm combination

# Step 3: Count occurrences of each PTM for each GNC to get the PTM Count
ptm_counts = df_pt_exp.groupby(['GNC', 'ptm']).size().reset_index(name='PTM Count')

# Merge the PTM counts with the grouped data to include the PTM counts
final_df = pd.merge(grouped_data, ptm_counts, on=['GNC', 'ptm'], how='left')

# The final_df now includes GNC, ptm, summed counts for each sample column, and the PTM Count

print(final_df.head())

# You can adjust the columns or format of final_df as needed for your analysis.


In [ ]:
# Define the sample columns
sample_columns = df_pt_exp.columns[2:44].tolist()

# Group by 'GNC' to sum amino acids for each sample (assuming 'GNC' is the protein/group identifier)
total_aa_by_sample_gnc = df_pt_exp.groupby('GNC')[sample_columns].sum().reset_index()

# Calculate PTM percentages for each sample
# This requires summing PTM occurrences for each sample in `grouped_data`, calculated previously
grouped_data['Total AA Count'] = grouped_data.apply(lambda row: total_aa_by_sample_gnc[total_aa_by_sample_gnc['GNC'] == row['GNC']][sample_columns].sum(axis=1).values[0], axis=1)

# Convert sample columns to numeric, ensuring no operation is attempted on non-numeric types
for col in sample_columns:
    grouped_data[col] = pd.to_numeric(grouped_data[col], errors='coerce')

# You might also need to ensure 'Total AA Count' is numeric if it's derived from these columns
grouped_data['Total AA Count'] = pd.to_numeric(grouped_data['Total AA Count'], errors='coerce')


# Calculate percentages for each PTM in each sample column
for sample in sample_columns:
    grouped_data[f'{sample} %'] = (grouped_data[sample] / grouped_data['Total AA Count']) * 100

# `grouped_data` now contains percentages for each PTM by sample




In [ ]:
print(grouped_data.head())

In [ ]:
grouped_data.head(30)

In [ ]:
# # Assuming 'grouped_data' and 'total_aa_by_sample_gnc' are already defined as per the previous context

# # Correctly calculate the 'Total AA Count' for each 'GNC' and sample column combination
# # This involves ensuring 'total_aa_by_sample_gnc' is also numeric and correctly referenced

# # # For simplicity and clarity, let's manually calculate totals and percentages for a single sample column first
# # sample_column = '02_OBi_Pak_P'  # Example sample column

# # Calculate the total amino acids for this sample across all GNCs
# total_aa_by_sample = df_pt_exp.groupby('GNC')[sample_column].sum().reset_index()

# # Ensure the total counts are numeric
# total_aa_by_sample[sample_column] = pd.to_numeric(total_aa_by_sample[sample_column], errors='coerce')

# # Before the merge, ensure GNC columns in both DataFrames are of the same data type (string)
# grouped_data['GNC'] = grouped_data['GNC'].astype(str)
# total_aa_by_sample['GNC'] = total_aa_by_sample['GNC'].astype(str)


# # Merge this total back into 'grouped_data' for percentage calculation
# grouped_data = pd.merge(grouped_data, total_aa_by_sample, on='GNC', suffixes=('', '_total'))

# # Now calculate the percentage for this sample column
# grouped_data[f'{sample_column} %'] = (grouped_data[sample_column] / grouped_data[f'{sample_column}_total']) * 100

# # To extend this to all sample columns, you would loop through 'sample_columns' and repeat the merge and percentage calculation


In [ ]:
# # Loop through each sample column to calculate percentages
# for sample_column in sample_columns:
#     # Ensure the total counts are numeric for current sample column
#     total_aa_by_sample[sample_column] = pd.to_numeric(total_aa_by_sample[sample_column], errors='coerce')

#     # Merge the total counts for the current sample column
#     # Make sure to perform type conversion for 'GNC' before merge if not already done
#     grouped_data = pd.merge(grouped_data, total_aa_by_sample[['GNC', sample_column]], on='GNC', suffixes=('', '_total'))

#     # Calculate the percentage for the current sample column
#     grouped_data[f'{sample_column} %'] = (grouped_data[sample_column] / grouped_data[f'{sample_column}_total']) * 100

#     # Drop the now unnecessary '_total' column to prevent it from interfering in future iterations
#     grouped_data.drop(columns=f'{sample_column}_total', inplace=True)


In [ ]:
print(grouped_data.head())

In [ ]:
selected_gncs = ["COL1A1", "COL1A2", "COL3A1", "TRYUP"]

#  '3-deoxyglucosone[R]' , '4-ONE+Delta_H(-2' , 'Acetyl_13C(2' , 'Acetyl[K]' , 'Acetyl[R]' , 'Acetyl[S](Ser->Glu[S]' ,
#   'Acetyl[T]' , 'AEC-MAEC_2H(4' , 'AEC-MAEC[S]' , 'AHA-Alkyne[M]' , 'Ala->Asn[A]' , 'Ala->Asp[A]' , 'Ala->Cys[A]' ,
#   'Ala->Glu[A]' , 'Ala->Met[A]' , 'Ala->Pro[A]' , 'Ala->Ser[A]' , 'Ala->Thr[A]' , 'Ala->Xle[A]' , 'Ammonia-loss[N]' ,
#  'Ammonium[E]' , 'Arg->Asn[R]' , 'Arg->Asp[R]' , 'Arg->Glu[R]' , 'Arg->GluSA[R]' , 'Arg->Met[R]' , 'Arg->Orn[R]' ,
#  'Arg->Phe[R]' , 'Arg->Thr[R]' , 'Arg->Trp[R]' , 'Arg2PG[R]' , 'Asn->Met[N]' , 'Asn->Phe[N]' , 'Bacillosamine[N]' ,
#  'BEMAD_ST_2H(6' , 'Benzoyl[K]' , 'Carbamidomethyl[E]' , 'Carbamidomethyl[K](Gly[K]' , 'Carbamidomethyl[M]' , 'Carbamyl[K]' ,
#  'Carbamyl[S]' , 'Carbamyl[T]' , 'Carbonyl[A]' , 'Carbonyl[E]' , 'Carbonyl[I]' , 'Carbonyl[Q]' , 'Carbonyl[V]' , 'Carboxy[D]' ,
#  'Carboxy[E]' , 'Carboxy[K]' , 'Carboxymethyl[K]' , 'Cation_Al[III][E]' , 'Cation_Fe[II][D]' , 'Cation_Mg[II][E]' , 'Chlorination[Y]' ,
#   'Cresylphosphate[T]' , 'Cyano[C]' , 'DAET[T]' , 'Deamidated[N]' , 'Deamidated[Q]' , 'Deamidated[R]' , 'Dehydrated[D]' , 'Dehydrated[S]' ,
#   'Dehydrated[T]' , 'Delta_H(2' , 'Delta_H(3' , 'Delta_H(4' , 'Deoxy[T]' , 'Dethiomethyl[M]' , 'dHex(1' , 'dHex[S]' , 'Didehydro[S]' ,
#   'Didehydro[T]' , 'Diethylphosphothione[T]' , 'Dimethyl[K](Ethyl[K]/Delta_H(4 1' , 'Dimethyl[K](Ethyl[K]/Delta_H(4 3' ,
#   'Dimethylphosphothione[S]' , 'Dioxidation[E]' , 'Dioxidation[I]' , 'Dioxidation[K]' , 'Dioxidation[M]' , 'Dioxidation[P](Pro->Glu[P]' ,
#  'Dioxidation[V]' , 'Ethanedithiol[S]' , 'Ethanedithiol[T]' , 'Ethanolamine[D]' , 'Ethanolyl[K]' , 'ethylamino[S]' ,
#  'Fluoro[Y]' , 'Formyl[K]' , 'Formyl[S](Ser->Asp[S]' , 'G-H1[R]' , 'Galactosyl[K]' , 'GEE[Q]' , 'GG[C](Dicarbamidomethyl[C]' ,
#   'GIST-Quat_2H(9' , 'Gln->Asp[Q]' , 'Gln->Gly[Q]' , 'Gln->His[Q]' , 'Gln->Met[Q]' , 'Gln->Thr[Q]' , 'Glu->Ala[E]' ,
#   'Glu->Cys[E]' , 'Glu->Gln[E]' , 'Glu->Met[E]' , 'Glu->Phe[E]' , 'Glu->Thr[E]' , 'Glu->Val[E]' , 'Glu[E]' , 'Glucuronyl[S]' ,
#  'GluGlu[E]' , 'Gly->Ala[G]' , 'Gly->Asn[G]' , 'Gly->Asp[G]' , 'Gly->Cys[G]' , 'Gly->Glu[G]' , 'Gly->His[G]' , 'Gly->Lys[G]' ,
#   'Gly->Met[G]' , 'Gly->Phe[G]' , 'Gly->Ser[G]' , 'Gly->Trp[G]' , 'Hep[S]' , 'Hex(1' , 'Hex(2' , 'Hex[S]' , 'HexN[N]' ,
#   'HexNAc(1' , 'HexNAc(2' , 'HexNAc[S]' , 'His->Asp[H]' , 'His->Glu[H]' , 'His->Met[H]' , 'Homocysteic_acid[M]' ,
#   'Hydroxamic_acid[E]' , 'Lipoyl[K]' , 'Lys->AminoadipicAcid[K]' , 'Lys->Arg[K]' , 'Lys->Asp[K]' , 'Lys->Gln[K]' ,
#  'Lys->Glu[K]' , 'Lys->Phe[K]' , 'Lys->Trp[K]' , 'maleimide[C]' , 'maleimide[K]' , 'Malonyl[K]' , 'Malonyl[S]' ,
#   'MeMePhosphorothioate[S]' , 'MercaptoEthanol[S]' , 'MercaptoEthanol[T]' , 'MesitylOxide[H]' , 'Met->Aha[M]' ,
#  'Met->Asn[M]' , 'Met->AspSA[M]' , 'Met->Glu[M]' , 'Met->Ser[M]' , 'Methyl_2H(3' , 'Methyl[D](Asp->Glu[D]' ,
#  'Methyl[K]' , 'Methyl[T]' , 'Methylmalonylation[S]' , 'methylol[K]' , 'Methylphosphonate[T]' , 'methylsulfonylethyl[K]' ,
#   'MG-H1[R](Delta_H(2' , 'monomethylphosphothione[S]' , 'NeuAc[S]' , 'Nitrosyl[C]' , 'O-Isopropylmethylphosphonate[S] 1' ,
#   'O-Methylphosphate[T]' , 'Oxidation[D]' , 'Oxidation[E]' , 'Oxidation[I]' , 'Oxidation[K]' , 'Oxidation[M]' ,
#  'Oxidation[P]' , 'Oxidation[Q]' , 'Pent(2' , 'Pentose[S]' , 'Phe->CamCys[F]' , 'Phe->Cys[F]' , 'Phe->Met[F]' ,
#   'Phe->Thr[F]' , 'Phospho[S]' , 'Phospho[T]' , 'PhosphoHexNAc[S]' , 'Pro->Ala[P]' , 'Pro->Arg[P]' , 'Pro->Asn[P]' ,
#   'Pro->Asp[P]' , 'Pro->Cys[P]' , 'Pro->Gln[P]' , 'Pro->HAVA[P]' , 'Pro->His[P]' , 'Pro->Phe[P]' , 'Pro->pyro-Glu[P]' ,
#   'Pro->Pyrrolidinone[P]' , 'Pro->Ser[P]' , 'Pro->Thr[P]' , 'Pro->Tyr[P]' , 'Pro->Xle[P]' , 's-GlcNAc[T]' , 'Ser->Asn[S]' ,
#  'Ser->Cys[S]' , 'Ser->Gln[S]' , 'Ser->Gly[S]' , 'Ser->Tyr[S]' , 'Succinyl[K]' , 'Sulfide[D]' , 'Sulfo[S]' , 'Thiazolidine[R]' ,
#   'thioacylPA[K]' , 'Thr->Ala[T]' , 'Thr->Asp[T]' , 'Thr->Pro[T]' , 'Thr->Ser[T]' , 'trifluoro[L]' , 'Trioxidation[F]' ,
#   'Tyr->Phe[Y]' , 'Val->Ala[V]' , 'Val->Asn[V]' , 'Val->Asp[V]' , 'Val->Gln[V]' , 'Val->Glu[V]' , 'Val->Ser[V]' , 'Val->Thr[V]' ,
#   'Val->Tyr[V]' , 'Val->Xle[V]' , 'Xle->Asn[L]' , 'Xle->Gln[I]' , 'Xle->Gln[L]' , 'Xle->Glu[I]' , 'Xle->Met[L]' ,
#   'Xle->Phe[I]' , 'Xle->Phe[L]' , 'Xle->Pro[I]' , 'Xle->Ser[L]' , 'Xlink_BS2G[96][K]' , 'Xlink_DFDNB[N]' , 'Xlink_DST[114][K]' ,
#   'Xlink_DST[132][K]' , 'Xlink_DTBP[87][K]' , 'Xlink_DTSSP[88][K]' , 'Xlink_EGS[226][K]'

selected_ptms = [
    "Oxidation[P]",
    "Deamidated[Q]",
    "Oxidation[M]",
    "Deamidated[N]",
    "Dioxidation[P](Pro->Glu[P]",
    "Oxidation[K]",
    "Pro->Asn[P]",
    "Pro->Asp[P]"
]


In [ ]:
# Normalize PTM names by stripping whitespace (if necessary)
final_df['ptm'] = final_df['ptm'].str.strip()

# Filter for selected GNCs and PTMs
df_ptm_filtered_gnc = final_df[final_df['GNC'].isin(selected_gncs)]
df_ptm_filtered = df_ptm_filtered_gnc[df_ptm_filtered_gnc['ptm'].isin(selected_ptms)]

# df_ptm_filtered now contains only the rows with the selected GNCs and PTMs

In [ ]:
df_ptm_filtered.info()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define amino acids and samples for dropdown
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
samples = df_ptm_filtered.columns[2:44]  # Excludes 'GNC', 'ptm', and 'PTM Count'

# Create a subplot figure with one row and len(selected_gncs) columns
fig = make_subplots(rows=1, cols=len(selected_gncs), subplot_titles=selected_gncs)

# Adding traces for each PTM in each GNC
for i, gnc in enumerate(selected_gncs, start=1):
    for ptm in selected_ptms:
        df_filtered = df_ptm_filtered[(df_ptm_filtered['GNC'] == gnc) & (df_ptm_filtered['ptm'] == ptm)]
        if not df_filtered.empty:
            # Assuming an equal distribution of PTM counts across amino acids for demonstration
            # Replace with your specific logic for associating counts to amino acids
            ptm_counts = df_filtered['PTM Count'].values[0] / len(amino_acids)  # Simplified assumption
            fig.add_trace(
                go.Bar(x=amino_acids, y=[ptm_counts]*len(amino_acids), name=ptm),
                row=1, col=i
            )

# Update layout for aesthetics
fig.update_layout(height=600, width=1200, title_text="PTM Counts per Amino Acid by Protein")
fig.update_layout(showlegend=True, legend_title_text='PTM')

# Dropdown menus for sample selection
dropdown_buttons = []

for sample in samples:
    dropdown_buttons.append(
        dict(
            method='restyle',
            args=['y', [df_ptm_filtered[sample].values.tolist()]*len(selected_ptms)],
            label=sample
        )
    )

# Note: The dropdown implementation here is a simplified placeholder. Adjust the 'args' to match the actual logic needed for your dataset.

fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ]
)

fig.show()


In [ ]:
def report_ptm_frequencies(df, gnc):
  """
  Reports relative ptm frequencies for a given protein (GNC) in a table format for each sample.

  Args:
      df: The DataFrame containing protein data.
      gnc: The protein name (GNC) to report on.
  """

  # Filter data for the specified GNC
  filtered_df = df[df['GNC'] == gnc]

  # Get sample names from column names (assuming columns 1-42 represent samples)
  sample_names = list(filtered_df)[1:43]  # Select sample columns

  # Group by sample columns
  grouped_by_sample = filtered_df.groupby(list(filtered_df)[1:43])


  def create_ptm_table(group):
    # Get ptm counts (using the extract_ptm_type function if necessary)
    if 'ptm' in group.columns:  # Check if 'ptm' column exists
      ptm_counts = group['ptm'].apply(extract_ptm_type).value_counts().to_frame(name='count')
    else:
      # Handle case where 'ptm' column is missing (optional)
      return pd.DataFrame({'ptm': ['No ptm column found'] * (len(group.columns) - 1), 'count': [0] * (len(group.columns) - 1)}, columns=['ptm', 'count'])

    # Check if there are any ptms (avoid division by zero)
    if ptm_counts.empty:
      # Create an empty PrettyTable with sample names as headers
      table = PrettyTable(["ptm (amino acid)"] + list(group)[1:43])
      table.add_row(["No ptms detected"] * (len(group.columns) - 1))  # Add a row indicating no ptms
      return table

    # Calculate total ptm counts for percentage calculation
    total_ptms = group['ptm'].count()  # Assuming all rows have ptm (if 'ptm' column exists)

    # Create a PrettyTable instance with header row (ptm)
    table = PrettyTable(["ptm"] + list(group)[1:43])

    # Calculate percentage ptm counts for each sample
    percentage_counts = (ptm_counts['count'] / total_ptms) * 100  # Percentage

    # Add rows with percentage ptm counts
    for index, row in ptm_counts.iterrows():
      table_row = [index] + percentage_counts.tolist()  # Combine ptm and percentages
      table.add_row(table_row)

    return table

  # Apply table creation function to each sample group
  sample_tables = grouped_by_sample.apply(create_ptm_table)

  # Print report for each sample
  for sample_name, table in sample_tables.items():
    print(f"Sample: {sample_name}")
    print(table)
    print("---")  # Separator between samples

# Assuming your DataFrame is named 'df' and you want to report for GNC='COL1A1'
report_ptm_frequencies(df_pt_exp.copy(), 'COL1A1')


In [ ]:
df_pt_exp.info()

In [ ]:
df_pt_exp.info()

##Explore conserved positions

In [ ]:
total_counts = df_pt_exp.groupby(['GNC', 'Position']).size().reset_index(name='total_counts')

# Group by GNC, Position, and amino_acid, then count and find the maximum count for each group
most_common_counts = df_pt_exp.groupby(['GNC', 'Position', 'amino_acid']).size().reset_index(name='counts')
most_common_counts = most_common_counts.sort_values(['GNC', 'Position', 'counts'], ascending=[True, True, False])
most_common_counts = most_common_counts.drop_duplicates(subset=['GNC', 'Position'], keep='first')

conservation_df = pd.merge(most_common_counts, total_counts, on=['GNC', 'Position'])

# Calculate the conservation percentage
conservation_threshold = 95  # This is your variable threshold
conservation_df['conservation_percentage'] = (conservation_df['counts'] / conservation_df['total_counts']) * 100

# Filter based on the threshold
conserved_positions = conservation_df[conservation_df['conservation_percentage'] >= conservation_threshold]
conservation_report = conserved_positions.groupby('GNC').size().reset_index(name='number_of_conserved_positions')

# Sort the report for better visualization, if desired
conservation_report = conservation_report.sort_values(by='number_of_conserved_positions', ascending=False)

# Display the report
conservation_report

##Explore modification patterns

In [ ]:
# Create a DataFrame focusing on modifications
modifications_df = df_pt_exp[df_pt_exp['ptm'].notna() | df_pt_exp['n_term_mod'].notna()]

# Combine N-terminal and other PTMs into a single column for simplicity, if they are usually exclusive
modifications_df['combined_ptm'] = modifications_df['n_term_mod'].fillna('') + modifications_df['ptm'].fillna('')

# Group by GNC, Position, amino_acid, and the combined modification, then count occurrences
mod_count = modifications_df.groupby(['GNC', 'Position', 'amino_acid', 'combined_ptm']).size().reset_index(name='mod_count')

# To find the total occurrences of each amino acid at each position within each GNC, including unmodified instances
total_aa_count = df_pt_exp.groupby(['GNC', 'Position', 'amino_acid']).size().reset_index(name='total_count')

# Merge the modification counts with the total counts to calculate percentages
mod_analysis_df = pd.merge(mod_count, total_aa_count, on=['GNC', 'Position', 'amino_acid'])

# Calculate the percentage of instances with a modification
mod_analysis_df['modification_percentage'] = (mod_analysis_df['mod_count'] / mod_analysis_df['total_count']) * 100

# Now you can filter or sort this DataFrame to focus on specific amino acids, positions, or GNCs

In [ ]:
import plotly.graph_objects as go

# Get unique GNC values for dropdowns
gnc_values = pivot_df.index.get_level_values('GNC').unique()

# Initialize figure with one heatmap (the first GNC by default)
first_gnc = gnc_values[0]
filtered_df = pivot_df.xs(first_gnc, level='GNC').sort_values(by='seq_index')  # Sort initially
fig = go.Figure(data=go.Heatmap(
        z=filtered_df.values,
        x=filtered_df.columns,  # Sample names
        y=filtered_df.index.get_level_values('seq_index'),  # Sequences
        colorscale='gray',
        reversescale=True )) # Reverse color scale

# Create dropdowns with sorting
dropdown_buttons = [
    {'label': gnc,
     'method': 'update',
     'args': [{'z': [pivot_df.xs(gnc, level='GNC').sort_values(by='seq_index').values],  # Sort within dropdown
               'x': [pivot_df.xs(gnc, level='GNC').columns],
               'y': [pivot_df.xs(gnc, level='GNC').sort_values(by='seq_index').index.get_level_values('seq_index')]}]}  # Sort within dropdown
    for gnc in gnc_values]

# Update figure layout with dropdown
fig.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
        }
    ]
)

# Update axes labels and figure title
fig.update_layout(
    title="Peptide Sequence Heatmap by GNC (Sorted by seq_index)",
    xaxis_title="Sample Name",
    yaxis_title="Sequence (Sorted by seq_index)",
    xaxis={'type': 'category'},  # Update if your sample names are better as categories
)

fig.show()


In [ ]:
df2.info()

In [ ]:
#df.shape
df2.info()

In [ ]:
scaler = StandardScaler()
df2_scaled = pd.DataFrame(scaler.fit_transform(df2), columns=df2.columns, index=df2.index)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# Example with the Elbow Method
inertia = []
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(df2_scaled)
    inertia.append(kmeans.inertia_)

In [ ]:

# Plot the inertia to find the elbow
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(2, 11), inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

# Example with Silhouette Score
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    cluster_labels = kmeans.fit_predict(df2_scaled)
    silhouette_avg = silhouette_score(df2_scaled, cluster_labels)
    print(f'For n_clusters = {i}, the silhouette score is {silhouette_avg}.')

In [ ]:
# Assuming the optimal number of clusters is determined to be k
k = 3  # Example value; replace with the number chosen based on the methods above
kmeans = KMeans(n_clusters=k, random_state=42)
df2['Cluster'] = kmeans.fit_predict(df2_scaled)

# Now, df2 has an additional column 'Cluster' indicating the cluster each sample belongs to.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
df2_pca = pca.fit_transform(df2_scaled)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df2_pca[:, 0], df2_pca[:, 1], c=df2['Cluster'])
plt.title('PCA - Cluster Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
 df2_pca_df = pd.DataFrame(df2_pca, columns=['PC1', 'PC2'])
df2_pca_df['Cluster'] = df2['Cluster']  # Assuming this column exists in df2

# Plotting
fig = px.scatter(df2_pca_df, x='PC1', y='PC2', color='Cluster',
                 title="PCA - Cluster Visualization",
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'},
                 color_continuous_scale=px.colors.qualitative.Set1)

# Improving the layout
fig.update_layout(legend_title_text='Cluster')
fig.show()

In [ ]:
df2_pca_df = pd.DataFrame(df2_pca, columns=['PC1', 'PC2'])
df2_pca_df['Cluster'] = df2['Cluster']  # Assuming this column exists in df2

# Plotting
fig = px.scatter(df2_pca_df, x='PC1', y='PC2', color='Cluster',
                 title="PCA - Cluster Visualization",
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'},
                 color_continuous_scale=px.colors.qualitative.Set1)

# Improving the layout
fig.update_layout(legend_title_text='Cluster')
fig.show()# Assuming 'combined_df' is your DataFrame
# Create a copy to work with to avoid SettingWithCopyWarning
filtered_combined_df = combined_df[combined_df['GNC'].isin(['COL1A1', 'COL1A2', 'COL3A1', 'TRYUP', 'K2C1'])].copy()

# Handle potential float and NaN values in 'start_position'
filtered_combined_df['start_position'] = filtered_combined_df['start_position'].fillna(0).astype(int)

# Create a unique identifier for each peptide combining 'GNC' and 'start_position'
filtered_combined_df['peptide_id'] = filtered_combined_df['GNC'] + "_" + filtered_combined_df['start_position'].astype(str)

# Group by peptide_id and aggregate the required information
aggregated_details = filtered_combined_df.groupby('peptide_id').agg({
    'Sequence': 'first',  # Get the first sequence for each peptide_id
    'GNC': 'first',  # Get the GNC (redundant but for clarity)
    'start_position': 'first',  # Get the starting position (already part of peptide_id but for clarity)
    'sample_name': lambda x: list(x.unique())  # Aggregate sample names into a list, ensuring uniqueness
}).reset_index(drop=True)

# Sort by 'GNC' and 'start_position' for readability
aggregated_details_sorted = aggregated_details.sort_values(by=['GNC', 'start_position'])

# Show the top of the sorted DataFrame
print(aggregated_details_sorted.head())


In [ ]:
print (combined_df)

In [ ]:
# Filter the DataFrame for rows where 'GNC&spp' or 'start_position' is missing
missing_annotations_df = combined_df[combined_df['GNC&spp'].isna() | combined_df['start_position'].isna()]

# Print the top 100 entries for the 'Proteins' column from these rows
#print(missing_annotations_df['Proteins'].head(400))

# Extract all protein IDs, split by '/', from the 'Proteins' column
unmatched_sequence_list = missing_annotations_df['Sequence'].str.split('/').explode()

# Find the 100 most common protein strings
top_400_peptides = unmatched_sequence_list.value_counts().head(400)


top_400_peptides.to_csv('top_400_proteins.txt', header=False, index=True)


# Get protein names and their counts
peptide_counts = unmatched_sequence_list.value_counts()

# Ensure at least one protein per count (using nlargest)
one_sequence_per_count = peptide_counts.nlargest(len(peptide_counts.unique()))

# Combine protein name and count into a DataFrame
peptide_result_df = pd.DataFrame({'Sequence': one_sequence_per_count.index, 'count': one_sequence_per_count.values})

# # Print the results (protein ID and count)
print(peptide_result_df)


In [ ]:
print (missing_annotations_df)

In [ ]:
# Get one protein ID for each unique count
unique_counts = top_400_proteins.sort_values(ascending=False)
# Get the first occurrence for each unique count using idxmin
one_protein_per_count = top_400_proteins.groupby(top_400_proteins.values).agg(lambda x: x.iloc[0])

# Print the results (protein ID and count)
top_400_proteins

# Print the results (protein ID and count)
#print(one_protein_per_count(50))

In [ ]:
# # Get one protein ID for each unique count
# unique_counts = top_400_proteins.sort_values(ascending=False)
# # Get the first occurrence for each unique count using idxmin
# one_protein_per_count = top_400_proteins.groupby(top_400_proteins.values).agg(lambda x: x.iloc[0])

# # Print the results (protein ID and count)
# top_400_proteins

# # Print the results (protein ID and count)
# #print(one_protein_per_count(50))

In [ ]:
# Print the results (protein ID and count)
#print(one_protein_per_count)

In [ ]:
#top_400_proteins

###   Direct Matching for Collagen-Motif Peptides

In [ ]:
# peptides_to_examine = [
#     'IITHPNFNGNTLDNDIMLIK',
#     'LGEHNIDVLEGNEQFINAAK',
#     'YSQGNVSAVGVTYDGHTALTR',
#     'TPPAGVFYQGWSATPIANGSLGHDIHHPR',
#     'GPPGPMGPPGIAGPPGESGR',
#     'SDLEMQYETLQEELMALKK'
# ]


# print("Peptides from sequence_position_dict:")
# for peptide in peptides_to_examine:
#     if peptide in sequence_position_dict:
#         details = sequence_position_dict[peptide]
#         # Check if 'gene' and 'position' keys exist before accessing
#         gene = details.get('gene', 'Gene information not available')
#         position = details.get('position', 'Position information not available')
#         print(f"Peptide: {peptide}, Gene: {gene}, Position: {position}")
#     else:
#         print(f"Peptide: {peptide} not found in sequence_position_dict.")


In [ ]:
# Define the directory for saving the output file
output_directory = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/sample1/'
os.makedirs(output_directory, exist_ok=True)

# Get the current timestamp
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with study names and timestamp
filename = f"{output_directory}{'_'.join(STUDY_NAMES)}_{now}_sequence_positions_better.json"

# Assuming 'combined_df' is to be saved, convert it to a dictionary
sequence_position_dict = combined_df.to_dict()
with open(filename, 'w') as json_file:
    json.dump(sequence_position_dict, json_file, indent=4)

print(f'Dictionary saved to JSON file: {filename}')

### Step 4: Identify Unmatched Peptides

After attempting direct matching, identify peptides that remain unmatched:

In [ ]:
#combined_df

In [ ]:
#Reporting
combined_df.head()
#Check the df again!
combined_df.head()
combined_df.shape
combined_df.info()
#combined_df.describe()
combined_df['Sequence'].nunique()

In [ ]:
# #To view the matches
# print("Collagen Matches:")
# for peptide, details in sequence_position_dict.items():
#     print(f"Peptide: {peptide}, Gene: {details['gene']}, Position: {details['position']}")
# print("\nNCP Matches:")
# for peptide, details in NCP_peptide_positions_dict.items():
#     print(f"Peptide: {peptide}, Gene: {details['gene']}, Position: {details['position']}")

In [ ]:
# # Report on the most frequent proteins - this is very SLOW!!!
# report_df = extract_protein_frequencies_and_associations(combined_df, 'Proteins')

# # Display or export `report_df` as needed
# print(report_df)

In [ ]:
# #Writing this report.csv
# # Construct the file path first
# csv_file_path = f'{BASE_PATH}/{STUDY_NAME}_prot_freq.csv'

# # Export the DataFrame to CSV using the correct method name
# report_df.to_csv(csv_file_path)

In [ ]:
# ##Check the df again!
# #combined_df.head()
# #combined_df.shape
# #combined_df.info()
# ##combined_df.describe()
# #combined_df['Sequence'].nunique()
# #combined_df['Miss.Clv.Sites'].nunique()
# combined_df.head()
# list(combined_df.columns)
# #print(combined_df[['GNC&spp', 'Sequence','Proteins', 'Positions', 'start_position']].head(40))
# spp_GNC_counts = combined_df['GNC&spp'].value_counts()

# # Print the counts of different entries
# print(spp_GNC_counts)

In [ ]:
#Generate smaller df - iltered_combined_df- for the Expand step
# Replace 'None' and empty strings with np.nan for uniformity
combined_df['GNC&spp'] = combined_df['GNC&spp'].replace({None: np.nan, '': np.nan})

# Filter rows where 'GNC&spp' is not NaN
filtered_combined_df = combined_df.dropna(subset=['GNC&spp'])

# Now, filtered_combined_df contains only the rows where 'GNC&spp' is not None, NaN, or blank
# You can perform your operations on filtered_combined_df

# Print the head of the filtered DataFrame to verify the filtering
print(filtered_combined_df.head())


In [ ]:
#COUNT GNC
unique_names_GNC = filtered_combined_df['GNC&spp'].unique()
name_counts_GNC = filtered_combined_df['GNC&spp'].value_counts()
print("Unique GNC:", unique_names_GNC)
print("Counts of each GNC:", name_counts_GNC)

## Expand df

In [ ]:
# # Assuming 'combined_df' is original dataframe with columns [''GNC&spp'', 'start_position', 'peptide_length', 'sequence']
# df_sorted = combined_df.sort_values(by=['GNC&spp','start_position', 'peptide_length','Sequence'], ascending=[True, False, False,True])

In [ ]:
# Expand combined_df to detail each peptide's amino acid positions and associated PTMs
#exp_df = expand_df_to(filtered_combined_df) only contains collagen
exp_df = expand_df_to(filtered_combined_df)

In [ ]:
# Display the first few rows of the expanded DataFrame to verify the result
print(exp_df.head())
# list(exp_df.columns)
# exp_df.shape

In [ ]:
unique_names_GNC = exp_df['GNC'].unique()
name_counts_GNC = exp_df['GNC'].value_counts()
print("Unique sample names:", unique_names_GNC)
print("Counts of each sample name:", name_counts_GNC)

In [ ]:
#Rename PTMs based on conditions
exp_df['ptm'] = exp_df.apply(
    lambda x: 'Hydroxylation' if x['aa'] == 'P' and x['ptm'] == 'Oxidation' else
             'DeamidationN' if x['aa'] == 'N' and x['ptm'] == 'Deamidated' else
             'DeamidationQ' if x['aa'] == 'Q' and x['ptm'] == 'Deamidated' else
             x['ptm'],  # Leave other PTMs unchanged
    axis=1
)

In [ ]:
unique_names = exp_df['sample_name'].unique()
name_counts = exp_df['sample_name'].value_counts()
print("Unique sample names:", unique_names)
print("Counts of each sample name:", name_counts)

In [ ]:
# Get the value counts of ptm entries
ptm_counts = exp_df['ptm'].value_counts()

# Print the ptm entries and their counts
print(ptm_counts.head(20))

# Calculate ptm_counts without aggregating sample_name
ptm_counts = exp_df.groupby(['sample_name','GNC', 'position', 'aa', 'ptm']).size().reset_index(name='ptm_count')

# Calculate total_counts without aggregating sample_name
total_counts = exp_df.groupby(['sample_name','GNC', 'position', 'aa']).size().reset_index(name='total_count')

# Merge ptm_counts with the original exp_df to preserve sample_name and score
df_merged = pd.merge(exp_df, ptm_counts, on=['sample_name','GNC', 'position', 'aa', 'ptm'], how='left')
df_merged = pd.merge(df_merged, total_counts, on=['sample_name','GNC', 'position', 'aa'], how='left')

# Calculate the percentage of aa in each position having a specific ptm
df_merged['ptm_percentage'] = (df_merged['ptm_count'] / df_merged['total_count']) * 100

# Filter to include only rows where ptm is not None or empty (i.e., ptm is present)
result_with_ptms = df_merged[df_merged['ptm'].notna() & (df_merged['ptm'] != '')]

# Display the filtered result with ptm details included
print(result_with_ptms[['GNC', 'sample_name', 'position', 'aa', 'ptm', 'ptm_percentage', 'sample_name', 'score']].head())


In [ ]:
#COUNT GNC
unique_names_GNC = exp_df['GNC'].unique()
name_counts_GNC = exp_df['GNC'].value_counts()
print("Unique GNC:", unique_names_GNC)
print("Counts of each GNC:", name_counts_GNC)

###Explore PTMs

###Reshaping exp_df

In [ ]:
def create_grouped_df(exp_df):
    """
    Create a grouped dataframe from the input dataframe `exp_df`.

    Args:
        exp_df (pandas.DataFrame): The input dataframe containing protein identification data.

    Returns:
        pandas.DataFrame: The grouped dataframe containing the requested analysis.
    """
    # Group the data by GNC, sample_name, and position
    grouped = exp_df.groupby(['GNC', 'sample_name', 'position'])

    # Create a list to store the grouped analysis
    grouped_analysis = []

    # Iterate over each group
    for group_keys, group_data in grouped:
        # Unpack the group keys
        gnc, sample_name, position = group_keys

        # Get the top 3 amino acids and their counts
        aa_counts = group_data['aa'].value_counts().head(3)
        top_aas = aa_counts.index.tolist()
        aa_counts = aa_counts.to_dict()

        # Initialize a dictionary to store the analysis for this group
        group_analysis = {
            'GNC': gnc,
            'sample_name': sample_name,
            'position': position
        }

        # Analyze the top 3 amino acids
        for i, aa in enumerate(top_aas, start=1):
            aa_key = f'aa{i}'
            group_analysis[aa_key] = aa  # Add the amino acid name
            n_aa_key = f'n_{aa_key}'
            score_mean_key = f'{aa_key}_score_mean'
            score_max_key = f'{aa_key}_score_max'
            score_sd_key = f'{aa_key}_score_SD'
            score_keys = [f'score{j}_{aa_key}' for j in range(1, 4)]
            spectra_keys = [f'spectraId{j}_{aa_key}' for j in range(1, 4)]

            # Count of the amino acid
            group_analysis[n_aa_key] = aa_counts[aa]

            # Mean, max, and standard deviation of the confidence scores
            aa_scores = group_data.loc[group_data['aa'] == aa, 'confidence']
            group_analysis[score_mean_key] = aa_scores.mean()
            group_analysis[score_max_key] = aa_scores.max()
            group_analysis[score_sd_key] = aa_scores.std()

            # Top 3 scores and spectra IDs
            aa_scores = group_data.loc[group_data['aa'] == aa, ['score', 'spectraId']].nlargest(3, 'score')
            for j, (score_key, spectra_key) in enumerate(zip(score_keys, spectra_keys)):
                if j < len(aa_scores):
                    group_analysis[score_key] = aa_scores.iloc[j]['score']
                    group_analysis[spectra_key] = aa_scores.iloc[j]['spectraId']
                else:
                    group_analysis[score_key] = None
                    group_analysis[spectra_key] = None

            # Analyze the PTMs for this amino acid
            aa_ptms = group_data.loc[group_data['aa'] == aa, 'ptm'].value_counts()
            top_ptms = aa_ptms.index.tolist()[:2]
            aa_ptms = aa_ptms.to_dict()

            for j, ptm in enumerate(top_ptms, start=1):
                ptm_key = f'ptm{j}_{aa_key}'
                ptm_perc_key = f'%{ptm_key}'
                ptm_score_mean_key = f'{ptm_key}_score_mean'
                ptm_score_max_key = f'{ptm_key}_score_max'
                ptm_score_sd_key = f'{ptm_key}_score_SD'
                ptm_score_keys = [f'score{k}_{ptm_key}' for k in range(1, 4)]
                ptm_spectra_keys = [f'spectraId{k}_{ptm_key}' for k in range(1, 4)]


                # PTM name and percentage
                group_analysis[ptm_key] = ptm
                group_analysis[ptm_perc_key] = aa_ptms.get(ptm, 0) / (group_data[(group_data['aa'] == aa) & (group_data['position'] == position)]['aa'].count()) * 100


                # Mean, max, and standard deviation of the confidence scores for the PTM
                ptm_scores = group_data.loc[(group_data['aa'] == aa) & (group_data['ptm'] == ptm), 'confidence']
                group_analysis[ptm_score_mean_key] = ptm_scores.mean()
                group_analysis[ptm_score_max_key] = ptm_scores.max()
                group_analysis[ptm_score_sd_key] = ptm_scores.std()

                # Top 3 scores and spectra IDs for the PTM
                ptm_scores = group_data.loc[(group_data['aa'] == aa) & (group_data['ptm'] == ptm), ['score', 'spectraId']].nlargest(3, 'score')
                for k, (score_key, spectra_key) in enumerate(zip(ptm_score_keys, ptm_spectra_keys)):
                    if k < len(ptm_scores):
                        group_analysis[score_key] = ptm_scores.iloc[k]['score']
                        group_analysis[spectra_key] = ptm_scores.iloc[k]['spectraId']
                    else:
                        group_analysis[score_key] = None
                        group_analysis[spectra_key] = None

        # Append the analysis for this group to the list
        grouped_analysis.append(group_analysis)

    # Create the grouped dataframe from the list of dictionaries
    grouped_df = pd.DataFrame(grouped_analysis)

    return grouped_df

In [ ]:
#-----------------------the SLOW one-----------------
#
group_df = create_grouped_df(exp_df)

In [ ]:
# with list constructor
col_list = list(group_df.columns)
# with tolist method
col_list = group_df.columns.tolist()
print(col_list)

In [ ]:
group_df

In [ ]:
print(group_df)

In [ ]:
PFIND_FOLDER1 = f'pFind_{STUDY_NAME1}'  # Folder path for study 1
directory_path = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME1}/{PFIND_FOLDER1}/result/'

# Define the directory for saving the output file
#directory_path = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/sample1/'
os.makedirs(directory_path , exist_ok=True)
group_df.to_csv(f'{directory_path}pFind_grouped2.csv', index=False)

exp_df.to_csv(f'{directory_path}pFind_exp2.csv', index=False)

In [ ]:
#grouped_CGPT_df = analyze_protein_data(test_df)

In [ ]:
#print(grouped_CGPT_df)
print(group_df)

In [ ]:
#----------------------------------Summary Table of Data

# Calculate counts for 'GNC&spp' across the entire DataFrame
counts = group_df['GNC'].value_counts()

# Filter 'GNC&spp' entries that meet the threshold criterion (e.g., count >= 100)
filtered_entries = counts[counts >= 100].index.tolist()

# Filter the DataFrame to only include those 'GNC&spp' entries
filtered_df = group_df[group_df['GNC'].isin(filtered_entries)]

# Prepare data for PrettyTable
# Initialize PrettyTable with 'File_Name' plus each 'GNC&spp' that meets the threshold as columns
columns = ['sample_name'] + filtered_entries
pt = PrettyTable()
pt.field_names = columns

# Populate PrettyTable rows with counts of 'GNC&spp' for each 'File_Name'
for file_name in filtered_df['sample_name'].unique():
    row_data = [file_name]
    for entry in filtered_entries:
        count = filtered_df[(filtered_df['sample_name'] == file_name) & (filtered_df['GNC'] == entry)].shape[0]
        row_data.append(count)
    pt.add_row(row_data)

# Set table title
pt.title = 'Summary Output'

# Print the PrettyTable
print(pt)


In [ ]:
# GNC to examine (replace with the GNC you want to analyze)
gnc_of_interest = 'COL3A1'

# Filter the DataFrame to only include the GNC of interest
gnc_df = group_df[group_df['GNC'] == gnc_of_interest]

# Print the GNC DataFrame
print(gnc_df)


In [ ]:
column_headers = gnc_df.columns.tolist()
print(column_headers)

In [ ]:
def analyze_protein_variations(df, gnc):
    # Filter for the specific GNC
    df = df[df['GNC'] == gnc]

    # Initialize result containers
    missing_aa_positions = []
    single_aa_positions = []
    multiple_aa_positions = []

    # Group by 'position' to analyze each position
    grouped = df.groupby('position')

    for position, group in grouped:
        # Identify missing amino acids
        if group[['aa1', 'aa2', 'aa3']].isna().all().all():
            missing_aa_positions.append(position)
            continue

        # Identify conserved amino acids (single AA across all samples)
        unique_aas = pd.concat([group['aa1'], group['aa2'], group['aa3']]).dropna().unique()
        if len(unique_aas) == 1:
            aa_count = group['n_aa1'].sum()  # Assuming 'n_aa1' counts occurrences of 'aa1'
            single_aa_positions.append((position, unique_aas[0], aa_count))
            continue

        # Analyze positions with multiple amino acids
        for aa_col in ['aa1', 'aa2', 'aa3']:
            for aa in group[aa_col].dropna().unique():
                aa_count = group[group[aa_col] == aa]['n_aa1'].sum()  # Assuming 'n_aa1' corresponds to 'aa1'
                top_score = group[group[aa_col] == aa][f'{aa_col}_score_max'].min()
                multiple_aa_positions.append((position, aa, aa_count, top_score))

    # Convert to DataFrames for easier handling
    single_aa_df = pd.DataFrame(single_aa_positions, columns=['Position', 'Amino Acid', 'Count'])
    multiple_aa_df = pd.DataFrame(multiple_aa_positions, columns=['Position', 'Amino Acid', 'Count', 'Top Score'])

    return missing_aa_positions, single_aa_df, multiple_aa_df

# Assuming 'df' is your DataFrame and 'COL1A2' is the GNC of interest
missing_aa_positions, single_aa_df, multiple_aa_df = analyze_protein_variations(gnc_df, 'COL3A1')

# Printing tables
# Missing Amino Acid Positions
print("Missing Amino Acid Positions:")
if missing_aa_positions:
    table = PrettyTable(['Position'])
    for position in missing_aa_positions:
        table.add_row([position])
    print(table)
else:
    print("No missing amino acid positions found.")

# Single Amino Acid Positions
print("\nSingle Amino Acid Positions:")
if not single_aa_df.empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count'])
    for row in single_aa_df.itertuples(index=False):
        table.add_row([row.position, row.aa1, row.countt])
    print(table)
else:
    print("No positions with single amino acids found.")


# # Print single amino acid positions
# print("\nSingle Amino Acid Positions:")
# if not protein_variations['Single AA Positions'].empty:
#     table = PrettyTable(['Position', 'Amino Acid', 'Count'])
#     for row in protein_variations['Sin# Single Amino Acid Positions
print("\nSingle Amino Acid Positions:")
if not single_aa_df.empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count'])
    for row in single_aa_df.itertuples(index=False):
        table.add_row([row.position, row.aa1, row.countt])
    print(table)
else:
    print("No positions with single amino acids found.")
gle AA Positions'].itertuples():
#         table.add_row([row.position, row.aa1, row.count])
#     print(table)
# else:
#     print("No positions with single amino acids found.")

# Multiple Amino Acid Positions
print("\nMultiple Amino Acid Positions:")
if not multiple_aa_df.empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
    for row in multiple_aa_df.itertuples(index=False):
        table.add_row([row.Position, row.Amino_Acid, row.Count, row.Top_Score])
    print(table)
else:
    print("No positions with multiple amino acids found.")


In [ ]:
import pandas as pd
from prettytable import PrettyTable

def analyze_protein_variations(gnc_df):
    # Ensure 'position' is not set as an index to avoid ambiguity
    gnc_df.reset_index(drop=True, inplace=True)

    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Analyze positions with missing amino acids
    # Filtering directly, so no ambiguity should arise here
    missing_aa_positions = gnc_df[gnc_df['aa1'].isna()].groupby('position').size().index.tolist()

    # Analyze positions with a single amino acid
    # This approach avoids ambiguity by working with grouped objects directly
    single_aa_summary = gnc_df.dropna(subset=['aa1']).groupby('position').filter(lambda x: x['aa1'].nunique() == 1)
    single_aa_summary = single_aa_summary.groupby(['position', 'aa1']).size().reset_index(name='count')

    # Analyze positions with multiple amino acids
    multiple_aa_positions = gnc_df.dropna(subset=['aa1']).groupby('position').filter(lambda x: x['aa1'].nunique() > 1)
    multiple_aa_analysis = multiple_aa_positions.groupby(['position', 'aa1']).size().reset_index(name='count')

    # Combine results
    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': multiple_aa_analysis
    }

    return summary


# Assuming 'gnc_df' is your DataFrame filtered for a specific GNC of interest
protein_variations = analyze_protein_variations(gnc_df)

# Print missing amino acid positions
print("Missing Amino Acid Positions:")
if protein_variations['Missing AA Positions']:
    table = PrettyTable(['Position'])
    table.add_rows([[position] for position in protein_variations['Missing AA Positions']])
    print(table)
else:
    print("No missing amino acid positions found.")

# Print single amino acid positions
print("\nSingle Amino Acid Positions:")
if not protein_variations['Single AA Positions'].empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count'])
    for row in protein_variations['Single AA Positions'].itertuples():
        table.add_row([row.position, row.aa1, row.count])
    print(table)
else:
    print("No positions with single amino acids found.")

# Print multiple amino acid positions
print("\nMultiple Amino Acid Positions:")
if not protein_variations['Multiple AA Positions'].empty:
    table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
    for row in protein_variations['Multiple AA Positions'].itertuples():
        table.add_row([row.position, row.aa1, row.count, row.top_score])
    print(table)
else:
    print("No positions with multiple amino acids found.")


In [ ]:
import pandas as pd

def analyze_protein_variations(gnc_df):
    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Initialize containers for the analysis results
    missing_aa_positions = []
    single_aa_summary = []
    multiple_aa_analysis = []

    # Analyze each position
    for position, group in grouped_by_position:
        # Identify missing amino acids by checking if all values in 'aa1' are NaN
        if group['aa1'].isna().all():
            missing_aa_positions.append(position)
            continue  # Move to the next position if the current one has no data

        # Analyze positions with a single amino acid
        if group['aa1'].nunique() == 1:
            aa = group['aa1'].dropna().unique()[0]
            count = group['n_aa1'].sum()  # Summing up occurrences across all samples
            single_aa_summary.append({'position': position, 'amino_acid': aa, 'count': count})
        else:
            # Analyze positions with multiple amino acids
            aa_counts = group['aa1'].value_counts().reset_index()
            aa_counts.columns = ['amino_acid', 'count']
            for _, row in aa_counts.iterrows():
                multiple_aa_analysis.append({'position': position, 'amino_acid': row['amino_acid'], 'count': row['count']})

    # Convert analysis results into DataFrame for easy handling
    missing_aa_df = pd.DataFrame(missing_aa_positions, columns=['Position'])
    single_aa_df = pd.DataFrame(single_aa_summary)
    multiple_aa_df = pd.DataFrame(multiple_aa_analysis)

    # Compile the summary into a dictionary
    summary = {
        'Missing AA Positions': missing_aa_df,
        'Single AA Positions': single_aa_df,
        'Multiple AA Positions': multiple_aa_df
    }

    return summary

# Assuming 'gnc_df' is your DataFrame filtered for a specific GNC
# Example usage:
# protein_variations = analyze_protein_variations(gnc_df)
# The 'protein_variations' dictionary now contains three keys with DataFrames as their values.


In [ ]:
# Print missing amino acid positions
print("Missing Amino Acid Positions:")
if protein_variations['Missing AA Positions']:
  table = PrettyTable(['Position'])
  table.add_rows([[position] for position in protein_variations['Missing AA Positions']])
  print(table)
else:
  print("No missing amino acid positions found.")

# Print single amino acid positions
print("\nSingle Amino Acid Positions:")
if protein_variations['Single AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count'])
  table.add_rows(protein_variations['Single AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with single amino acids found.")

# Print multiple amino acid positions
print("\nMultiple Amino Acid Positions:")
if protein_variations['Multiple AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
  table.add_rows(protein_variations['Multiple AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with multiple amino acids found.")

In [ ]:
def analyze_protein_data(df, gnc):
    # Filter the DataFrame for the specified GNC
    df_gnc = group_df[df['GNC'] == gnc]

    # Prepare the output DataFrame structure
    columns = ['position', 'sample_name', 'amino_acid', 'count', 'score1', 'PTM_type', 'PTM_score']
    analysis_results = pd.DataFrame(columns=columns)

    # Iterate through each unique position
    for position in df_gnc['position'].unique():
        df_position = df_gnc[df_gnc['position'] == position]

        # Iterate through each sample_name
        for sample in df_position['sample_name'].unique():
            df_sample = df_position[df_position['sample_name'] == sample]

            # Analyze amino acids and PTMs for this sample and position
            for aa_col in ['aa1', 'aa2', 'aa3']:
                aa_data = df_sample[[aa_col, f'n_{aa_col}', f'score1_{aa_col}']].dropna()

                for _, row in aa_data.iterrows():
                    amino_acid = row[aa_col]
                    count = row[f'n_{aa_col}']
                    score1 = row[f'score1_{aa_col}']

                    # Add PTM analysis here if applicable
                    ptm_types = []  # Placeholder for actual PTM analysis
                    ptm_scores = []  # Placeholder for actual PTM analysis

                    # Append to results DataFrame
                    result_row = pd.DataFrame([[position, sample, amino_acid, count, score1, ptm_types, ptm_scores]], columns=columns)
                    analysis_results = pd.concat([analysis_results, result_row], ignore_index=True)

    return analysis_results

# Assuming 'df' is your DataFrame and 'COL3A1' is the GNC of interest
results_df = analyze_protein_data(group_df, 'COL3A1')


In [ ]:
def analyze_protein_variations(gnc_df):
    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Analyze positions with missing amino acids
    missing_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].isna().any())['position'].tolist()

    # Analyze positions with a single amino acid
    single_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() == 1)
    single_aa_summary = single_aa_positions[['position', 'aa1']].value_counts().reset_index(name='count')

    # Analyze positions with multiple amino acids
    multiple_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() > 1)
    multiple_aa_analysis = []
    if not multiple_aa_positions.empty:  # Check if there are positions with multiple amino acids
        for (position, idx), group_data in multiple_aa_positions.groupby(level=0):
            # Check for any non-missing amino acid column (handle potential MultiIndex)
            amino_acid_cols = ['aa1', 'aa2', 'aa3']  # Assuming amino acid columns
            for aa_col in amino_acid_cols:
                if not group_data[aa_col].isna().all():
                    aa = group_data[aa_col].dropna().iloc[0]  # Get the first non-missing amino acid
                    count = group_data[aa_col].value_counts().get(aa, 0)  # Count occurrences

                    multiple_aa_analysis.append({'position': position, 'aa1': aa, 'count': count})
                    break  # Exit the inner loop after finding a non-missing amino acid column

    # Combine results
    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': multiple_aa_analysis
    }

    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': multiple_aa_analysis
    }
    return summary

In [ ]:
protein_variations = analyze_protein_variations(gnc_df)

In [ ]:
def analyze_protein_variations(gnc_df):
    # Group data by position
    grouped_by_position = gnc_df.groupby('position')

    # Analyze positions with missing amino acids
    missing_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].isna().all())['position'].tolist()

    # Analyze positions with a single amino acid
    single_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() == 1)
    single_aa_summary = single_aa_positions[['position', 'aa1']].value_counts().reset_index(name='count')

    # Analyze positions with multiple amino acids
    multiple_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() > 1)
    multiple_aa_analysis = []
    if not multiple_aa_positions.empty:  # Check if there are positions with multiple amino acids
        for (position, idx), group_data in multiple_aa_positions:
            # Check for any non-missing amino acid column (handle potential MultiIndex)
            amino_acid_cols = ['aa1', 'aa2', 'aa3']  # Assuming amino acid columns
            for aa_col in amino_acid_cols:
                if not group_data[aa_col].isna().all():
                    aa = group_data[aa_col].dropna().iloc[0]  # Get the first non-missing amino acid
                    count = group_data[aa_col].value_counts().get(aa, 0)  # Count occurrences

                    # Access PTM scores dynamically based on the amino acid column
                    ptm_scores = [group_data[f'ptm{i}_{aa_col}'].dropna().iloc[idx] for i in range(1, len(group_data.columns) // 2 + 1) if not pd.isna(group_data[f'ptm{i}_{aa_col}'].iloc[idx])]
                    top_score = max(ptm_scores) if ptm_scores else None

                    multiple_aa_analysis.append({'position': position, 'aa1': aa, 'count': count, 'top_score': top_score})
                    break  # Exit the inner loop after finding a non-missing amino acid column

    # Combine results
    summary = {
        'Missing AA Positions': missing_aa_positions,
        'Single AA Positions': single_aa_summary,
        'Multiple AA Positions': pd.DataFrame(multiple_aa_analysis)
    }
    return summary

# Analyze variations in the protein
protein_variations = analyze_protein_variations(gnc_df.copy())

In [ ]:

# # Function to analyze variations in a protein
# def analyze_protein_variations(gnc_df):
#   # Group data by position
#   grouped_by_position = gnc_df.groupby('position')

#   # Analyze positions with missing amino acids
#   missing_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].isna().any())['position'].tolist()

#   # Analyze positions with a single amino acid
#   single_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() == 1)
#   single_aa_summary = single_aa_positions[['position', 'aa1']].value_counts().reset_index(name='count')

#   # Analyze positions with multiple amino acids
#   multiple_aa_positions = grouped_by_position.filter(lambda x: x['aa1'].nunique() > 1)
#   multiple_aa_analysis = []
#   for position, group_data in multiple_aa_positions:
#     for aa_col in ['aa1', 'aa2', 'aa3']:  # Assuming you have 3 amino acid columns (aa1, aa2, aa3)
#       if not pd.isna(group_data[aa_col].any()):  # Check if there's data in this column
#         aa = group_data[aa_col].iloc[0]  # Assuming the first non-null value is the amino acid
#         count = group_data[aa_col].value_counts().get(aa, 0)  # Count occurrences (default to 0 if not present)
#         ptm_scores = [group_data[f'ptm{i}_score1'].iloc[idx] for i in range(1, len(group_data.columns) // 2 + 1) if not pd.isna(group_data[f'ptm{i}_{aa_col}'].iloc[idx])]
#         top_score = max(ptm_scores) if ptm_scores else None
#         multiple_aa_analysis.append({'position': position, 'aa1': aa, 'count': count, 'top_score': top_score})


#   # Combine results
#   summary = {
#       'Missing AA Positions': missing_aa_positions,
#       'Single AA Positions': single_aa_summary,
#       'Multiple AA Positions': multiple_aa_df
#   }
#   return summary

# # Analyze variations in the protein
# protein_variations = analyze_protein_variations(gnc_df.copy())

# Print the summary tables (assuming PrettyTable is available)
from prettytable import PrettyTable

# Print missing amino acid positions
print("Missing Amino Acid Positions:")
if protein_variations['Missing AA Positions']:
  table = PrettyTable(['Position'])
  table.add_rows([[position] for position in protein_variations['Missing AA Positions']])
  print(table)
else:
  print("No missing amino acid positions found.")

# Print single amino acid positions
print("\nSingle Amino Acid Positions:")
if protein_variations['Single AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count'])
  table.add_rows(protein_variations['Single AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with single amino acids found.")

# Print multiple amino acid positions
print("\nMultiple Amino Acid Positions:")
if protein_variations['Multiple AA Positions'].any():
  table = PrettyTable(['Position', 'Amino Acid', 'Count', 'Top Score'])
  table.add_rows(protein_variations['Multiple AA Positions'].to_records(index=False))
  print(table)
else:
  print("No positions with multiple amino acids found.")


In [ ]:
# # Filter the DataFrame to find rows with ptm "Deamidated"
# filtered_ptm_df = df_filtered[df_filtered['ptm'] == "Deamidated"]

# # Check if any rows were found
# if not filtered_ptm_df.empty:
#   # Print a limited number of rows (e.g., the first 3)
#   print(filtered_ptm_df.head(3))
# else:
#   # If no rows are found, print a message
#   print("No rows found with ptm='Deamidated'")

In [ ]:
# unique_names = result_with_ptms['sample_name'].unique()
# name_counts = result_with_ptms['sample_name'].value_counts()
# print("Unique sample names:", unique_names)
# print("Counts of each sample name:", name_counts)